In [1]:
def collectAllData(outDirectory, year_it,
                  trips, activities, PTs, PtoPTs, LT, PLT, VLT, ST, name):


    storage_client = storage.Client.from_service_account_json('outputs/beam-core-b829a9287f0e.json')
    bucket_name = 'beam-core-outputs'
    bucket = storage_client.get_bucket(bucket_name)

    if is_GEO:
        
        BGs = gpd.read_file('/vsicurl/https://github.com/LBNL-UCB-STI/beam-core-analysis/raw/main/Users/Zach/scenario/sfbay-blockgroups-2010/641aa0d4-ce5b-4a81-9c30-8790c4ab8cfb202047-1-wkkklf.j5ouj.shp')
        
        trips = addGeometryIdToDataFrame(trips, BGs, 'originX', 'originY', 'startBlockGroup')
        trips = addGeometryIdToDataFrame(trips, BGs, 'destinationX', 'destinationY', 'endBlockGroup')
        activities = addGeometryIdToDataFrame(activities, BGs, 'activityLocationX', 'activityLocationY', 'activityBlockGroup')
        
        print('Add block ID infos')
        # block_info = pd.read_csv('local_files/bg_w_geog_labels.csv')
        block_info = pd.read_csv('https://github.com/LBNL-UCB-STI/beam-core-analysis/raw/main/Users/Nazanin/bg_w_geog_labels.csv')
        trips['startBlockGroup'] = trips['startBlockGroup'].fillna(0).astype(int)
        trips['endBlockGroup'] = trips['endBlockGroup'].fillna(0).astype(int)
        activities['activityBlockGroup'] = activities['activityBlockGroup'].fillna(0).astype(int)
        PTs['startBlockGroup'] = PTs['startBlockGroup'].fillna(0).astype(int)
        PTs['endBlockGroup'] = PTs['endBlockGroup'].fillna(0).astype(int)
        PtoPTs['startBlockGroup'] = PtoPTs['startBlockGroup'].fillna(0).astype(int)
        PtoPTs['endBlockGroup'] = PtoPTs['endBlockGroup'].fillna(0).astype(int)
        if is_LT:
            LT['BlockGroup'] = LT['BlockGroup'].fillna(0).astype(int)
            PLT['BlockGroup'] = PLT['BlockGroup'].fillna(0).astype(int)
            VLT['BlockGroup'] = VLT['BlockGroup'].fillna(0).astype(int)
        
        trips = pd.merge(trips, block_info,  how='left',  left_on = 'startBlockGroup',right_on = 'bgid')
        trips = pd.merge(trips, block_info,  how='left',  left_on = 'endBlockGroup',right_on = 'bgid',suffixes = ['_start','_end'])
        activities = pd.merge(activities, block_info,  how='left',  left_on = 'activityBlockGroup',right_on = 'bgid',)
        PTs = pd.merge(PTs, block_info,  how='left',  left_on = 'startBlockGroup',right_on = 'bgid',)
        PTs = pd.merge(PTs, block_info,  how='left',  left_on = 'endBlockGroup',right_on = 'bgid',suffixes = ['_start','_end'])
        PtoPTs = pd.merge(PtoPTs, block_info,  how='left',  left_on = 'startBlockGroup',right_on = 'bgid',)
        PtoPTs = pd.merge(PtoPTs, block_info,  how='left',  left_on = 'endBlockGroup',right_on = 'bgid',suffixes = ['_start','_end'])
        if is_LT:
            LT = pd.merge(LT, block_info,  how='left',  left_on = 'BlockGroup',right_on = 'bgid',)
            PLT = pd.merge(PLT, block_info,  how='left',  left_on = 'BlockGroup',right_on = 'bgid',)
            VLT = pd.merge(VLT, block_info,  how='left',  left_on = 'BlockGroup',right_on = 'bgid',)

    # PTs = addGeometryIdToDataFrame(PTs, BGs, 'startX', 'startY', 'startBlockGroup')
    # PTs = addGeometryIdToDataFrame(PTs, BGs, 'endX', 'endY', 'endBlockGroup')
    PTs.index.set_names('PathTraversalID', inplace=True)
    
    
    print('writing results on', outDirectory+ 'postprocessOutputs/'+year_it)
    
    bucket.blob(outDirectory[23:]+ 'postprocessOutputs/'+year_it+'/trips.csv.gz').upload_from_string(trips.to_csv(index=True), content_type='text/csv')
    bucket.blob(outDirectory[23:]+ 'postprocessOutputs/'+year_it+'/vehicles.csv.gz').upload_from_string(PTs.to_csv(index=True), content_type='text/csv')
    bucket.blob(outDirectory[23:]+ 'postprocessOutputs/'+year_it+'/activities.csv.gz').upload_from_string(activities.to_csv(index=True), content_type='text/csv')
    bucket.blob(outDirectory[23:]+ 'postprocessOutputs/'+year_it+'/personToVehicles.csv.gz').upload_from_string(PtoPTs.to_csv(index=True), content_type='text/csv')
    if is_LT:
        bucket.blob(outDirectory[23:]+ 'postprocessOutputs/'+year_it+'/linkTable.csv.gz').upload_from_string(LT.to_csv(index=True), content_type='text/csv')
        bucket.blob(outDirectory[23:]+ 'postprocessOutputs/'+year_it+'/personLinkTable.csv.gz').upload_from_string(PLT.to_csv(index=True), content_type='text/csv')
        bucket.blob(outDirectory[23:]+ 'postprocessOutputs/'+year_it+'/vehicleLinkTable.csv.gz').upload_from_string(VLT.to_csv(index=True), content_type='text/csv')
    
    bucket.blob(outDirectory[23:]+ 'postprocessOutputs/'+year_it+'/summaryTable.csv.gz').upload_from_string(ST.to_csv(index=True), content_type='text/csv')
    

#     trips.to_csv(outDirectory[23:]+ 'postprocessOutputs/'+year_it+'/trips.csv.gz', index=True)
#     PTs.to_csv(outDirectory[23:]+ 'postprocessOutputs/'+year_it+'/vehicles.csv.gz', index=True)
#     activities.to_csv(outDirectory[23:]+ 'postprocessOutputs/'+year_it+'/activities.csv.gz', index=True)
#     PtoPTs.to_csv(outDirectory[23:]+ 'postprocessOutputs/'+year_it+'/personToVehicles.csv.gz', index=True)
#     if is_LT:
#         LT.to_csv(outDirectory[23:]+ 'postprocessOutputs/'+year_it+'/linkTable.csv.gz', index=True)
#         PLT.to_csv(outDirectory[23:]+ 'postprocessOutputs/'+year_it+'/personLinkTable.csv.gz', index=True)
#         VLT.to_csv(outDirectory[23:]+ 'postprocessOutputs/'+year_it+'/vehicleLinkTable.csv.gz', index=True) 
#     ST.to_csv(outDirectory[23:]+ 'postprocessOutputs/'+year_it+'/summaryTable.csv.gz', index=True)
    
#     if is_plot:
#         plot_and_save_figures(outDirectory+ 'postprocessOutputs/'+year_it+'/figures/', ST, name)
    
    print('File uploaded successfully.')


In [2]:
! pip install geopandas
! pip install pandas
! pip install pygeos
! pip install boto
! pip install s3fs
! pip install shapely
! pip install gcsfs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gzip
import time
from itertools import groupby
import geopandas as gpd
from google.cloud import storage
# import seaborn as sns
# %matplotlib inline
# from io import BytesIO
# from google.cloud import storage

# import os
# client = language.LanguageServiceClient.from_service_account_json("/path/to/file.json")
# storage_client = storage.Client()
# bucket = storage_client.get_bucket('beam-core-outputs')
# path = "gs://beam-core-outputs/"
# df = pd.read_csv(path)

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_2041613/2848130802.py:14: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html)

In [3]:
#Inputs filepath, data_names, names

nrows = None #None for all rows
is_leg_mode = True
is_SMART = True
is_NYC = None


# #######################################################################################################################
# #############NYC##############
if is_NYC:
    fp = "s3://beam-outputs/output/newyork/"
    output_nm = 'SummaryTable_NYC_baseline17.csv'
    len_id_transit = 10
    fp_res = 'outputs/'
    is_NYC = True
    is_WC = False
    is_LT = False
    is_GEO = False
    is_collect_data = False
    is_plot = False
    data = {
        #           'new-york-split_scenario-part-1__2022-06-03_17-21-39_iqs/ITERS/it.0/0.events.csv.gz':'Baseline Part1', 
        #           'new-york-split_scenario-part-2__2022-06-15_14-26-18_cuu/ITERS/it.0/0.events.csv.gz':'Baseline Part2', 
        #           'new-york-split_scenario-part-3__2022-06-15_14-26-18_dmi/ITERS/it.0/0.events.csv.gz':'Baseline Part3',
        #           'new-york-300k-calibration-7__2022-06-25_17-40-03_aui/ITERS/it.10/10.events.csv.gz':'Run7',
        #           'new-york-300k-calibration-11__2022-06-28_21-55-14_hbf/ITERS/it.10/10.events.csv.gz':'Run11',
        #           'new-york-300k-calibration-12__2022-06-28_21-54-34_tav/ITERS/it.10/10.events.csv.gz':'Run12',
        #           'new-york-300k-calibration-13__2022-06-29_17-21-34_psu/ITERS/it.10/10.events.csv.gz':'Run13',
        #           'new-york-300k-calibration-15__2022-06-30_15-57-54_zqu/ITERS/it.10/10.events.csv.gz':'Run15',
        #           'new-york-300k-calibration-16__2022-07-04_14-46-10_wvt/ITERS/it.10/10.events.csv.gz':'Run16',
        #           'new-york-300k-calibration-17__2022-07-05_20-44-02_ccs/ITERS/it.10/10.events.csv.gz':'Run17',
        #           'new-york-450k-calibration-18__2022-07-08_15-39-57_pxy/ITERS/it.10/10.events.csv.gz':'Run18',
        #           'new-york-450k-calibration-19__2022-07-12_17-14-44_esm/ITERS/it.10/10.events.csv.gz':'Run19',
        #           'new-york-450k-calibration-20__2022-07-12_22-31-28_czx/ITERS/it.10/10.events.csv.gz':'Run20',
        #           'new-york-baseline-0-of-10__2022-07-17_01-19-13_fgz/ITERS/it.5/5.events.csv.gz':'Run21-0/10-fullpop',
        #           'new-york-baseline-3-of-10__2022-07-17_01-19-21_soy/ITERS/it.5/5.events.csv.gz':'Run21-3/10-fullpop',
        #           'new-york-baseline-4-of-10__2022-07-17_01-19-20_qig/ITERS/it.5/5.events.csv.gz':'Run21-4/10-fullpop',
        #           'new-york-baseline-5-of-10__2022-07-17_01-19-12_yyt/ITERS/it.5/5.events.csv.gz':'Run21-5/10-fullpop',
        #           'new-york-baseline-6-of-10__2022-07-17_01-19-12_dgi/ITERS/it.5/5.events.csv.gz':'Run21-6/10-fullpop',
        #           'new-york-baseline-7-of-10__2022-07-17_01-19-14_qip/ITERS/it.5/5.events.csv.gz':'Run21-7/10-fullpop',
        #           'new-york-baseline-8-of-10__2022-07-17_01-19-11_oko/ITERS/it.5/5.events.csv.gz':'Run21-8/10-fullpop',
        #           'new-york-baseline-2-of-10__2022-07-19_01-38-47_ryr/ITERS/it.0/0.events.csv.gz':'Run21-2/10-fullpop',
        #           'new-york-baseline-9-of-10__2022-07-19_01-38-46_hfm/ITERS/it.0/0.events.csv.gz':'Run21-9/10-fullpop',
        #           'new-york-baseline-1-of-10__2022-07-19_15-06-02_ewc/ITERS/it.0/0.events.csv.gz':'Run21-1/10-fullpop',
        #           'new-york-300k-may2020-calibration-1__2022-07-23_00-00-16_plu/ITERS/it.10/10.events.csv.gz':'Future-Run1',
        #           'new-york-300k-may2020-calibration-1__2022-07-23_00-02-02_esg/ITERS/it.10/10.events.csv.gz':'Future-Run2',
        #           'new-york-300k-may2020-calibration-1__2022-07-23_00-02-03_gzm/ITERS/it.10/10.events.csv.gz':'Future-Run4',
        #           'new-york-300k-may2020-calibration-5__2022-07-25_19-38-26_fjs/ITERS/it.10/10.events.csv.gz':'Future-Run5',
        #           'new-york-300k-may2020-calibration-6__2022-07-25_19-38-18_wqn/ITERS/it.10/10.events.csv.gz':'Future-Run6',
        #           'new-york-300k-may2020-calibration-7__2022-07-25_19-38-21_gri/ITERS/it.10/10.events.csv.gz':'Future-Run7',
        #           'new-york-300k-may2020-calibration-8__2022-07-25_19-40-44_xde/ITERS/it.10/10.events.csv.gz':'Future-Run8',
        #           'new-york-300k-may2020-calibration-6__2022-08-06_00-15-58_tzw/ITERS/it.10/10.events.csv.gz':'Future-Run6_2',
        #           'new-york-300k-may2020-calibration-9__2022-08-06_00-16-00_fmo/ITERS/it.10/10.events.csv.gz':'Future-Run9',
        #           'new-york-300k-may2020-calibration-10__2022-08-06_00-15-54_zlo/ITERS/it.10/10.events.csv.gz':'Future-Run10',
        #           'new-york-300k-may2020-calibration-12__2022-08-13_10-30-40_cvt/ITERS/it.10/10.events.csv.gz':'may2020-12',
        #           'new-york-300k-may2020-calibration-13__2022-08-13_10-30-40_rkc/ITERS/it.10/10.events.csv.gz':'may2020-13',
        #           'new-york-300k-august2020-calibration-1__2022-08-13_10-57-13_nig/ITERS/it.10/10.events.csv.gz':'aug2020-1',
        #           'new-york-300k-august2020-calibration-2__2022-08-13_10-57-12_cap/ITERS/it.10/10.events.csv.gz':'aug2020-2',
        #           'new-york-300k-jan2022-calibration-1__2022-08-13_10-57-17_xey/ITERS/it.10/10.events.csv.gz':'jan2022-1',
        #           'new-york-300k-jan2022-calibration-2__2022-08-13_10-57-14_zuc/ITERS/it.10/10.events.csv.gz':'jan2022-2',
        #           'new-york-300k-june2022-calibration-1__2022-08-13_10-57-14_osw/ITERS/it.10/10.events.csv.gz':'jun2022-1',
        #           'new-york-300k-june2022-calibration-2__2022-08-13_10-57-14_lqo/ITERS/it.10/10.events.csv.gz':'jun2022-2',
        #           'new-york-300k-august2021-calibration-1__2022-08-13_10-57-15_wpa/ITERS/it.10/10.events.csv.gz':'aug2021-1',
        #           'new-york-300k-august2021-calibration-2__2022-08-13_10-57-12_pds/ITERS/it.10/10.events.csv.gz':'aug2021-2',
        # 'new-york-600k-may2020-calibration-14__2022-08-18_20-20-25_ynh/ITERS/it.10/10.events.csv.gz':'may2020-3',
        # 'new-york-600k-august2020-calibration-3__2022-08-18_20-20-33_qid/ITERS/it.10/10.events.csv.gz':'aug2020-3',
        # 'new-york-600k-august2021-calibration-3__2022-08-18_20-20-22_ylh/ITERS/it.10/10.events.csv.gz':'aug2021-3',
        # 'new-york-600k-jan2022-calibration-3__2022-08-18_20-20-23_eun/ITERS/it.10/10.events.csv.gz':'jan2022-3',
        # 'new-york-600k-june2022-calibration-3__2022-08-18_20-20-33_uov/ITERS/it.10/10.events.csv.gz':'june2022-3'
        #     'new-york-june2022-0-of-10__2022-09-07_04-43-41_eoc/ITERS/it.5/5.events.csv.gz':'june2022-4',
        #     'new-york-august2021-0-of-10__2022-09-07_04-41-33_wmw/ITERS/it.5/5.events.csv.gz':'aug2021-4',
        #     'new-york-august2020-0-of-10__2022-09-07_04-41-27_hgn/ITERS/it.5/5.events.csv.gz':'aug2020-4',
        #     'new-york-may2020-0-of-10__2022-09-07_04-41-31_dku/ITERS/it.5/5.events.csv.gz':'may2020-4',
        # 'new-york-may2020-0-of-10__2022-09-21_17-07-17_aon/ITERS/it.10/10.events.csv.gz':'may2020-5',
        # 'new-york-august2020-0-of-10__2022-09-21_17-07-13_pug/ITERS/it.10/10.events.csv.gz':'august2020-5',
        # 'new-york-august2021-0-of-10__2022-09-21_17-07-09_yan/ITERS/it.10/10.events.csv.gz':'august2021-5',
        # 'new-york-june2022-0-of-10__2022-09-21_17-07-28_ojg/ITERS/it.10/10.events.csv.gz':'june2022-5',
        # 'new-york-jan2022-0-of-10__2022-09-21_17-07-49_qdx/ITERS/it.10/10.events.csv.gz':'jan2022-5',
        # 'new-york-baseline-0-of-10__2022-12-06_04-19-22_kra/ITERS/it.5/5.events.csv.gz':'baseline-GTFS-Capacities-it5',
        # 'new-york-may2020-0-of-10__2022-12-05_20-53-27_plq/ITERS/it.10/10.events.csv.gz':'may2020-GTFS-Capacities-it10',
        # 'new-york-august2020-0-of-10__2022-12-06_00-48-35_czx/ITERS/it.10/10.events.csv.gz':'august2020-GTFS-Capacities-it10',
        # 'new-york-august2021-0-of-10__2022-12-05_06-08-36_lne/ITERS/it.5/5.events.csv.gz':'august2021-GTFS-Capacities-it5',
        # 'new-york-jan2022-0-of-10__2022-12-06_04-19-20_pub/ITERS/it.10/10.events.csv.gz':'jan2022-GTFS-Capacities-it10',
        # 'new-york-june2022-0-of-10__2022-12-08_06-41-28_yhp/ITERS/it.0/0.events.csv.gz':'june2022-GTFS-Capacities-it0'
    # 'new-york-baseline-0-of-10__2022-12-13_06-39-57_mua/ITERS/it.5/5.events.csv.gz':'baseline-GTFS-Capacities4-it5',
    #             # 'new-york-baseline-0-of-10__2022-12-13_06-42-03_uyo/ITERS/it.5/5.events.csv.gz':'baseline-GTFS-Capacities-it5',
    #             # 'new-york-baseline-0-of-10__2022-12-13_06-41-05_kcb/ITERS/it.5/5.events.csv.gz':'baseline-GTFS-Capacities2-it5',
    # 'new-york-baseline-0-of-10__2022-12-13_06-40-14_tqf/ITERS/it.5/5.events.csv.gz':'baseline-GTFS-Capacities3-it5',
    # 'new-york-may2020-0-of-10__2022-12-12_18-04-01_zgo/ITERS/it.10/10.events.csv.gz':'may2020-GTFS-Capacities-it10',
    # # 'new-york-may2020-0-of-10__2022-12-12_18-04-01_zgo/ITERS/it.5/5.events.csv.gz':'may2020-GTFS-Capacities-it5',
    # 'new-york-may2020-0-of-10__2022-12-13_06-41-05_sat/ITERS/it.10/10.events.csv.gz':'may2020-GTFS-Capacities2-it10',
    # # 'new-york-may2020-0-of-10__2022-12-13_06-41-05_sat/ITERS/it.5/5.events.csv.gz':'may2020-GTFS-Capacities2-it5',
    # 'new-york-may2020-0-of-10__2022-12-13_06-40-55_fwv/ITERS/it.10/10.events.csv.gz':'may2020-GTFS-Capacities3-it10',
    # # 'new-york-may2020-0-of-10__2022-12-13_06-40-55_fwv/ITERS/it.5/5.events.csv.gz':'may2020-GTFS-Capacities3-it5',
    # 'new-york-may2020-0-of-10__2022-12-13_06-39-44_ije/ITERS/it.10/10.events.csv.gz':'may2020-GTFS-Capacities4-it10',
    # # 'new-york-may2020-0-of-10__2022-12-13_06-39-44_ije/ITERS/it.5/5.events.csv.gz':'may2020-GTFS-Capacities4-it5',
    # 'new-york-august2020-0-of-10__2022-12-12_18-03-58_xrx/ITERS/it.10/10.events.csv.gz':'august2020-GTFS-Capacities-it10',
    # # 'new-york-august2020-0-of-10__2022-12-12_18-03-58_xrx/ITERS/it.5/5.events.csv.gz':'august2020-GTFS-Capacities-it15',
    # 'new-york-august2020-0-of-10__2022-12-13_06-36-10_qre/ITERS/it.10/10.events.csv.gz':'august2020-GTFS-Capacities2-it10',
    # # 'new-york-august2020-0-of-10__2022-12-13_06-36-10_qre/ITERS/it.5/5.events.csv.gz':'august2020-GTFS-Capacities2-it5',
    # 'new-york-august2020-0-of-10__2022-12-13_06-36-08_mpe/ITERS/it.10/10.events.csv.gz':'august2020-GTFS-Capacities3-it10',
    # # 'new-york-august2020-0-of-10__2022-12-13_06-36-08_mpe/ITERS/it.5/5.events.csv.gz':'august2020-GTFS-Capacities3-it15',
    # 'new-york-august2021-0-of-10__2022-12-12_18-03-59_mbq/ITERS/it.10/10.events.csv.gz':'august2021-GTFS-Capacities-it10',
    # # 'new-york-august2021-0-of-10__2022-12-12_18-03-59_mbq/ITERS/it.5/5.events.csv.gz':'august2021-GTFS-Capacities-it5',
    # 'new-york-august2021-0-of-10__2022-12-13_06-36-08_ldo/ITERS/it.10/10.events.csv.gz':'august2021-GTFS-Capacities2-it10',
    # # 'new-york-august2021-0-of-10__2022-12-13_06-36-08_ldo/ITERS/it.5/5.events.csv.gz':'august2021-GTFS-Capacities2-it5',
    # 'new-york-august2021-0-of-10__2022-12-13_06-36-53_dwe/ITERS/it.10/10.events.csv.gz':'august2021-GTFS-Capacities4-it10',
    # # 'new-york-august2021-0-of-10__2022-12-13_06-36-53_dwe/ITERS/it.5/5.events.csv.gz':'august2021-GTFS-Capacities4-it5',
    # 'new-york-jan2022-0-of-10__2022-12-12_18-04-01_bua/ITERS/it.10/10.events.csv.gz':'jan2022-GTFS-Capacities-it10',
    # # 'new-york-jan2022-0-of-10__2022-12-12_18-04-01_bua/ITERS/it.5/5.events.csv.gz':'jan2022-GTFS-Capacities-it5',
    # 'new-york-jan2022-0-of-10__2022-12-13_06-36-09_kzo/ITERS/it.10/10.events.csv.gz':'jan2022-GTFS-Capacities2-it10',
    # # 'new-york-jan2022-0-of-10__2022-12-13_06-36-09_kzo/ITERS/it.5/5.events.csv.gz':'jan2022-GTFS-Capacities2-it5',
    # 'new-york-jan2022-0-of-10__2022-12-13_06-36-18_udd/ITERS/it.10/10.events.csv.gz':'jan2022-GTFS-Capacities3-it10',
    # # 'new-york-jan2022-0-of-10__2022-12-13_06-36-18_udd/ITERS/it.5/5.events.csv.gz':'jan2022-GTFS-Capacities3-it5',
    # 'new-york-jan2022-0-of-10__2022-12-13_06-36-47_jux/ITERS/it.10/10.events.csv.gz':'jan2022-GTFS-Capacities4-it10',
    # # 'new-york-jan2022-0-of-10__2022-12-13_06-36-47_jux/ITERS/it.5/5.events.csv.gz':'jan2022-GTFS-Capacities4-it15',
        
    # '/ITERS/it.5/5.events.csv.gz':'baseline_5',
    # '/ITERS/it.5/5.events.csv.gz':'baseline_6',
    # 'new-york-may2020-0-of-10__2022-12-18_08-02-54_wwu/ITERS/it.5/5.events.csv.gz':'may2020_5',
    # 'new-york-may2020-0-of-10__2022-12-18_08-02-58_sdt/ITERS/it.5/5.events.csv.gz':'may2020_6',
    # 'new-york-august2020-0-of-10__2022-12-18_07-59-31_hac/ITERS/it.5/5.events.csv.gz':'aug2020_5',
    # 'new-york-august2020-0-of-10__2022-12-18_07-59-36_vkr/ITERS/it.5/5.events.csv.gz':'aug2020_6',
    # 'new-york-august2021-0-of-10__2022-12-18_08-03-04_dwz/ITERS/it.5/5.events.csv.gz':'aug2021_5',
    # 'new-york-august2021-0-of-10__2022-12-18_08-03-07_mib/ITERS/it.5/5.events.csv.gz':'aug2021_6',
    # 'new-york-jan2022-0-of-10__2022-12-18_07-59-32_fzx/ITERS/it.5/5.events.csv.gz':'jan2022_5',
    # 'new-york-jan2022-0-of-10__2022-12-18_07-59-36_dmm/ITERS/it.5/5.events.csv.gz':'jan2022_6',
    # 'new-york-june2022-0-of-10__2022-12-18_07-59-38_qdi/ITERS/it.5/5.events.csv.gz':'jun2022_1',
    # 'new-york-june2022-0-of-10__2022-12-18_07-59-34_zfe/ITERS/it.5/5.events.csv.gz':'jun2022_2',
    # 'new-york-june2022-0-of-10__2022-12-18_07-59-33_qqx/ITERS/it.5/5.events.csv.gz':'jun2022_3',
    # 'new-york-june2022-0-of-10__2022-12-18_07-59-31_muj/ITERS/it.5/5.events.csv.gz':'jun2022_4',
    # 'new-york-may2020-0-of-10__2022-12-18_17-38-08_gfm/ITERS/it.5/5.events.csv.gz':'may2020_5',
    # 'new-york-may2020-0-of-10__2022-12-18_17-38-04_hfs/ITERS/it.5/5.events.csv.gz':'may2020_6',
    # # '/ITERS/it.5/5.events.csv.gz':'aug2020_5',
    # 'new-york-august2020-0-of-10__2022-12-18_17-38-04_phk/ITERS/it.5/5.events.csv.gz':'aug2020_6',
    # 'new-york-august2021-0-of-10__2022-12-18_17-38-01_fuq/ITERS/it.5/5.events.csv.gz':'aug2021_5',
    # 'new-york-august2021-0-of-10__2022-12-18_17-38-11_adj/ITERS/it.5/5.events.csv.gz':'aug2021_6',
    # # '/ITERS/it.5/5.events.csv.gz':'jan2022_5',
    # 'new-york-jan2022-0-of-10__2022-12-18_17-38-05_bel/ITERS/it.5/5.events.csv.gz':'jan2022_6',
    # # '/ITERS/it.5/5.events.csv.gz':'jun2022_1',
    # # '/ITERS/it.5/5.events.csv.gz':'jun2022_2',
    # 'new-york-june2022-0-of-10__2022-12-18_17-38-08_yzn/ITERS/it.5/5.events.csv.gz':'jun2022_3',
    # 'new-york-june2022-0-of-10__2022-12-18_17-38-04_dcx/ITERS/it.5/5.events.csv.gz':'jun2022_4',
    # 'new-york-baseline-0-of-10__2022-12-19_23-42-50_zlc/ITERS/it.5/5.events.csv.gz':'baseline7',
    # 'new-york-baseline-0-of-10__2022-12-19_23-42-51_nvw/ITERS/it.5/5.events.csv.gz':'baseline8',
    # 'new-york-baseline-0-of-10__2022-12-19_23-42-53_uyr/ITERS/it.5/5.events.csv.gz':'baseline9',
    # 'new-york-may2020-0-of-10__2022-12-19_23-42-52_frv/ITERS/it.5/5.events.csv.gz':'may2020_7',
    # 'new-york-may2020-0-of-10__2022-12-19_23-42-55_dbm/ITERS/it.5/5.events.csv.gz':'may2020_8',
    # 'new-york-may2020-0-of-10__2022-12-19_23-42-54_nmv/ITERS/it.5/5.events.csv.gz':'may2020_9',
    # 'new-york-august2020-0-of-10__2022-12-19_23-42-59_wkn/ITERS/it.5/5.events.csv.gz':'aug2020_7',
    # 'new-york-august2020-0-of-10__2022-12-19_23-42-53_bkp/ITERS/it.5/5.events.csv.gz':'aug2020_8',
    # 'new-york-august2020-0-of-10__2022-12-19_23-42-49_zpt/ITERS/it.5/5.events.csv.gz':'aug2020_9',
    # 'new-york-august2021-0-of-10__2022-12-19_23-42-50_xif/ITERS/it.5/5.events.csv.gz':'aug2021_7',
    # 'new-york-august2021-0-of-10__2022-12-19_23-42-52_dsq/ITERS/it.5/5.events.csv.gz':'aug2021_8',
    # 'new-york-august2021-0-of-10__2022-12-19_23-42-52_keu/ITERS/it.5/5.events.csv.gz':'aug2021_9',
    # 'new-york-jan2022-0-of-10__2022-12-19_23-42-51_hxu/ITERS/it.5/5.events.csv.gz':'jan2022_7',
    # 'new-york-jan2022-0-of-10__2022-12-19_23-42-49_pkq/ITERS/it.5/5.events.csv.gz':'jan2022_8',
    # 'new-york-jan2022-0-of-10__2022-12-19_23-42-51_hrx/ITERS/it.5/5.events.csv.gz':'jan2022_9',
    # 'new-york-june2022-0-of-10__2022-12-19_23-42-50_mtd/ITERS/it.5/5.events.csv.gz':'jun2022_7',
    # 'new-york-june2022-0-of-10__2022-12-19_23-42-53_qol/ITERS/it.5/5.events.csv.gz':'jun2022_8',
    # 'new-york-june2022-0-of-10__2022-12-19_23-42-48_caq/ITERS/it.5/5.events.csv.gz':'jun2022_9',
    # 'new-york-august2020-0-of-10__2022-12-21_09-42-56_idw/ITERS/it.5/5.events.csv.gz':'aug2020_10',
    # 'new-york-august2020-0-of-10__2022-12-21_09-42-57_fgn/ITERS/it.5/5.events.csv.gz':'aug2020_11',
    # 'new-york-august2020-0-of-10__2022-12-21_01-14-46_xiy/ITERS/it.5/5.events.csv.gz':'aug2020_12',
    # 'new-york-august2020-0-of-10__2022-12-21_01-42-00_tli/ITERS/it.5/5.events.csv.gz':'aug2020_13',
    # 'new-york-august2020-0-of-10__2022-12-21_09-42-54_txi/ITERS/it.5/5.events.csv.gz':'aug2020_14',
    # 'new-york-august2021-0-of-10__2022-12-21_01-14-45_rsd/ITERS/it.5/5.events.csv.gz':'aug2021_10',
    # 'new-york-august2021-0-of-10__2022-12-21_01-14-44_pct/ITERS/it.5/5.events.csv.gz':'aug2021_11',
    # 'new-york-august2021-0-of-10__2022-12-21_01-14-46_ilu/ITERS/it.5/5.events.csv.gz':'aug2021_12',
    # 'new-york-august2021-0-of-10__2022-12-21_01-42-02_ifx/ITERS/it.5/5.events.csv.gz':'aug2021_13',
    # 'new-york-august2021-0-of-10__2022-12-21_01-40-11_sdu/ITERS/it.5/5.events.csv.gz':'aug2021_14',
    # 'new-york-baseline-0-of-10__2022-12-21_01-14-46_pfw/ITERS/it.5/5.events.csv.gz':'Baseline_10',
    # 'new-york-baseline-0-of-10__2022-12-21_01-14-48_arn/ITERS/it.5/5.events.csv.gz':'Baseline_11',
    # 'new-york-baseline-0-of-10__2022-12-21_01-14-55_zfx/ITERS/it.5/5.events.csv.gz':'Baseline_12',
    # 'new-york-baseline-0-of-10__2022-12-21_01-41-59_aat/ITERS/it.5/5.events.csv.gz':'Baseline_13',
    # 'new-york-baseline-0-of-10__2022-12-21_01-40-07_esl/ITERS/it.5/5.events.csv.gz':'Baseline_14',
    # 'new-york-baseline-0-of-10__2022-12-21_01-40-10_lrg/ITERS/it.5/5.events.csv.gz':'Baseline_15',
    # 'new-york-jan2022-0-of-10__2022-12-21_01-40-16_irv/ITERS/it.5/5.events.csv.gz':'jan2022_11',
    # 'new-york-jan2022-0-of-10__2022-12-21_01-42-02_fvl/ITERS/it.5/5.events.csv.gz':'jan2022_12',
    # 'new-york-jan2022-0-of-10__2022-12-21_01-42-08_gvz/ITERS/it.5/5.events.csv.gz':'jan2022_13',
    # 'new-york-may2020-0-of-10__2022-12-21_01-14-46_cjo/ITERS/it.5/5.events.csv.gz':'may2020_10',
    # 'new-york-may2020-0-of-10__2022-12-21_01-14-44_axx/ITERS/it.5/5.events.csv.gz':'may2020_11',
    # 'new-york-may2020-0-of-10__2022-12-21_01-14-46_ize/ITERS/it.5/5.events.csv.gz':'may2020_12',
    # 'new-york-may2020-0-of-10__2022-12-21_01-14-45_pqr/ITERS/it.5/5.events.csv.gz':'may2020_13',
    # 'new-york-may2020-0-of-10__2022-12-21_01-14-51_nuc/ITERS/it.5/5.events.csv.gz':'may2020_14',
    # 'new-york-jan2022-0-of-10__2022-12-21_01-14-50_gyp/ITERS/it.5/5.events.csv.gz':'jan2022_10',
    # 'new-york-june2022-0-of-10__2022-12-21_01-14-49_tkf/ITERS/it.5/5.events.csv.gz':'jun2022_10',
    # 'new-york-baseline-0-of-10__2022-12-27_22-27-09_unl/ITERS/it.5/5.events.csv.gz':'baseline_16',
    # 'new-york-baseline-0-of-10__2022-12-27_22-28-53_lrk/ITERS/it.5/5.events.csv.gz':'baseline_17',
    # 'new-york-baseline-0-of-10__2022-12-27_22-27-09_mfg/ITERS/it.5/5.events.csv.gz':'baseline_18',
    # 'new-york-may2020-0-of-10__2022-12-27_22-27-12_jkl/ITERS/it.5/5.events.csv.gz':'may2020_15',
    # 'new-york-may2020-0-of-10__2022-12-27_22-27-19_onq/ITERS/it.0/0.events.csv.gz':'may2020_16-it0',
    # 'new-york-may2020-0-of-10__2022-12-27_22-27-12_gxt/ITERS/it.5/5.events.csv.gz':'may2020_17',
    # 'new-york-august2020-0-of-10__2022-12-27_22-27-09_bvj/ITERS/it.0/0.events.csv.gz':'aug2020_15-it0',
    # 'new-york-august2020-0-of-10__2022-12-27_22-27-08_qyq/ITERS/it.5/5.events.csv.gz':'aug2020_16',
    # 'new-york-august2020-0-of-10__2022-12-27_22-27-11_gql/ITERS/it.5/5.events.csv.gz':'aug2020_17',
    # 'new-york-august2021-0-of-10__2022-12-27_22-27-07_cqe/ITERS/it.5/5.events.csv.gz':'aug2021_15',
    # 'new-york-august2021-0-of-10__2022-12-27_22-27-08_drl/ITERS/it.0/0.events.csv.gz':'aug2021_16-it0',
    # 'new-york-august2021-0-of-10__2022-12-27_22-28-50_otm/ITERS/it.5/5.events.csv.gz':'aug2021_17',
    # 'new-york-jan2022-0-of-10__2022-12-27_22-28-53_rxs/ITERS/it.5/5.events.csv.gz':'jan2022_14',
    # 'new-york-jan2022-0-of-10__2022-12-27_22-28-52_cfs/ITERS/it.5/5.events.csv.gz':'jan2022_15',
    # 'new-york-jan2022-0-of-10__2022-12-27_22-27-10_mht/ITERS/it.5/5.events.csv.gz':'jan2022_16',
    # 'new-york-june2022-0-of-10__2022-12-28_03-33-09_hek/ITERS/it.5/5.events.csv.gz':'may2022_11',
    # 'new-york-june2022-0-of-10__2022-12-28_03-33-06_yvi/ITERS/it.5/5.events.csv.gz':'may2022_12',
    # 'new-york-june2022-0-of-10__2022-12-28_03-33-06_psr/ITERS/it.5/5.events.csv.gz':'may2022_13',
    # 'new-york-baseline-0-of-10__2023-01-03_19-59-12_nwn/ITERS/it.5/5.events.csv.gz':'baseline-0',
    # # '/ITERS/it.5/5.events.csv.gz':'baseline-1',
    # 'new-york-baseline-2-of-10__2023-01-03_19-59-06_brm/ITERS/it.5/5.events.csv.gz':'baseline-2',
    # 'new-york-baseline-3-of-10__2023-01-03_19-59-07_kqr/ITERS/it.5/5.events.csv.gz':'baseline-3',
    # 'new-york-baseline-4-of-10__2023-01-03_19-59-09_fbb/ITERS/it.5/5.events.csv.gz':'baseline-4',
    # 'new-york-baseline-5-of-10__2023-01-03_19-59-11_tjh/ITERS/it.5/5.events.csv.gz':'baseline-5',
    # 'new-york-baseline-6-of-10__2023-01-03_19-59-12_zwm/ITERS/it.5/5.events.csv.gz':'baseline-6',
    # 'new-york-baseline-7-of-10__2023-01-03_19-59-19_bcr/ITERS/it.5/5.events.csv.gz':'baseline-7',
    # 'new-york-baseline-8-of-10__2023-01-03_19-59-14_olx/ITERS/it.5/5.events.csv.gz':'baseline-8',
    # 'new-york-baseline-9-of-10__2023-01-03_19-59-07_xcc/ITERS/it.5/5.events.csv.gz':'baseline-9',
    # 'new-york-jan2022-0-of-10__2023-01-04_00-19-31_leg/ITERS/it.5/5.events.csv.gz':'jan2022-0',
    # 'new-york-jan2022-1-of-10__2023-01-04_00-19-31_bus/ITERS/it.5/5.events.csv.gz':'jan2022-1',
    # 'new-york-jan2022-2-of-10__2023-01-04_00-19-30_bqw/ITERS/it.5/5.events.csv.gz':'jan2022-2',
    # 'new-york-jan2022-3-of-10__2023-01-04_00-19-36_noe/ITERS/it.5/5.events.csv.gz':'jan2022-3',
    # 'new-york-jan2022-4-of-10__2023-01-04_00-19-33_nop/ITERS/it.5/5.events.csv.gz':'jan2022-4',
    # 'new-york-jan2022-5-of-10__2023-01-04_00-19-37_pnp/ITERS/it.5/5.events.csv.gz':'jan2022-5',
    # 'new-york-jan2022-6-of-10__2023-01-04_00-19-29_kny/ITERS/it.5/5.events.csv.gz':'jan2022-6',
    # 'new-york-jan2022-7-of-10__2023-01-04_00-19-39_qwt/ITERS/it.5/5.events.csv.gz':'jan2022-7',
    # 'new-york-jan2022-8-of-10__2023-01-04_00-19-35_fzq/ITERS/it.5/5.events.csv.gz':'jan2022-8',
    # 'new-york-jan2022-9-of-10__2023-01-04_00-19-31_dgg/ITERS/it.5/5.events.csv.gz':'jan2022-9',
    # 'new-york-august2020-0-of-10__2023-01-04_02-12-28_pqo/ITERS/it.5/5.events.csv.gz':'aug2020-18',
    # 'new-york-august2020-0-of-10__2023-01-04_02-12-35_ftf/ITERS/it.5/5.events.csv.gz':'aug2020-19',
    # 'new-york-august2020-0-of-10__2023-01-04_02-12-31_sfq/ITERS/it.0/0.events.csv.gz':'aug2020-20-it0',
    # 'new-york-august2021-0-of-10__2023-01-04_02-12-37_qvi/ITERS/it.5/5.events.csv.gz':'aug2021-18',
    # 'new-york-august2021-0-of-10__2023-01-04_02-12-35_ial/ITERS/it.5/5.events.csv.gz':'aug2021-19',
    # 'new-york-august2021-0-of-10__2023-01-04_02-12-38_bet/ITERS/it.5/5.events.csv.gz':'aug2021-20',
    # 'new-york-may2020-0-of-10__2023-01-04_00-45-47_ipb/ITERS/it.5/5.events.csv.gz':'may2020-18',
    # 'new-york-may2020-0-of-10__2023-01-04_00-45-48_hfx/ITERS/it.5/5.events.csv.gz':'may2020-19',
    # 'new-york-june2022-0-of-10__2023-01-03_23-31-41_eiq/ITERS/it.5/5.events.csv.gz':'may2022-14',
    # 'new-york-june2022-0-of-10__2023-01-03_23-31-41_ciy/ITERS/it.5/5.events.csv.gz':'may2022-15',
    # 'new-york-june2022-0-of-10__2023-01-03_23-31-42_zmm/ITERS/it.5/5.events.csv.gz':'may2022-16',
    # 'new-york-may2020-0-of-10__2023-01-10_16-54-32_cmn/ITERS/it.5/5.events.csv.gz':'may2020-20',
    # 'new-york-may2020-0-of-10__2023-01-10_16-54-35_qsu/ITERS/it.5/5.events.csv.gz':'may2020-21',
    # 'new-york-may2020-0-of-10__2023-01-10_16-54-50_ckx/ITERS/it.5/5.events.csv.gz':'may2020-22',
    # 'new-york-may2020-0-of-10__2023-01-10_16-54-50_qmq/ITERS/it.5/5.events.csv.gz':'may2020-23',
    # 'new-york-may2020-0-of-10__2023-01-10_16-55-09_oal/ITERS/it.5/5.events.csv.gz':'may2020-24',
    # 'new-york-may2020-0-of-10__2023-01-10_16-55-11_foj/ITERS/it.5/5.events.csv.gz':'may2020-25',
    # 'new-york-may2020-0-of-10__2023-01-10_16-55-36_lvz/ITERS/it.5/5.events.csv.gz':'may2020-26',
    'new-york-baseline-0-of-10__2023-01-03_19-59-12_nwn/ITERS/it.5/5.events.csv.gz':'baseline-0',
    'new-york-baseline-1-of-10__2023-01-03_19-59-13_jgp/ITERS/it.5/5.events.csv.gz':'baseline-1',
    'new-york-baseline-2-of-10__2023-01-03_19-59-06_brm/ITERS/it.5/5.events.csv.gz':'baseline-2',
    'new-york-baseline-3-of-10__2023-01-03_19-59-07_kqr/ITERS/it.5/5.events.csv.gz':'baseline-3',
    'new-york-baseline-4-of-10__2023-01-03_19-59-09_fbb/ITERS/it.5/5.events.csv.gz':'baseline-4',
    'new-york-baseline-5-of-10__2023-01-03_19-59-11_tjh/ITERS/it.5/5.events.csv.gz':'baseline-5',
    'new-york-baseline-6-of-10__2023-01-03_19-59-12_zwm/ITERS/it.5/5.events.csv.gz':'baseline-6',
    'new-york-baseline-7-of-10__2023-01-03_19-59-19_bcr/ITERS/it.5/5.events.csv.gz':'baseline-7',
    'new-york-baseline-8-of-10__2023-01-03_19-59-14_olx/ITERS/it.5/5.events.csv.gz':'baseline-8',
    'new-york-baseline-9-of-10__2023-01-03_19-59-07_xcc/ITERS/it.5/5.events.csv.gz':'baseline-9',
    'new-york-may2020-0-of-10__2023-01-12_18-51-34_sju/ITERS/it.5/5.events.csv.gz':'may2020-0',
    'new-york-may2020-1-of-10__2023-01-12_18-51-33_nyd/ITERS/it.5/5.events.csv.gz':'may2020-1',
    'new-york-may2020-2-of-10__2023-01-12_18-51-29_yow/ITERS/it.5/5.events.csv.gz':'may2020-2',
    'new-york-may2020-3-of-10__2023-01-12_18-51-36_inx/ITERS/it.5/5.events.csv.gz':'may2020-3',
    'new-york-may2020-4-of-10__2023-01-12_18-51-31_iwm/ITERS/it.5/5.events.csv.gz':'may2020-4',
    'new-york-may2020-5-of-10__2023-01-12_18-51-34_jui/ITERS/it.5/5.events.csv.gz':'may2020-5',
    'new-york-may2020-6-of-10__2023-01-12_18-51-34_gqr/ITERS/it.5/5.events.csv.gz':'may2020-6',
    'new-york-may2020-7-of-10__2023-01-12_18-51-28_juu/ITERS/it.5/5.events.csv.gz':'may2020-7',
    'new-york-may2020-8-of-10__2023-01-12_18-51-36_ehv/ITERS/it.5/5.events.csv.gz':'may2020-8',
    'new-york-may2020-9-of-10__2023-01-12_18-51-54_bas/ITERS/it.5/5.events.csv.gz':'may2020-9',
    'new-york-august2020-0-of-10__2023-01-10_16-52-48_lgr/ITERS/it.5/5.events.csv.gz':'aug2020-0',
    'new-york-august2020-1-of-10__2023-01-10_16-52-46_gib/ITERS/it.5/5.events.csv.gz':'aug2020-1',
    'new-york-august2020-2-of-10__2023-01-14_21-19-35_flk/ITERS/it.5/5.events.csv.gz':'aug2020-2',
    'new-york-august2020-3-of-10__2023-01-15_01-54-44_txq/ITERS/it.5/5.events.csv.gz':'aug2020-3',
    'new-york-august2020-4-of-10__2023-01-10_16-52-43_rio/ITERS/it.5/5.events.csv.gz':'aug2020-4',
    'new-york-august2020-5-of-10__2023-01-10_16-53-00_vjr/ITERS/it.5/5.events.csv.gz':'aug2020-5',
    'new-york-august2020-6-of-10__2023-01-10_16-52-53_bhh/ITERS/it.5/5.events.csv.gz':'aug2020-6',
    'new-york-august2020-7-of-10__2023-01-10_16-53-01_drt/ITERS/it.5/5.events.csv.gz':'aug2020-7',
    'new-york-august2020-8-of-10__2023-01-10_16-53-15_qgy/ITERS/it.5/5.events.csv.gz':'aug2020-8',
    'new-york-august2020-9-of-10__2023-01-10_16-53-14_pdt/ITERS/it.5/5.events.csv.gz':'aug2020-9',
    'new-york-august2021-0-of-10__2023-01-12_18-51-43_rxk/ITERS/it.5/5.events.csv.gz':'aug2021-0',
    'new-york-august2021-1-of-10__2023-01-12_18-51-29_aza/ITERS/it.5/5.events.csv.gz':'aug2021-1',
    'new-york-august2021-2-of-10__2023-01-12_18-51-33_xqs/ITERS/it.5/5.events.csv.gz':'aug2021-2',
    'new-york-august2021-3-of-10__2023-01-12_18-51-29_abg/ITERS/it.5/5.events.csv.gz':'aug2021-3',
    'new-york-august2021-4-of-10__2023-01-12_18-51-34_lwg/ITERS/it.5/5.events.csv.gz':'aug2021-4',
    'new-york-august2021-5-of-10__2023-01-12_18-51-34_air/ITERS/it.5/5.events.csv.gz':'aug2021-5',
    'new-york-august2021-6-of-10__2023-01-12_18-51-28_vbj/ITERS/it.5/5.events.csv.gz':'aug2021-6',
    'new-york-august2021-7-of-10__2023-01-12_18-51-29_dnu/ITERS/it.5/5.events.csv.gz':'aug2021-7',
    'new-york-august2021-8-of-10__2023-01-12_18-51-29_uzv/ITERS/it.5/5.events.csv.gz':'aug2021-8',
    'new-york-august2021-9-of-10__2023-01-19_00-58-43_fwc/ITERS/it.5/5.events.csv.gz':'aug2021-9',
    'new-york-jan2022-0-of-10__2023-01-04_00-19-31_leg/ITERS/it.5/5.events.csv.gz':'jan2022-0',
    'new-york-jan2022-1-of-10__2023-01-14_04-04-57_ztd/ITERS/it.5/5.events.csv.gz':'jan2022-1',
    'new-york-jan2022-2-of-10__2023-01-04_00-19-30_bqw/ITERS/it.5/5.events.csv.gz':'jan2022-2',
    'new-york-jan2022-3-of-10__2023-01-04_00-19-36_noe/ITERS/it.5/5.events.csv.gz':'jan2022-3',
    'new-york-jan2022-4-of-10__2023-01-04_00-19-33_nop/ITERS/it.5/5.events.csv.gz':'jan2022-4',
    'new-york-jan2022-5-of-10__2023-01-04_00-19-37_pnp/ITERS/it.5/5.events.csv.gz':'jan2022-5',
    'new-york-jan2022-6-of-10__2023-01-04_00-19-29_kny/ITERS/it.5/5.events.csv.gz':'jan2022-6',
    'new-york-jan2022-7-of-10__2023-01-04_00-19-39_qwt/ITERS/it.5/5.events.csv.gz':'jan2022-7',
    'new-york-jan2022-8-of-10__2023-01-04_00-19-35_fzq/ITERS/it.5/5.events.csv.gz':'jan2022-8',
    'new-york-jan2022-9-of-10__2023-01-04_00-19-31_dgg/ITERS/it.5/5.events.csv.gz':'jan2022-9',
    'new-york-june2022-0-of-10__2023-01-10_16-53-18_iir/ITERS/it.5/5.events.csv.gz':'may2022-0',
    'new-york-june2022-1-of-10__2023-01-10_16-53-23_mrs/ITERS/it.0/0.events.csv.gz':'may2022-1',
    'new-york-june2022-2-of-10__2023-01-10_16-53-36_log/ITERS/it.5/5.events.csv.gz':'may2022-2',
    'new-york-june2022-3-of-10__2023-01-10_16-53-36_lvd/ITERS/it.0/0.events.csv.gz':'may2022-3',
    'new-york-june2022-4-of-10__2023-01-10_16-53-44_afe/ITERS/it.0/0.events.csv.gz':'may2022-4',
    'new-york-june2022-5-of-10__2023-01-10_16-53-58_gry/ITERS/it.5/5.events.csv.gz':'may2022-5',
    'new-york-june2022-6-of-10__2023-01-10_16-53-59_qdi/ITERS/it.5/5.events.csv.gz':'may2022-6',
    'new-york-june2022-7-of-10__2023-01-10_16-54-13_fwj/ITERS/it.0/0.events.csv.gz':'may2022-7',
    'new-york-june2022-8-of-10__2023-01-10_16-54-18_okl/ITERS/it.5/5.events.csv.gz':'may2022-8',
    'new-york-june2022-9-of-10__2023-01-10_16-54-38_jvb/ITERS/it.5/5.events.csv.gz':'may2022-9',

                }
    shares_pop = [
                0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
        # 0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333            
        # 0.1,0.1,0.1,0.1,0.1,
                 #0.059,0.059,0.059,0.059,0.059
                # 0.0295,0.0295,0.0295,0.0295,
                    ]
    data_names = data.keys()
    plan_names = []
    for data_name in data_names:
        plan_names.append(data_name[:-13]+'plans.csv.gz')

    names = data.values()
############################################RH###########################################################################
# fp = ""
# output_nm = 'SummaryTable_Pilates_RHM5.csv'
# len_id_transit = 3
# fp_res = 'outputs/'
# is_NYC = False
# is_WC = False
# is_LT = False
# is_GEO = False
# is_collect_data = True
# is_plot = False
# data = {
# Dima            
# # 's3://beam-outputs/output/newyork/nyc-ridehail-200k__2022-09-29_11-25-35_sor/ITERS/it.0/0.events.csv.gz':'Develop',
# # 's3://beam-outputs/output/newyork/nyc-ridehail-200k__2022-09-29_11-36-23_zux/ITERS/it.0/0.events.csv.gz':'RHM1',
# # 's3://beam-outputs/output/newyork/nyc-2-ridehail-200k__2022-09-29_11-44-14_esl/ITERS/it.0/0.events.csv.gz':'RHM2',
# # 's3://beam-outputs/output/newyork/nyc-2-ridehail-200k__2022-10-20_09-07-49_fii/ITERS/it.0/0.events.csv.gz':'RHM2-2',
# # Cri
# # 's3://beam-outputs/output/sfbay/sfbay-smart2-2rh-calibr__2022-10-24_02-03-37_ism/ITERS/it.0/0.events.csv.gz':'2RH',
# # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr__2022-10-23_21-30-34_jxx/ITERS/it.0/0.events.csv.gz':'1RH',
# # 's3://beam-outputs/output/sfbay/sfbay-smart2-default__2022-10-23_21-30-32_gzm/ITERS/it.0/0.events.csv.gz':'Default',
#                 's3://beam-outputs/output/sfbay/sfbay-smart2-default__2022-10-25_19-09-11_vgb/ITERS/it.0/0.events.csv.gz':'Default',
#                 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr__2022-10-25_19-09-17_bcy/ITERS/it.0/0.events.csv.gz':'1RH',
#                 's3://beam-outputs/output/sfbay/sfbay-smart2-2rh-calibr__2022-10-25_19-09-17_rup/ITERS/it.0/0.events.csv.gz':'2RH',
# # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr1__2022-10-26_06-11-56_wjs/ITERS/it.0/0.events.csv.gz':'Calibration1',
# # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr2__2022-10-26_06-11-55_etk/ITERS/it.0/0.events.csv.gz':'Calibration2',
# # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr3__2022-10-26_06-11-59_hee/ITERS/it.0/0.events.csv.gz':'Calibration3',
# # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr4__2022-10-26_06-11-56_suk/ITERS/it.0/0.events.csv.gz':'Calibration4',
# # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr5__2022-10-26_06-12-04_vfy/ITERS/it.0/0.events.csv.gz':'Calibration5',
# # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr6__2022-10-26_06-12-15_cum/ITERS/it.0/0.events.csv.gz':'Calibration6',
# # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr7__2022-10-26_06-12-44_xox/ITERS/it.0/0.events.csv.gz':'Calibration7',
#                 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr1__2022-10-27_03-00-05_whk/ITERS/it.0/0.events.csv.gz':'Calibration1',
#                 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr2__2022-10-27_03-00-05_czg/ITERS/it.0/0.events.csv.gz':'Calibration2',
# # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr3__2022-10-27_03-00-01_zto/ITERS/it.0/0.events.csv.gz':'Calibration3',
#                 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr4__2022-10-27_02-59-56_dzh/ITERS/it.0/0.events.csv.gz':'Calibration4',
#                 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr5__2022-10-27_02-59-56_eyc/ITERS/it.0/0.events.csv.gz':'Calibration5',
#                 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr6__2022-10-27_03-00-03_xin/ITERS/it.0/0.events.csv.gz':'Calibration6',
#                 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr7__2022-10-27_02-59-55_bgo/ITERS/it.0/0.events.csv.gz':'Calibration7',

                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr8__2022-11-03_16-41-48_mni/ITERS/it.0/0.events.csv.gz':'Calibration8',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr9__2022-11-03_16-41-50_fxk/ITERS/it.0/0.events.csv.gz':'Calibration9',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr10__2022-11-03_16-41-53_jgw/ITERS/it.0/0.events.csv.gz':'Calibration10',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr11__2022-11-02_23-39-15_veq/ITERS/it.0/0.events.csv.gz':'Calibration11',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr12__2022-11-03_16-41-51_kgv/ITERS/it.0/0.events.csv.gz':'Calibration12',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr13__2022-11-03_16-41-59_ovy/ITERS/it.0/0.events.csv.gz':'Calibration13',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr14__2022-11-03_16-42-01_hpf/ITERS/it.0/0.events.csv.gz':'Calibration14',

                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr11__2022-11-04_20-55-14_twf/ITERS/it.0/0.events.csv.gz':'Calibration15',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr11__2022-11-04_20-55-16_xdb/ITERS/it.0/0.events.csv.gz':'Calibration16',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr11__2022-11-04_20-54-51_nwf/ITERS/it.0/0.events.csv.gz':'Calibration17',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr11__2022-11-04_20-55-11_wtq/ITERS/it.0/0.events.csv.gz':'Calibration18',
# # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr14__2022-11-03_16-42-01_hpf/ITERS/it.0/0.events.csv.gz':'Calibration19',
# # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr14__2022-11-03_16-42-01_hpf/ITERS/it.0/0.events.csv.gz':'Calibration20',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr11__2022-11-04_20-55-00_hdd/ITERS/it.0/0.events.csv.gz':'Calibration21',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr11__2022-11-04_20-54-54_hzv/ITERS/it.0/0.events.csv.gz':'Calibration22',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr11__2022-11-04_20-55-02_vaf/ITERS/it.0/0.events.csv.gz':'Calibration23',
# # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr14__2022-11-03_16-42-01_hpf/ITERS/it.0/0.events.csv.gz':'Calibration24',
# # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr14__2022-11-03_16-42-01_hpf/ITERS/it.0/0.events.csv.gz':'Calibration25',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr11__2022-11-04_20-55-08_hhb/ITERS/it.0/0.events.csv.gz':'Calibration26',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr11__2022-11-04_20-55-34_bom/ITERS/it.0/0.events.csv.gz':'Calibration27',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr28__2022-11-08_19-27-44_ycf/ITERS/it.0/0.events.csv.gz':'Calibration28',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr29__2022-11-08_03-21-25_dax/ITERS/it.0/0.events.csv.gz':'Calibration29',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr30__2022-11-08_03-21-06_pcm/ITERS/it.0/0.events.csv.gz':'Calibration30',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr31__2022-11-08_03-21-16_uqn/ITERS/it.0/0.events.csv.gz':'Calibration31',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr32__2022-11-08_03-21-05_nhu/ITERS/it.0/0.events.csv.gz':'Calibration32',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr33__2022-11-08_03-21-02_kmt/ITERS/it.0/0.events.csv.gz':'Calibration33',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr34__2022-11-08_03-20-56_glv/ITERS/it.0/0.events.csv.gz':'Calibration34',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr35__2022-11-08_03-20-57_acx/ITERS/it.0/0.events.csv.gz':'Calibration35',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr36__2022-11-08_03-20-57_kld/ITERS/it.0/0.events.csv.gz':'Calibration36',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr37__2022-11-08_03-21-04_qyn/ITERS/it.0/0.events.csv.gz':'Calibration37',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr38__2022-11-08_03-20-57_jyd/ITERS/it.0/0.events.csv.gz':'Calibration38',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr39__2022-11-08_03-21-00_ktd/ITERS/it.0/0.events.csv.gz':'Calibration39',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr40__2022-11-08_03-21-28_kvn/ITERS/it.0/0.events.csv.gz':'Calibration40',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr41__2022-11-08_03-21-26_pkc/ITERS/it.0/0.events.csv.gz':'Calibration41',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr41__2022-11-08_19-27-34_kyc/ITERS/it.0/0.events.csv.gz':'Calibration42',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr41__2022-11-08_19-27-27_iwq/ITERS/it.0/0.events.csv.gz':'Calibration43',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr11__2022-11-04_20-55-34_bom/ITERS/it.0/0.events.csv.gz':'Calibration44',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr11__2022-11-04_20-55-34_bom/ITERS/it.0/0.events.csv.gz':'Calibration45',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr11__2022-11-04_20-55-34_bom/ITERS/it.0/0.events.csv.gz':'Calibration46',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr11__2022-11-04_20-55-34_bom/ITERS/it.0/0.events.csv.gz':'Calibration47',
                # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr11__2022-11-04_20-55-34_bom/ITERS/it.0/0.events.csv.gz':'Calibration48',
    # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr49__2022-11-10_01-36-45_ufm/ITERS/it.0/0.events.csv.gz':'Calibration49',
    # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr50__2022-11-10_01-36-41_rgz/ITERS/it.0/0.events.csv.gz':'Calibration50',
    # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr51__2022-11-10_01-36-37_uvi/ITERS/it.0/0.events.csv.gz':'Calibration51',
    # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr52__2022-11-10_01-36-38_zpx/ITERS/it.0/0.events.csv.gz':'Calibration52',
    # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr53__2022-11-10_01-37-05_wpe/ITERS/it.0/0.events.csv.gz':'Calibration53',
    # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr54__2022-11-10_01-37-07_mze/ITERS/it.0/0.events.csv.gz':'Calibration54',
    # # 's3://beam-outputs/output/sfbay//ITERS/it.0/0.events.csv.gz':'Calibration55',
    # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr56__2022-11-10_01-38-04_avo/ITERS/it.0/0.events.csv.gz':'Calibration56',
    # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr57__2022-11-10_01-37-54_qmz/ITERS/it.0/0.events.csv.gz':'Calibration57',
    # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr58__2022-11-10_01-38-00_zbn/ITERS/it.0/0.events.csv.gz':'Calibration58',
    # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr59__2022-11-10_01-38-18_oiv/ITERS/it.0/0.events.csv.gz':'Calibration59',
    # 's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr60__2022-11-10_01-39-04_bzm/ITERS/it.0/0.events.csv.gz':'Calibration60',
#     's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr41__2022-11-15_02-54-11_rex/ITERS/it.0/0.events.csv.gz':'Calibration44',
#     's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr61__2022-11-15_03-52-29_qjw/ITERS/it.0/0.events.csv.gz':'Calibration61',
#     's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr62__2022-11-15_03-52-30_ckn/ITERS/it.0/0.events.csv.gz':'Calibration62',
#     's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr63__2022-11-15_03-52-30_ylj/ITERS/it.0/0.events.csv.gz':'Calibration63',
#     's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr64__2022-11-15_03-52-28_hub/ITERS/it.0/0.events.csv.gz':'Calibration64',
#     's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr65__2022-11-15_03-52-28_ikv/ITERS/it.0/0.events.csv.gz':'Calibration65',
#     's3://beam-outputs/output/sfbay/sfbay-smart2-rh-calibr60__2022-11-15_03-52-29_bkb/ITERS/it.0/0.events.csv.gz':'Calibration66',
# }
# data_names = data.keys()
# plan_names = []
# for data_name in data_names:
#     plan_names.append(data_name[:-13]+'plans.csv.gz')

# names = data.values()
############################################SMART###########################################################################
if is_SMART:
    fp = ""
    output_nm = 'ST-smart2_35.csv'
    len_id_transit = 3
    fp_res = 'outputs/'
    is_NYC = False
    is_WC = False
    is_LT = False
    is_GEO = False
    is_collect_data = False
    is_plot = False
    is_filter_by_region = False
    region_filter = 'SF'
    data = {
                    # 's3://beam-outputs/pilates-outputs/sfbay-base-20220409/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'Baseline/year-2018-iteration-5',
                    # 's3://beam-outputs/pilates-outputs/sfbay-RH_fleetsz_0.125-20220408/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_fleetsz_0.125',
                    # 's3://beam-outputs/pilates-outputs/sfbay-RH_fleetsz_0.25-20220408/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_fleetsz_0.25',
                    # 's3://beam-outputs/pilates-outputs/sfbay-RH_fleetsz_0.5-20220408/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_fleetsz_0.50',
                    # 's3://beam-outputs/pilates-outputs/sfbay-RH_fleetsz_1.75-20220408/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_fleetsz_1.75',
                    # 's3://beam-outputs/pilates-outputs/sfbay-transit_frequencies_0.5-20220528/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-transit_frequencies_0.5',
                    # 's3://beam-outputs/s3://beam-outputs/pilates-outputs/sfbay-transit_frequencies_1.5-20220529/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-transit_frequencies_1.5',
                    # 'pilates-outputs/sfbay-transit_frequencies_2.0-20220529/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-transit_frequencies_2.0',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_repo_0.0-20220613/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_repo_0.0',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_repo_0.5-20220613/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_repo_0.5',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_repo_1.5-20220613/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_repo_1.5',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_repo_3.0-20220613/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_repo_3.0',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_radius_0.2-20220610/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_radius_0.2',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_radius_0.5-20220610/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_radius_0.5',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_radius_1.5-20220614/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_radius_1.5',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_radius_5.0-20220616/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_radius_5.0',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_wt_0.2-20220617/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_wt_0.2',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_wt_0.5-20220617/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_wt_0.5',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_wt_0.8-20220617/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_wt_0.8',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_wt_1.6-20220617/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_wt_1.6',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_detour_0.0-20220618/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_detour_0.0',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_detour_0.5-20220618/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_detour_0.5',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_detour_0.75-20220618/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_detour_0.75',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_detour_1.5-20220618/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_detour_1.5',
                    # 's3://beam-outputs/pilates-outputs/sfbay-pilates_june_test-20220616/beam/year-2010-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-cp_pilatesJune2022',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_price_0.5-20220616/beam/year-2010-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_price_0.5',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_invrepo_dem_0.0_dist_0.0-20220627/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_invrepo_dem_0.0_dist_0.0',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_invrepo_dem_0.2_dist_0.9-20220627/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_invrepo_dem_0.2_dist_0.9',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_invrepo_dem_0.4_dist_0.3-20220628/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_invrepo_dem_0.4_dist_0.3',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_invrepo_dem_0.4_dist_0.6-20220628/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_invrepo_dem_0.4_dist_0.6',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_invrepo_dem_0.4_dist_1.0-20220628/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_invrepo_dem_0.4_dist_1.0',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_invrepo_dem_0.6_dist_0.9-20220627/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_invrepo_dem_0.6_dist_0.9',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_invrepo_dem_1.0_dist_0.9-20220627/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_invrepo_dem_1.0_dist_0.9',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rh_invrepo_dem_1.0_dist_1.0-20220628/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-rh_invrepo_dem_1.0_dist_1.0',
                    # 's3://beam-outputs/output/sfbay/sfbay-pilates-50pop-5veh__2022-07-20_21-48-51_yti/ITERS/it.2/2.events.csv.gz':'Disabilities 5% RH WC 50% RH fleet',
                    # 's3://beam-outputs/output/sfbay/sfbay-pilates-50pop-10veh__2022-07-20_21-48-52_ucj/ITERS/it.2/2.events.csv.gz':'Disabilities 10% RH WC 50% RH fleet',
                    # 's3://beam-outputs/output/sfbay/sfbay-pilates-50pop-20veh__2022-07-20_21-48-54_xwf/ITERS/it.2/2.events.csv.gz':'Disabilities 20% RH WC 50% RH fleet',
                    # 's3://beam-outputs/output/sfbay/sfbay-pilates-50pop-50veh__2022-07-20_21-49-07_oyq/ITERS/it.2/2.events.csv.gz':'Disabilities 50% RH WC 50% RH fleet',
                    # 's3://beam-outputs/output/sfbay/sfbay-pilates-50pop-100veh__2022-07-20_21-49-00_lgi/ITERS/it.2/2.events.csv.gz':'Disabilities 100% RH WC 50% RH fleet',
                    # 's3://beam-outputs/output/sfbay/sfbay-pilates-50pop-5veh__2022-07-15_04-13-50_ntn/ITERS/it.2/2.events.csv.gz':'Disabilities 5% RH WC 100% RH fleet',
                    # 's3://beam-outputs/output/sfbay/sfbay-pilates-50pop-10veh__2022-07-15_04-13-44_okn/ITERS/it.2/2.events.csv.gz':'Disabilities 10% RH WC 100% RH fleet',
                    # 's3://beam-outputs/output/sfbay/sfbay-pilates-50pop-20veh__2022-07-15_04-13-46_szz/ITERS/it.2/2.events.csv.gz':'Disabilities 20% RH WC 100% RH fleet',
                    # 's3://beam-outputs/output/sfbay/sfbay-pilates-50pop-50veh__2022-07-15_04-13-48_qbr/ITERS/it.2/2.events.csv.gz':'Disabilities 50% RH WC 100% RH fleet',
                    # 's3://beam-outputs/output/sfbay/sfbay-pilates-50pop-100veh__2022-07-15_04-13-44_apr/ITERS/it.2/2.events.csv.gz':'Disabilities 100% RH WC 100% RH fleet',
                    # 's3://beam-outputs/pilates-outputs/sfbay-base-20220409/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'Baseline-july2022',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhprice-0.0-20220814/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_price_0.0',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhprice-0.5-20220814/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_price_0.5',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhprice-1.5-20220815/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_price_1.5',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhprice-2.0-20220815/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_price_2.0',
                    # 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220822/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'Baseline-august2022',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhfleet-0.125-20220822/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_fleetsz_0.125',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhfleet-0.25-20220822/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_fleetsz_0.25',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhfleet-0.50-20220822/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_fleetsz_0.50',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhfleet-1.75-20220822/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_fleetsz_1.75',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhfleet-2.25-20220822/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_fleetsz_2.25',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhfleet-3.0-20220822/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_fleetsz_3.0',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhprice0.0-rhfleet1.0-20220904/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_price_0.0_fleetsz_1.0',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhprice0.05-rhfleet2.25-20220905/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_price_0.05_fleetsz_2.25',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhprice0.25-rhfleet1.0-20220903/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_price_0.25_fleetsz_1.0',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhprice0.25-rhfleet2.25-20220904/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_price_0.25_fleetsz_2.25',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhprice0.5-rhfleet0.5-20220903/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_price_0.5_fleetsz_0.5',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhprice0.5-rhfleet1.0-20220903/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_price_0.5_fleetsz_1.0',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhprice0.5-rhfleet2.25-20220903/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_price_0.5_fleetsz_2.25',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhprice1.5-rhfleet0.125-20220903/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_price_1.5_fleetsz_0.125',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhprice1.5-rhfleet0.25-20220904/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_price_1.5_fleetsz_0.25',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhprice1.5-rhfleet0.5-20220904/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_price_1.5_fleetsz_0.5',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhprice1.5-rhfleet1.0-20220904/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_price_1.5_fleetsz_1.5',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhprice1.5-rhfleet2.25-20220905/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_price_1.5_fleetsz_2.25',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhprice2.0-rhfleet0.5-20220905/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_price_2.0_fleetsz_0.5',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhprice2.0-rhfleet1.0-20220905/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_price_2.0_fleetsz_1.0',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhprice2.5-rhfleet0.125-20220905/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_price_2.5_fleetsz_0.125',
                    # 's3://beam-outputs/pilates-outputs/sfbay-rhprice2.5-rhfleet0.25-20220905/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'sfbay-RH_price_2.5_fleetsz_0.25',
                    # 'gs://beam-core-outputs/sfbay-2010-accessibility-fleetpct-0.05-wheelchair-0.5-20220917/beam/year-2012-iteration-3/ITERS/it.0/0.events.csv.gz':'5% RH WC 50% RH fleet',
                    # 'gs://beam-core-outputs/sfbay-2010-accessibility-fleetpct—0.1wheelchair-0.5-20220917/beam/year-2012-iteration-3/ITERS/it.0/0.events.csv.gz':'10% RH WC 50% RH fleet',
                    # 'gs://beam-core-outputs/sfbay-2010-accessibility-fleetpct-0.2-wheelchair-0.5-20220917/beam/year-2012-iteration-3/ITERS/it.0/0.events.csv.gz':'20% RH WC 50% RH fleet',
                    # 'gs://beam-core-outputs/beam-core-outputs/sfbay-2010-accessibility-fleetpct-0.5-wheelchair-0.5-20220917/beam/year-2012-iteration-3/ITERS/it.0/0.events.csv.gz':'50% RH WC 50% RH fleet',
                    # 'gs://beam-core-outputs/sfbay-2010-accessibility-fleetpct-1.0-wheelchair-0.5-20220917/beam/year-2012-iteration-3/ITERS/it.0/0.events.csv.gz':'100% RH WC 50% RH fleet',
                    # Ambarish
                    # 's3://beam-outputs/pilates-outputs/15thSep2019/baseline/beam/sfbay-smart-base-pilates__2019-09-13_17-22-29/ITERS/it.15/15.events.csv.gz':'2010 Baseline',
                    # Telecommute
                    # 's3://beam-outputs/pilates-outputs/sfbay-telecommute-baseline-2022-10-8/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'Telecommuting_Baseline-year2018-it5',
                    # 's3://beam-outputs/pilates-outputs/sfbay-telecommute-baseline-2022-10-8/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz':'Telecommuting_Baseline-year2021-it5',
                    # TR
                    # 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220816/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz':'Baseline TR 2021',
                    # 's3://beam-outputs/pilates-outputs/sfbay-TR-20220812/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz':'Future TR 2021',
                    # 's3://beam-outputs/pilates-outputs/sfbay-baseline-asis-20221209/beam/year-2010-iteration-4/ITERS/it.0/0.events.csv.gz':'Baseline-asis0',
                    # 's3://beam-outputs/pilates-outputs/sfbay-baseline-beam_rh_calibration-20221209/beam/year-2010-iteration-4/ITERS/it.0/0.events.csv.gz':'Baseline-rh_beam0',
                    # 's3://beam-outputs/pilates-outputs/sfbay-pilates-baseline-asis-20221213/beam/year-2010-iteration-4/ITERS/it.0/0.events.csv.gz':'Baseline-asis1',
                    # 's3://beam-outputs/pilates-outputs/sfbay-pilates-baseline-rh_beam-20221213/beam/year-2010-iteration-4/ITERS/it.0/0.events.csv.gz':'Baseline-rh_beam1',
                    # 's3://beam-outputs/pilates-outputs/sfbay-pilates-baseline-rh_beam_asim1-20221213/beam/year-2010-iteration-4/ITERS/it.0/0.events.csv.gz':'Baseline-rh_beam_asim1',
                    # 's3://beam-outputs/pilates-outputs/sfbay-pilates-baseline-rh_beam_asim2-20221213/beam/year-2010-iteration-4/ITERS/it.0/0.events.csv.gz':'Baseline-rh_beam_asim2',
                    # 's3://beam-outputs/pilates-outputs/sfbay-pilates-baseline-rh_beam_asim3-20221213/beam/year-2010-iteration-4/ITERS/it.0/0.events.csv.gz':'Baseline-rh_beam_asim3',


        # 's3://beam-outputs/pilates-outputs/sfbay_1fleet_100price_100fleet_30pct_20230222/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'1Fl 1Pr 1Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_baseline_30pct_20230218/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'2Fl 1Pr 1Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_100price_100fleet_30pct_20230218/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 1Pr 1Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_100price_164fleet_30pct_20230218/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 1Pr 1.64Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_100price_200fleet_30pct_20230218/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 1Pr 2Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_100price_400fleet_30pct_20230218/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 1Pr 4Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_100price_1000fleet_30pct_20230218/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 1Pr 10Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_2fleets_100price_164fleet_30pct_20230218/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'2Fl 1Pr 1.64Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_2fleets_100price_200fleet_30pct_20230218/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'2Fl 1Pr 2Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_2fleets_100price_400fleet_30pct_20230218/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'2Fl 1Pr 4Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_2fleets_100price_1000fleet_30pct_20230218/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'2Fl 1Pr 10Flz',
    # 's3://beam-outputs/pilates-outputs/sfbay_2fleets_47price_100fleet_30pct_20230226/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'2Fl 0.47Pr 1Flz',
    # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_47price_100fleet_30pct_20230226/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 0.47Pr 1Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_47price_164fleet_30pct_20230226/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 0.47Pr 1.64Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_47price_200fleet_30pct_20230226/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 0.47Pr 2Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_47price_400fleet_30pct_20230226/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 0.47Pr 4Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_47price_1000fleet_30pct_20230226/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 0.47Pr 10Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_2fleets_27price_100fleet_30pct_20230221/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'2Fl 0.27Pr 1Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_27price_100fleet_30pct_20230218/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 0.27Pr 1Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_27price_164fleet_30pct_20230218/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 0.27Pr 1.64Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_27price_200fleet_30pct_20230218/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 0.27Pr 2Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_27price_400fleet_30pct_20230218/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 0.27Pr 4Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_27price_1000fleet_30pct_20230218/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 0.27Pr 10Flz',
    # 's3://beam-outputs/pilates-outputs/sfbay_2fleets_12_5price_100fleet_30pct_20230226/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'2Fl 0.125Pr 1Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_12_5price_100fleet_30pct_20230226/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 0.125Pr 1Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_12_5price_164fleet_30pct_20230226/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 0.125Pr 1.64Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_12_5price_200fleet_30pct_20230226/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 0.125Pr 2Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_12_5price_400fleet_30pct_20230226/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 0.125Pr 4Flz',
    # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_12_5price_1000fleet_30pct_20230226/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 0.125Pr 10Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_2fleets_6_25_price_100fleet_30pct_20230223/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'2Fl 0.0625Pr 1Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_6_25price_100fleet_30pct_20230218/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 0.0625Pr 1Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_6_25price_164fleet_30pct_20230218/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 0.0625Pr 1.64Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_6_25price_200fleet_30pct_20230223/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 0.0625Pr 2Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_6_25_price_400fleet_30pct_20230223/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 0.0625Pr 4Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_6_25_price_1000fleet_30pct_20230223/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl 0.0625Pr 10Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_1fleet_mix_price_100fleet_30pct_20230223/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'1Fl MixPr 1Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_2fleets_mix_price_100fleet_30pct_20230226/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'2Fl MixPr 1Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_mixprice_100fleet_30pct_20230223/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl MixPr 1Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_mix_price_164fleet_30pct_20230226/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl MixPr 1.64Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_mixprice_200fleet_30pct_20230223/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl MixPr 2Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_mixprice_400fleet_30pct_20230223/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl MixPr 4Flz',
        # 's3://beam-outputs/pilates-outputs/sfbay_5fleets_mixprice_1000fleet_30pct_20230223/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'5Fl MixPr 10Flz',
                # 's3://beam-outputs/pilates-outputs/sfbay_baseline_20230331/beam/year-2018-iteration-2/ITERS/it.0/0.events.csv.gz':'Base-20230331', # BART ridership
                            # 'gs://beam-core-outputs/sfbay-baseline-20230410-omx/beam/year-2020-iteration-2/ITERS/it.0/0.events.csv.gz':'OMX-Baseline-20.2', # Test OMX
                    # 'gs://beam-core-outputs/sfbay-baseline-20230416-omx/beam/year-2020-iteration-2/ITERS/it.0/0.events.csv.gz':'OMX-Baseline2-20.2', # Test OMX
                            # 'gs://beam-core-outputs/sfbay-baseline-20230418-omx-tr/beam/year-2020-iteration-2/ITERS/it.0/0.events.csv.gz':'OMX-Baseline2-tr-20.2', # Test OMX
                    # 'gs://beam-core-outputs/sfbay-baseline-20230425-omx/beam/year-2018-iteration-2/ITERS/it.0/0.events.csv.gz':'OMX-Baseline3-tr-18.2', # Test OMX
                    # 'gs://beam-core-outputs/sfbay-baseline-20230425-omx/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'OMX-Baseline3-tr-19.2', # Test OMX
                    # 'gs://beam-core-outputs/sfbay-baseline-20230425-omx/beam/year-2020-iteration--1/ITERS/it.0/0.events.csv.gz':'OMX-Baseline3-tr-20.0', # Test OMX
                    # 'gs://beam-core-outputs/sfbay-baseline-20230425-omx/beam/year-2020-iteration-1/ITERS/it.0/0.events.csv.gz':'OMX-Baseline3-tr-20.1', # Test OMX
                    # 'gs://beam-core-outputs/sfbay-baseline-20230425-omx/beam/year-2020-iteration-2/ITERS/it.0/0.events.csv.gz':'OMX-Baseline3-tr-20.2', # Test OMX
                # 'gs://beam-core-outputs/sfbay-baseline-TR-testBA-20230427/beam/year-2018-iteration-2/ITERS/it.0/0.events.csv.gz':'TR-testBA', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230427/beam/year-2018-iteration--1/ITERS/it.0/0.events.csv.gz':'Base-18.0', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230427/beam/year-2018-iteration-1/ITERS/it.0/0.events.csv.gz':'Base-18.1', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230427/beam/year-2018-iteration-2/ITERS/it.0/0.events.csv.gz':'Base-18.2', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230427/beam/year-2018-iteration-3/ITERS/it.0/0.events.csv.gz':'Base-18.3', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230427/beam/year-2018-iteration-4/ITERS/it.0/0.events.csv.gz':'Base-18.4', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230427/beam/year-2019-iteration--1/ITERS/it.0/0.events.csv.gz':'Base-19.0', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230427/beam/year-2019-iteration-1/ITERS/it.0/0.events.csv.gz':'Base-19.1', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230427/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'Base-19.2', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230427/beam/year-2019-iteration-3/ITERS/it.0/0.events.csv.gz':'Base-19.3', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230427/beam/year-2019-iteration-4/ITERS/it.0/0.events.csv.gz':'Base-19.4', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230427/beam/year-2020-iteration--1/ITERS/it.0/0.events.csv.gz':'Base-20.0', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230427/beam/year-2020-iteration-1/ITERS/it.0/0.events.csv.gz':'Base-20.1', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230427/beam/year-2020-iteration-2/ITERS/it.0/0.events.csv.gz':'Base-20.2', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230427/beam/year-2020-iteration-3/ITERS/it.0/0.events.csv.gz':'Base-20.3', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230427/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Base-20.4', # Test OMX
            # 'gs://beam-core-outputs/sfbay-tr-20230427/beam/year-2018-iteration--1/ITERS/it.0/0.events.csv.gz':'TR-18.0', # Test OMX
            # 'gs://beam-core-outputs/sfbay-tr-20230427/beam/year-2018-iteration-1/ITERS/it.0/0.events.csv.gz':'TR-18.1', # Test OMX
            # 'gs://beam-core-outputs/sfbay-tr-20230427/beam/year-2018-iteration-2/ITERS/it.0/0.events.csv.gz':'TR-18.2', # Test OMX
            # 'gs://beam-core-outputs/sfbay-tr-20230427/beam/year-2018-iteration-3/ITERS/it.0/0.events.csv.gz':'TR-18.3', # Test OMX
            # 'gs://beam-core-outputs/sfbay-tr-20230427/beam/year-2018-iteration-4/ITERS/it.0/0.events.csv.gz':'TR-18.4', # Test OMX
            # 'gs://beam-core-outputs/sfbay-tr-20230427/beam/year-2019-iteration--1/ITERS/it.0/0.events.csv.gz':'TR-19.0', # Test OMX
            # 'gs://beam-core-outputs/sfbay-tr-20230427/beam/year-2019-iteration-1/ITERS/it.0/0.events.csv.gz':'TR-19.1', # Test OMX
            # 'gs://beam-core-outputs/sfbay-tr-20230427/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'TR-19.2', # Test OMX
            # 'gs://beam-core-outputs/sfbay-tr-20230427/beam/year-2019-iteration-3/ITERS/it.0/0.events.csv.gz':'TR-19.3', # Test OMX
            # 'gs://beam-core-outputs/sfbay-tr-20230427/beam/year-2019-iteration-4/ITERS/it.0/0.events.csv.gz':'TR-19.4', # Test OMX
            # 'gs://beam-core-outputs/sfbay-tr-20230427/beam/year-2020-iteration--1/ITERS/it.0/0.events.csv.gz':'TR-20.0', # Test OMX
            # 'gs://beam-core-outputs/sfbay-tr-20230427/beam/year-2020-iteration-1/ITERS/it.0/0.events.csv.gz':'TR-20.1', # Test OMX
            # 'gs://beam-core-outputs/sfbay-tr-20230427/beam/year-2020-iteration-2/ITERS/it.0/0.events.csv.gz':'TR-20.2', # Test OMX
            # 'gs://beam-core-outputs/sfbay-tr-20230427/beam/year-2020-iteration-3/ITERS/it.0/0.events.csv.gz':'TR-20.3', # Test OMX
            # 'gs://beam-core-outputs/sfbay-tr-20230427/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'TR-20.4', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230503-omx/beam/year-2018-iteration--1/ITERS/it.0/0.events.csv.gz':'Base2-18.0', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230503-omx/beam/year-2018-iteration-1/ITERS/it.0/0.events.csv.gz':'Base2-18.1', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230503-omx/beam/year-2018-iteration-2/ITERS/it.0/0.events.csv.gz':'Base2-18.2', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230503-omx/beam/year-2018-iteration-3/ITERS/it.0/0.events.csv.gz':'Base2-18.3', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230503-omx/beam/year-2018-iteration-4/ITERS/it.0/0.events.csv.gz':'Base2-18.4', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230503-omx/beam/year-2019-iteration--1/ITERS/it.0/0.events.csv.gz':'Base2-19.0', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230503-omx/beam/year-2019-iteration-1/ITERS/it.0/0.events.csv.gz':'Base2-19.1', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230503-omx/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'Base2-19.2', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230503-omx/beam/year-2019-iteration-3/ITERS/it.0/0.events.csv.gz':'Base2-19.3', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230503-omx/beam/year-2019-iteration-4/ITERS/it.0/0.events.csv.gz':'Base2-19.4', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230503-omx/beam/year-2020-iteration--1/ITERS/it.0/0.events.csv.gz':'Base2-20.0', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230503-omx/beam/year-2020-iteration-1/ITERS/it.0/0.events.csv.gz':'Base2-20.1', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230503-omx/beam/year-2020-iteration-2/ITERS/it.0/0.events.csv.gz':'Base2-20.2', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230503-omx/beam/year-2020-iteration-3/ITERS/it.0/0.events.csv.gz':'Base2-20.3', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230503-omx/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Base2-20.4', # Test OMX
            # 'gs://beam-core-outputs/sfbay-TR-20230505/beam/year-2018-iteration--1/ITERS/it.0/0.events.csv.gz':'TR-18.0', # Test OMX
            # 'gs://beam-core-outputs/sfbay-TR-20230505/beam/year-2018-iteration-1/ITERS/it.0/0.events.csv.gz':'TR-18.1', # Test OMX
            # 'gs://beam-core-outputs/sfbay-TR-20230505/beam/year-2018-iteration-2/ITERS/it.0/0.events.csv.gz':'TR-18.2', # Test OMX
            # 'gs://beam-core-outputs/sfbay-TR-20230505/beam/year-2018-iteration-3/ITERS/it.0/0.events.csv.gz':'TR-18.3', # Test OMX
            # 'gs://beam-core-outputs/sfbay-TR-20230505/beam/year-2018-iteration-4/ITERS/it.0/0.events.csv.gz':'TR-18.4', # Test OMX
            # 'gs://beam-core-outputs/sfbay-TR-20230505/beam/year-2019-iteration--1/ITERS/it.0/0.events.csv.gz':'TR-19.0', # Test OMX
            # 'gs://beam-core-outputs/sfbay-TR-20230505/beam/year-2019-iteration-1/ITERS/it.0/0.events.csv.gz':'TR-19.1', # Test OMX
            # 'gs://beam-core-outputs/sfbay-TR-20230505/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'TR-19.2', # Test OMX
            # 'gs://beam-core-outputs/sfbay-TR-20230505/beam/year-2019-iteration-3/ITERS/it.0/0.events.csv.gz':'TR-19.3', # Test OMX
            # 'gs://beam-core-outputs/sfbay-TR-20230505/beam/year-2019-iteration-4/ITERS/it.0/0.events.csv.gz':'TR-19.4', # Test OMX
            # 'gs://beam-core-outputs/sfbay-TR-20230505/beam/year-2020-iteration--1/ITERS/it.0/0.events.csv.gz':'TR-20.0', # Test OMX
            # 'gs://beam-core-outputs/sfbay-TR-20230505/beam/year-2020-iteration-1/ITERS/it.0/0.events.csv.gz':'TR-20.1', # Test OMX
            # 'gs://beam-core-outputs/sfbay-TR-20230505/beam/year-2020-iteration-2/ITERS/it.0/0.events.csv.gz':'TR-20.2', # Test OMX
            # 'gs://beam-core-outputs/sfbay-TR-20230505/beam/year-2020-iteration-3/ITERS/it.0/0.events.csv.gz':'TR-20.3', # Test OMX
            # 'gs://beam-core-outputs/sfbay-TR-20230505/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'TR-20.4', # Test OMX
            # 'gs://beam-core-outputs/sfbay-baseline-20230510/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Base4-20.4', # Test OMX -Solo correct
            # 'gs://beam-core-outputs/sfbay-TR-20230511/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Poolfree4-20.4', # Test OMX- Pool free solo correct
            # 'gs://beam-core-outputs/sfbay-baseline-atlas/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Base5-20.4', # Test ATLAS
            # 'gs://beam-core-outputs/sfbay-baseline-20230517/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Base6-20.4', # Test TRN correction
            # 'gs://beam-core-outputs/sfbay-TR-20230519/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'TR6-20.4', # TR compared to Base 6
            # 'gs://beam-core-outputs/sfbay-baseline-20230523/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Base7-20.4', # other corrections to TRN Zach did

            # 'gs://beam-core-outputs/sfbay-baseline2018-30pct-20230710/beam/year-2018-iteration--1/ITERS/it.0/0.events.csv.gz':'Base18.0',
            # 'gs://beam-core-outputs/sfbay-tr-30pct-20230716/beam/year-2018-iteration--1/ITERS/it.0/0.events.csv.gz':'TR18.0',
            # 'gs://beam-core-outputs/sfbay-baseline2018-30pct-20230710/beam/year-2018-iteration-1/ITERS/it.0/0.events.csv.gz':'Base18.1',
            # 'gs://beam-core-outputs/sfbay-tr-30pct-20230716/beam/year-2018-iteration-1/ITERS/it.0/0.events.csv.gz':'TR18.1',
            # 'gs://beam-core-outputs/sfbay-baseline2018-30pct-20230710/beam/year-2018-iteration-2/ITERS/it.0/0.events.csv.gz':'Base18.2',
            # 'gs://beam-core-outputs/sfbay-tr-30pct-20230716/beam/year-2018-iteration-2/ITERS/it.0/0.events.csv.gz':'TR18.2',
            # 'gs://beam-core-outputs/sfbay-baseline2018-30pct-20230710/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'Base18.5',
            # 'gs://beam-core-outputs/sfbay-tr-30pct-20230716/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'TR18.5',
            # 'gs://beam-core-outputs/sfbay-baseline2018-30pct-20230710/beam/year-2018-iteration-7/ITERS/it.0/0.events.csv.gz':'Base18.7',
            # 'gs://beam-core-outputs/sfbay-tr-30pct-20230716/beam/year-2018-iteration-7/ITERS/it.0/0.events.csv.gz':'TR18.7',
            # 'gs://beam-core-outputs/sfbay-baseline2018-30pct-20230710/beam/year-2018-iteration-9/ITERS/it.0/0.events.csv.gz':'Base18.9',
            # 'gs://beam-core-outputs/sfbay-tr-30pct-20230716/beam/year-2018-iteration-9/ITERS/it.0/0.events.csv.gz':'TR18.9',
            # 'gs://beam-core-outputs/sfbay-baseline2018-30pct-20230710/beam/year-2018-iteration-10/ITERS/it.0/0.events.csv.gz':'Base18.10',
            # 'gs://beam-core-outputs/sfbay-tr-30pct-20230716/beam/year-2018-iteration-10/ITERS/it.0/0.events.csv.gz':'TR18.10',
#             'gs://beam-core-outputs/sfbay-baseline-30pct-20230603/beam/year-2019-iteration--1/ITERS/it.0/0.events.csv.gz':'Base19.0',
#             'gs://beam-core-outputs/sfbay-tr-30pct-20230603/beam/year-2019-iteration--1/ITERS/it.0/0.events.csv.gz':'TR19.0',
#             'gs://beam-core-outputs/sfbay-baseline-30pct-20230603/beam/year-2019-iteration-1/ITERS/it.0/0.events.csv.gz':'Base19.1',
#             'gs://beam-core-outputs/sfbay-tr-30pct-20230603/beam/year-2019-iteration-1/ITERS/it.0/0.events.csv.gz':'TR19.1',
#             'gs://beam-core-outputs/sfbay-baseline-30pct-20230603/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'Base19.2',
#             'gs://beam-core-outputs/sfbay-tr-30pct-20230603/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'TR19.2',
#             'gs://beam-core-outputs/sfbay-baseline-30pct-20230603/beam/year-2019-iteration-3/ITERS/it.0/0.events.csv.gz':'Base19.3',
#             'gs://beam-core-outputs/sfbay-tr-30pct-20230603/beam/year-2019-iteration-3/ITERS/it.0/0.events.csv.gz':'TR19.3',
#             'gs://beam-core-outputs/sfbay-baseline-30pct-20230603/beam/year-2019-iteration-4/ITERS/it.0/0.events.csv.gz':'Base19.4',
#             'gs://beam-core-outputs/sfbay-tr-30pct-20230603/beam/year-2019-iteration-4/ITERS/it.0/0.events.csv.gz':'TR19.4',
        
#             'gs://beam-core-outputs/sfbay-baseline-30pct-20230603/beam/year-2020-iteration--1/ITERS/it.0/0.events.csv.gz':'Base20.0',
#             'gs://beam-core-outputs/sfbay-tr-30pct-20230603/beam/year-2020-iteration--1/ITERS/it.0/0.events.csv.gz':'TR20.0',
#             'gs://beam-core-outputs/sfbay-baseline-30pct-20230603/beam/year-2020-iteration-1/ITERS/it.0/0.events.csv.gz':'Base20.1',
#             'gs://beam-core-outputs/sfbay-tr-30pct-20230603/beam/year-2020-iteration-1/ITERS/it.0/0.events.csv.gz':'TR20.1',
#             'gs://beam-core-outputs/sfbay-baseline-30pct-20230603/beam/year-2020-iteration-2/ITERS/it.0/0.events.csv.gz':'Base20.2',
#             'gs://beam-core-outputs/sfbay-tr-30pct-20230603/beam/year-2020-iteration-2/ITERS/it.0/0.events.csv.gz':'TR20.2',
#             'gs://beam-core-outputs/sfbay-baseline-30pct-20230603/beam/year-2020-iteration-3/ITERS/it.0/0.events.csv.gz':'Base20.3',
#             'gs://beam-core-outputs/sfbay-tr-30pct-20230603/beam/year-2020-iteration-3/ITERS/it.0/0.events.csv.gz':'TR20.3',
            # 'gs://beam-core-outputs/sfbay-baseline-30pct-20230603/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Base20.4',
            # 'gs://beam-core-outputs/sfbay-tr-30pct-20230603/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'TR20.4',
                    
        'gs://beam-core-outputs/sfbay-baseline-20230526/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Baseline', # other corrections...BEAM 0.9.5
        # 'gs://beam-core-outputs/sfbay-incentives-20230609/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'incentives', 
        # 'gs://beam-core-outputs/sfbay-incentives-20230618/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'incentives', # include walk transit and bike transit
        # 'gs://beam-core-outputs/sfbay-cordon-20230609/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Cordon', 
        # 'gs://beam-core-outputs/sfbay-incentives-20230612/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'incentives', 
        # 'gs://beam-core-outputs/sfbay-cordon-20230612/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Cordon',
        # 'gs://beam-core-outputs/sfbay-detour0125-20230623/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Detour-0.125', 
        # 'gs://beam-core-outputs/sfbay-detour05-20230623/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Detour-0.5', 
        # 'gs://beam-core-outputs/sfbay-detour3-20230623/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Detour-3', 
        # 'gs://beam-core-outputs/sfbay-detour10-20230623/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Detour-10', 
        # 'gs://beam-core-outputs/sfbay-repo_0_125-20230603/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Repo-0.125',
        # 'gs://beam-core-outputs/sfbay-repo_0_5-20230603/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Repo-0.5',
        # 'gs://beam-core-outputs/sfbay-repo_3-20230603/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Repo-3',
        # 'gs://beam-core-outputs/sfbay-repo_10-20230603/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Repo-10',
        # 'gs://beam-core-outputs/sfbay-tr_capacity_0_25-20230608/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'capacity-0.25',
        # 'gs://beam-core-outputs/sfbay-tr_capacity_0_5-20230608/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'capacity-0.5',
        # 'gs://beam-core-outputs/sfbay-tr_capacity_1_5-20230608/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'capacity-1.5',
        # 'gs://beam-core-outputs/sfbay-tr_capacity_2-20230607/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'capacity-2',
        # 'gs://beam-core-outputs/sfbay-tr-frequency_0_25-20230614/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'freq-0.25',
        # 'gs://beam-core-outputs/sfbay-tr-frequency_0_50-20230614/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'freq-0.5',
        # 'gs://beam-core-outputs/sfbay-tr-frequency_1_5-20230614/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'freq-1.5',
        # 'gs://beam-core-outputs/sfbay-tr-frequency_2-20230616/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'freq-2',
        # 'gs://beam-core-outputs/sfbay-rh-price_0_125-20230614/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'RHprice-0.125',
        # 'gs://beam-core-outputs/sfbay-rh-price_0_5-20230614/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'RHprice-0.5',
        # 'gs://beam-core-outputs/sfbay-rh-price_2-20230614/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'RHprice-2',
        # 'gs://beam-core-outputs/sfbay-rh-price_5-20230614/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'RHprice-5',
        # 'gs://beam-core-outputs/sfbay-tr-speed0_25-20230626/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'TRspeed-0_25',
        # 'gs://beam-core-outputs/sfbay-tr-speed0_5-20230626/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'TRspeed-0_5',
        # 'gs://beam-core-outputs/sfbay-tr-speed1_5-20230623/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'TRspeed-1_5',
        # 'gs://beam-core-outputs/sfbay-tr-speed2_0-20230623/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'TRspeed-2_0',
        # 'gs://beam-core-outputs/sfbay-tr-discount-100-20230703/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'TRdisc-100',
        # 'gs://beam-core-outputs/sfbay-tr-discount-75-20230703/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'TRdisc-75',
        # 'gs://beam-core-outputs/sfbay-tr-discount-50-20230703/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'TRdisc-50',
        # 'gs://beam-core-outputs/sfbay-tr-discount-25-20230703/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'TRdisc-25',
        # 'gs://beam-core-outputs/sfbay-fleetsz_0_125-20230609/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'RHfltsz-0_125',
        # 'gs://beam-core-outputs/sfbay-fleetsz_0_5-20230607/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'RHfltsz-0_5',
        # 'gs://beam-core-outputs/sfbay-fleetsz_3-20230607/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'RHfltsz-3',
        # 'gs://beam-core-outputs/sfbay-fleetsz_10-20230607/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'RHfltsz-10',
        # 'gs://beam-core-outputs/sfbay-rh-detour0125-20230623/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'RHdetour-0_125',
        # 'gs://beam-core-outputs/sfbay-rh-detour05-20230623/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'RHdetour-0_5',
        # 'gs://beam-core-outputs/sfbay-rh-detour3-20230623/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'RHdetour-3',
        # 'gs://beam-core-outputs/sfbay-rh-detour10-20230623/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'RHdetour-10',
        # 'gs://beam-core-outputs/sfbay-wb-incentives-25-20230705/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'WBincent-25',
        # 'gs://beam-core-outputs/sfbay-wb-incentives-50-20230703/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'WBincent-50',
        # 'gs://beam-core-outputs/sfbay-wb-incentives-100-20230630/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'WBincent-100',
        # 'gs://beam-core-outputs/sfbay-wb-incentives-200-20230630/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'WBincent-200',
        # 'gs://beam-core-outputs/sfbay-telecommuting-baseline-20230616/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Baseline',
        # 'gs://beam-core-outputs/sfbay-telecommuting-0p36-20230620/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Tele-036',
        # 'gs://beam-core-outputs/sfbay-telecommuting-0p76-20230621/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Tele-076',
        # 'gs://beam-core-outputs/sfbay-telecommuting-1p56-20230617/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Tele-156',
        # 'gs://beam-core-outputs/sfbay-telecommuting-3p32-20230621/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Tele-332',
        # 'gs://beam-core-outputs/sfbay-telecommuting-5p24-20230617/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Tele-524',
        # 'gs://beam-core-outputs/sfbay-telecommuting-8p60-20230620/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'Tele-860'
        # 'gs://beam-core-outputs/sfbay-cordon-MTA-20230703/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'MTA-Cordon',
        # 'gs://beam-core-outputs/sfbay-cordon-MTA-nobeamtoll-20230703/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz':'MTA-Cordon-noBEAM'
        
        
    # 'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-11_01-22-35_vjo/ITERS/it.0/0.events.csv.gz': '0.3pop-baseline',
    # 'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-11_01-22-34_ufh/ITERS/it.0/0.events.csv.gz': '0.3pop-fewerdocks',
    # 'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-11_01-22-57_uyl/ITERS/it.0/0.events.csv.gz': '0.3pop-moredocks',
    # 'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-11_01-22-47_grg/ITERS/it.0/0.events.csv.gz': '0.3pop-fewerbikes',
#     'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-11_01-23-11_avr/ITERS/it.0/0.events.csv.gz': '0.3pop-morebikes',
#     'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-11_07-19-25_ioy/ITERS/it.0/0.events.csv.gz': '0.3pop-nocar400',
#     'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-11_07-19-25_xiw/ITERS/it.0/0.events.csv.gz': '0.3pop-nocar1600',
#     'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-11_07-44-20_wkt/ITERS/it.0/0.events.csv.gz': '0.3pop-cheaperbikes',
#     'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-11_07-58-25_dst/ITERS/it.0/0.events.csv.gz': '0.3pop-nosharedbikes',
#     'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-11_07-50-06_dta/ITERS/it.0/0.events.csv.gz': '0.3pop-freebikes',
#     'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-11_01-36-53_pkp/ITERS/it.0/0.events.csv.gz': '0.3pop-twicebikes',
#     'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-11_02-42-45_unm/ITERS/it.0/0.events.csv.gz': '0.3pop-twicebikes-nocar400',
#     'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-11_08-08-48_htd/ITERS/it.0/0.events.csv.gz': '0.3pop-twicebikes-nocar1600',
        

    # 'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-07-12_05-21-47_yib/ITERS/it.0/0.events.csv.gz': '0.1pop-neweline',


#     'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-11_17-14-51_fdd/ITERS/it.0/0.events.csv.gz': '0.1pop-baseline',
#     'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-11_17-14-40_xaa/ITERS/it.0/0.events.csv.gz': '0.1pop-fewerdocks',
#     'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-11_17-14-39_wfo/ITERS/it.0/0.events.csv.gz': '0.1pop-moredocks',
#     'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-11_17-14-52_kok/ITERS/it.0/0.events.csv.gz': '0.1pop-fewerbikes',
#     'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-11_17-14-52_gvj/ITERS/it.0/0.events.csv.gz': '0.1pop-morebikes',
#     'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-05_04-21-06_quo/ITERS/it.0/0.events.csv.gz': '0.1pop-nocar400',
#     'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-05_04-21-45_iev/ITERS/it.0/0.events.csv.gz': '0.1pop-nocar1600',
#     'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-05_06-15-59_hov/ITERS/it.0/0.events.csv.gz': '0.1pop-cheaperbikes',
#     'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-11_23-12-58_mii/ITERS/it.0/0.events.csv.gz': '0.1pop-noshared',
#     'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-05_06-39-27_kjk/ITERS/it.0/0.events.csv.gz': '0.1pop-freebikes',
#     'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-11_17-14-54_mkb/ITERS/it.0/0.events.csv.gz': '0.1pop-twicebikes',
#     'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-05_06-39-50_iqq/ITERS/it.0/0.events.csv.gz': '0.1pop-twicebikes-nocar400',
#     'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-05_06-39-27_urc/ITERS/it.0/0.events.csv.gz': '0.1pop-twicebikes-nocar1600',
    
        # 'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-06-30_14-55-00_qsk/ITERS/it.0/0.events.csv.gz': '0.1pop-baseline-highBaseCost',

    # 'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-07-16_02-25-59_jqi/ITERS/it.0/0.events.csv.gz': '0.3pop-baseline',
    # 'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-07-16_02-27-49_znc/ITERS/it.0/0.events.csv.gz': '0.3pop-fewerdocks',
    # 'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-07-16_02-29-11_ybc/ITERS/it.0/0.events.csv.gz': '0.3pop-moredocks',
    # 'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-07-16_21-01-43_vnv/ITERS/it.0/0.events.csv.gz': '0.3pop-fewerbikes',
    # 'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-07-16_08-13-08_rmx/ITERS/it.0/0.events.csv.gz': '0.3pop-morebikes',
    # 'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-07-16_08-57-44_qlv/ITERS/it.0/0.events.csv.gz': '0.3pop-nocar400',
    # 'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-07-16_02-29-41_jwh/ITERS/it.0/0.events.csv.gz': '0.3pop-nocar1600',
    # 'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-07-16_14-44-12_hwy/ITERS/it.0/0.events.csv.gz': '0.3pop-cheaperbikes',
    # 'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-07-16_07-45-36_aep/ITERS/it.0/0.events.csv.gz': '0.3pop-noshared',
    # 'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-07-16_21-09-07_xhl/ITERS/it.0/0.events.csv.gz': '0.3pop-freebikes',
    # 'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-07-16_08-46-59_nmr/ITERS/it.0/0.events.csv.gz': '0.3pop-twicebikes',
    # 'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-07-16_15-23-32_bmh/ITERS/it.0/0.events.csv.gz': '0.3pop-twicebikes-nocar400',
    # 'gs://beam-core-outputs/output/sfbay/sfbay-pilates-base__2023-07-16_15-23-42_omr/ITERS/it.0/0.events.csv.gz': '0.3pop-twicebikes-nocar1600',

        

    }
    data_names = data.keys()
    plan_names = []
    for data_name in data_names:
        plan_names.append(data_name[:-13]+'plans.csv.gz')

    names = data.values()

##########################################GOOGLE CLOUD#############################################################################
# fp = ""
# output_nm = 'SummaryTable_Pilates_GoogleCloud2.csv'
# len_id_transit = 3
# fp_res = 'outputs/'
# is_NYC = False
# is_LT = False
# is_WC = True
# is_GEO = False
# is_collect_data = False
# is_plot = False

# data = {               's3://beam-outputs/pilates-outputs/sfbay-base-20220409/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'Baseline/year-2018-iteration-5',
#                 'gs://beam-core-outputs/sfbay-2010-base-20220810/beam/year-2011-iteration-3/ITERS/it.0/0.events.csv.gz':'Baseline - google cloud - year 2011 iter 3',
#                 'gs://beam-core-outputs/fbay-2010-TR-20220810/beam/year-2011-iteration-3/ITERS/it.0/0.events.csv.gz':'TR - google cloud - year 2010 iter 3',

#                 }
# data_names = data.keys()
# plan_names = []
# for data_name in data_names:
#     plan_names.append(data_name[:-13]+'plans.csv.gz')

# names = data.values()

#######################################################################################################################
# Transi Rich Scenarios
#######################################################################################################################
# fp = ""
# output_nm = 'SummaryTable_Pilates_TR.csv'
# len_id_transit = 3
# fp_res = 'outputs/'
# is_NYC = False
# is_WC = False
# is_LT = False
# is_GEO = False
# is_collect_data = True
# is_plot = False

# data = {
                # 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220816/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'Baseline 2018',
                # 's3://beam-outputs/pilates-outputs/sfbay-TR-20220812/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'Future 2018',
                #     's3://beam-outputs/pilates-outputs/sfbay-baseline-20220816/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz':'Baseline 2019',
                # 's3://beam-outputs/pilates-outputs/sfbay-TR-20220812/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz':'Future 2019',
                #     's3://beam-outputs/pilates-outputs/sfbay-baseline-20220816/beam/year-2020-iteration-5/ITERS/it.0/0.events.csv.gz':'Baseline 2020',
                # 's3://beam-outputs/pilates-outputs/sfbay-TR-20220812/beam/year-2020-iteration-5/ITERS/it.0/0.events.csv.gz':'Future 2020',
#                     's3://beam-outputs/pilates-outputs/sfbay-baseline-20220816/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz':'Baseline 2021',
#                 's3://beam-outputs/pilates-outputs/sfbay-TR-20220812/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz':'Future 2021',
#             
    # 's3://beam-outputs/pilates-outputs/sfbay_baseline_20230224/beam/year-2018-iteration-1/ITERS/it.0/0.events.csv.gz':'BS 2018-1',
    # 's3://beam-outputs/pilates-outputs/sfbay_baseline_TR_20230224/beam/year-2018-iteration-1/ITERS/it.0/0.events.csv.gz':'TR 2018-1',
    # 's3://beam-outputs/pilates-outputs/sfbay_baseline_20230224/beam/year-2018-iteration-2/ITERS/it.0/0.events.csv.gz':'BS 2018-2',
    # 's3://beam-outputs/pilates-outputs/sfbay_baseline_TR_20230224/beam/year-2018-iteration-2/ITERS/it.0/0.events.csv.gz':'TR 2018-2',
    # 's3://beam-outputs/pilates-outputs/sfbay_baseline_20230224/beam/year-2018-iteration-3/ITERS/it.0/0.events.csv.gz':'BS 2018-3',
    # 's3://beam-outputs/pilates-outputs/sfbay_baseline_TR_20230224/beam/year-2018-iteration-3/ITERS/it.0/0.events.csv.gz':'TR 2018-3',
    # 's3://beam-outputs/pilates-outputs/sfbay_baseline_20230224/beam/year-2018-iteration-4/ITERS/it.0/0.events.csv.gz':'BS 2018-4',
    # 's3://beam-outputs/pilates-outputs/sfbay_baseline_TR_20230224/beam/year-2018-iteration-4/ITERS/it.0/0.events.csv.gz':'TR 2018-4',
    # 's3://beam-outputs/pilates-outputs/sfbay_baseline_20230224/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'BS 2018-5',
    # 's3://beam-outputs/pilates-outputs/sfbay_baseline_TR_20230224/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz':'TR 2018-5',
    # 's3://beam-outputs/pilates-outputs/sfbay_baseline_20230224/beam/year-2018-iteration-6/ITERS/it.0/0.events.csv.gz':'BS 2018-6',
    # 's3://beam-outputs/pilates-outputs/sfbay_baseline_TR_20230224/beam/year-2018-iteration-6/ITERS/it.0/0.events.csv.gz':'TR 2018-6',
        
    # 's3://beam-outputs/pilates-outputs/sfbay_baseline_20230224/beam/year-2019-iteration-1/ITERS/it.0/0.events.csv.gz':'BS 2019-1',
    # 's3://beam-outputs/pilates-outputs/sfbay_baseline_TR_20230224/beam/year-2019-iteration-1/ITERS/it.0/0.events.csv.gz':'TR 2019-1',
    # 's3://beam-outputs/pilates-outputs/sfbay_baseline_20230224/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'BS 2019-2',
    # 's3://beam-outputs/pilates-outputs/sfbay_baseline_TR_20230224/beam/year-2019-iteration-2/ITERS/it.0/0.events.csv.gz':'TR 2019-2',
    # 's3://beam-outputs/pilates-outputs/sfbay_baseline_20230224/beam/year-2019-iteration-3/ITERS/it.0/0.events.csv.gz':'BS 2019-3',
    # 's3://beam-outputs/pilates-outputs/sfbay_baseline_TR_20230224/beam/year-2019-iteration-3/ITERS/it.0/0.events.csv.gz':'TR 2019-3',
    # 's3://beam-outputs/pilates-outputs/sfbay_baseline_20230224/beam/year-2019-iteration-4/ITERS/it.0/0.events.csv.gz':'BS 2019-4',
    # 's3://beam-outputs/pilates-outputs/sfbay_baseline_TR_20230224/beam/year-2019-iteration-4/ITERS/it.0/0.events.csv.gz':'TR 2019-4',
    # 's3://beam-outputs/pilates-outputs/sfbay_baseline_20230224/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz':'BS 2019-5',
    # 's3://beam-outputs/pilates-outputs/sfbay_baseline_TR_20230224/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz':'TR 2019-5',
#     's3://beam-outputs/pilates-outputs/sfbay_baseline_20230224/beam/year-2019-iteration-6/ITERS/it.0/0.events.csv.gz':'BS 2019-6',
#     's3://beam-outputs/pilates-outputs/sfbay_baseline_TR_20230224/beam/year-2019-iteration-6/ITERS/it.0/0.events.csv.gz':'TR 2019-6',
        
# }
                    
# data_names = data.keys()
# plan_names = []
# for data_name in data_names:
#     plan_names.append(data_name[:-13]+'plans.csv.gz')

# names = data.values()
#######################################################################################################################

if is_GEO:
    BGs = gpd.read_file('inputs/641aa0d4-ce5b-4a81-9c30-8790c4ab8cfb202047-1-wkkklf.j5ouj.shp')


# Nomenclature

# mode share per distance
binnames = {1: "0-1", 2: "1-2", 3: "2-5", 4: "5-15", 5: "15+"}
distanceBinsMiles = np.array([0, 1, 2, 5, 15, 500])
    
PTsColumns = [
    'vehicle','time','type','mode','length','vehicleType','arrivalTime','departureTime',
        'capacity','secondaryFuel','primaryFuelType','secondaryFuelType','numPassengers','primaryFuel', 'startX', 'startY', 'endX', 'endY'
        ]
if is_LT:
    PTsColumns.append('links')
    
MCsColumns = ['person','time','type','mode','length', 'legModes']
MCsColumns_no_leg_modes = ['person','time','type','mode','length']


summaryTable = pd.DataFrame()

PTsModes = np.array(['walk','bike','bike_Sharing','bike_Sharing_empty','car','car_emer','car_hov2_emer','car_hov3_emer','car_RideHail','car_RideHail_empty','car_RideHail_WC','car_RideHail_WC_empty',
                               'car_CAV','car_hov2','car_hov3','bus','tram','rail','subway',
                               'cable_car','ferry','bus_empty','tram_empty','rail_empty',
                               'subway_empty','cable_car_empty','ferry_empty'])

PTsModesNames = ['Walk','Bike','Bike Sharing','Empty Bike Sharing','Car','Car Emergency','Car HOV2 Emergency','Car HOV3 Emergency','Ride Hail','Empty Ride Hail','Ride Hail WC','Empty Ride Hail WC',
                           'CAV','Car HOV2','Car HOV3','Bus','Tram','Rail','Subway',
                           'Cable Car','Ferry','Empty Bus','Empty Tram','Empty Rail',
                           'Empty Subway','Empty Cable Car','Empty Ferry',]

transit_modes = ['bus', 'subway', 'tram', 'rail','cable_car', 'ferry']

transit_MCmodes = ['bus', 'subway', 'tram', 'rail', 'walk_transit', 'ride_hail_transit',
                            'drive_transit', 'cable_car','bike_transit']

MCsModes = np.array([ 'bus', 'subway', 'tram', 'rail', 'car', 'hov3_teleportation', 
                            'bike', 'hov2_teleportation', 'walk', 'car_hov2', 'car_hov3', 
                            'walk_transit', 'ride_hail', 'ride_hail_transit', 'ride_hail_pooled', 
                            'drive_transit', 'cable_car','bike_transit'])

MCsModesNames = [ 'Bus', 'Subway', 'Tram', 'Rail', 'Car', 'HOV3 Passenger', 'Bike', 
                   'HOV2 Passenger', 'Walk', 'HOV2 Driver', 'HOV3 Driver', 
                   'Walk-Transit', 'Ride Hail', 'Ride Hail-Transit', 'Ride Hail Pooled', 
                   'Drive-Transit', 'Cable Car', 'Bike-Transit']

primaryFuelTypes =['Biodiesel','Diesel','Gasoline','Electricity','Food']

CtV = {0:'SUM',
      1:'AV',
      2:'VAR',
      3:'STD',
      4:'MIN',
      5:'Q1ST',
      6:'Q2ND',
      7:'Q3RD',
      8:'MAX',
      }

route_e_correction_cv = 1.1660
route_e_correction_hev = 1.1252
route_e_correction_ev = 1.3958


# Regions: {'SF': [,,,,],
#           'Cordon':[,,,,]}

In [4]:
start = time.time()
print('Start time',start, 's')

Start time 1689882680.547969 s


In [5]:
# def filter_by_region(region_filter, MCs, PTs, PEVs, PLVs, RPs, trips, PtoPTss):
    
    
    
    

In [6]:
def DA(data, code):
    
    #data is a list of values
    #code - operation
    #0 - Sum
    #1 - Mean
    #2 - Var
    #3 - Std error
    #4 - Min
    #5 - Q1
    #6 - Q2
    #7 - Q3
    #8 - Max
    if len(data)>0:
        if code == 0:
            value = np.sum(data)
        elif code == 1:
            value = np.mean(data)
        elif code == 2:
            value = np.var(data)
        elif code == 3:
            value = np.std(data)
        elif code == 4:
            value = np.min(data)
        elif code == 5:
            value = np.percentile(data, 25)
        elif code == 6:
            value = np.percentile(data, 50)
        elif code == 7:
            value = np.percentile(data, 75)
        elif code == 8:
            value = np.max(data)
    else:
        value = np.nan

        
    return value

In [7]:
def personToPathTraversal(PTs, PEVs, PLVs, personToTripDeparture):
    print('personToPathTraversal...')
    no_legs_after_time_check = []
    no_legs = []
    start = time.time()
    for pt_mode in PTs['mode'].value_counts().keys():
        if pt_mode == 'car_hov2':
            print('expected PtoPTs from occupancy ',pt_mode,  int(np.sum(PTs[PTs['mode']==pt_mode]['occupancy']))/2)
        elif pt_mode == 'car_hov3':
            print('expected PtoPTs from occupancy ',pt_mode,  int(np.sum(PTs[PTs['mode']==pt_mode]['occupancy']))/3)
        else:
            print('expected PtoPTs from occupancy ',pt_mode,  int(np.sum(PTs[PTs['mode']==pt_mode]['occupancy'])))

    print('expected PtoPTs from occupancy TOT = ', (np.sum(PTs['occupancy'])-
          np.sum(PTs[PTs['mode']=='car_hov2']['occupancy'])/2-
          np.sum(PTs[PTs['mode']=='car_hov3']['occupancy'])/3*2)
         )
    print('len of PEV = ', len(PEVs))
    print('len of PLV = ', len(PLVs))

    vehicleToPT = PTs.groupby('vehicle').apply(lambda x: list(x.index)).apply(
        lambda x: {y: [] for y in x}).to_dict()
    PEVlookup = PEVs[['person', 'vehicle', 'time']].value_counts().to_dict()
    PLVlookup = PLVs.groupby(['person', 'vehicle']).apply(lambda x: list(x.time)).to_dict()

    for key, counts in PEVlookup.items():
        person = key[0]
        vehicle = key[1]
        departureTime = key[2]
        n_new_leg = 0
        if vehicle in vehicleToPT:
            legs = vehicleToPT[vehicle]
            if (person, vehicle) in PLVlookup:
                if person in personToTripDeparture:
                    planTrips = personToTripDeparture[person]
                    tripsLeavingBeforeDeparture = [-1] + [t['planID'] for t in planTrips if
                                                          t['departureTime'] <= (departureTime+1)]
                                                                                     #+ 1800)]
                else:
                    tripsLeavingBeforeDeparture = [-1]
                    print('no person',person, 'on personToTripDeparture...','vehicle',vehicle,'departureTime',departureTime,'personToTripDeparture',personToTripDeparture[person])

                lastTripBeforeDeparture = tripsLeavingBeforeDeparture[-1]
                if lastTripBeforeDeparture == -1:
                    print('hmm...if person on personToTripDeparture, no plans starting before departure')
                
                endTimes = PLVlookup[(person, vehicle)]
                plvsAfterDeparture = [t for t in endTimes if t > departureTime]
                if len(plvsAfterDeparture) > 0:
                    firstPLVafterDeparture = plvsAfterDeparture[0]
                    n_new_leg = 0
                    for leg in legs.keys():
                        ptDepartureTime = PTs.at[leg, 'departureTime']
                        if (ptDepartureTime >= departureTime) & (ptDepartureTime < firstPLVafterDeparture):
                            n_new_leg +=1
                            legs[leg].append((person, lastTripBeforeDeparture))
                            
                else:
                    for leg in legs.keys():
                        n_new_leg = 0
                        ptDepartureTime = PTs.at[leg, 'departureTime']
                        if ptDepartureTime >= departureTime:
                            n_new_leg +=1
                            legs[leg].append((person, lastTripBeforeDeparture))
            else:

                if person in personToTripDeparture:
                    planTrips = personToTripDeparture[person]
                    tripsLeavingBeforeDeparture = [-1] + [t['planID'] for t in planTrips if
                                                          t['departureTime'] <= (departureTime+1)]
                                                                                     #+ 1800)]
                else:
                    tripsLeavingBeforeDeparture = [-1]
                    print('no person',person, 'on personToTripDeparture...','vehicle',vehicle,'departureTime',departureTime,'personToTripDeparture',personToTripDeparture[person])

                lastTripBeforeDeparture = tripsLeavingBeforeDeparture[-1]
                if lastTripBeforeDeparture == -1:
                    print('hmm...if person on personToTripDeparture, no plans starting before departure')
                n_new_leg = 0
                for leg in legs.keys():
                    ptDepartureTime = PTs.at[leg, 'departureTime']
                    if ptDepartureTime >= departureTime:
                        n_new_leg +=1
                        legs[leg].append((person, lastTripBeforeDeparture))
            if n_new_leg == 0:
                no_legs_after_time_check.append(vehicle)
#                 print("Warning: no vehicle legs (after time check) for person, vehicle, depTime", person, vehicle, departureTime)
        else:
                no_legs.append(vehicle)
#             print("Warning: no vehicle legs for person, vehicle, depTime", person, vehicle, departureTime)

    PtoPTssList = [(veh, pathTraversalID, passenger, planIndex) for veh, vehicleLegs in
                                vehicleToPT.items() for pathTraversalID, passengers in vehicleLegs.items() for
                                (passenger, planIndex) in passengers if len(passengers) > 0]
    PtoPTss = pd.MultiIndex.from_tuples(PtoPTssList,
                                                     name=['vehicleID', 'pathTraversalID', 'personID',
                                                           'planIndex']).to_frame()
    modes_PtoPTss = []
    lengths_PtoPTss = []
    durations_PtoPTss = []
    prim_fuel_type_PtoPTss = []
    prim_fuel_PtoPTss = []
    sec_fuel_PtoPTss = []
    
    if is_GEO:
        blocks_start = []
        blocks_end = []
    if is_LT:
        links_PtoPTss = []
        depTime_PtoPTss = []
        arrTime_PtoPTss = []
        # print('len PtoPTss before remove nan in links', len(PtoPTss))
        # PtoPTss.dropna(subset=['links'])
        # print('len PtoPTss after remove nan in links', len(PtoPTss))
    for pt_id in PtoPTss['pathTraversalID']:
        modes_PtoPTss.append(PTs.at[pt_id,'mode'])
        lengths_PtoPTss.append(PTs.at[pt_id,'length'])
        durations_PtoPTss.append(PTs.at[pt_id,'duration'])
        prim_fuel_type_PtoPTss.append(PTs.at[pt_id,'primaryFuelType'])
        prim_fuel_PtoPTss.append(PTs.at[pt_id,'primaryFuel'])
        sec_fuel_PtoPTss.append(PTs.at[pt_id,'secondaryFuel'])
        if is_GEO:
            blocks_start.append(PTs.at[pt_id,'startBlockGroup'])
            blocks_end.append(PTs.at[pt_id,'endBlockGroup'])
        if is_LT:
            depTime_PtoPTss.append(PTs.at[pt_id,'departureTime'])
            arrTime_PtoPTss.append(PTs.at[pt_id,'arrivalTime'])
            links_PtoPTss.append(PTs.at[pt_id,'links'])

    PtoPTss['mode'] = modes_PtoPTss
    PtoPTss['length'] = lengths_PtoPTss
    PtoPTss['duration'] = durations_PtoPTss
    PtoPTss['primaryFuelType'] = prim_fuel_type_PtoPTss
    PtoPTss['primaryFuel'] = prim_fuel_PtoPTss
    PtoPTss['secondaryFuel'] = sec_fuel_PtoPTss
    
    if is_GEO:
        PtoPTss['startBlockGroup'] = blocks_start
        PtoPTss['endBlockGroup'] = blocks_end

    if is_LT:
        PtoPTss['links'] = links_PtoPTss
        PtoPTss['arrivalTime'] = depTime_PtoPTss
        PtoPTss['departureTime'] = arrTime_PtoPTss

    vehicles_2 = []
    for pt_id in PtoPTss['pathTraversalID']:
        vehicles_2.append(PTs.at[pt_id,'vehicle'][:len_id_transit])
    vehicles_2 = np.array(vehicles_2)
    PtoPTss['vehicle2'] = vehicles_2
    
    modes,counts = np.unique(modes_PtoPTss, return_counts = True)
    for mode, count in zip(modes, counts):
        print('len PtoPTs after matching agents and vehicles',mode,count)
        
        
    print('no legs found:', len(no_legs))
    print('no legs found after time check:', len(no_legs_after_time_check))
    print('no vehicle body found, probably because with zero duration (discarded by PT):', 
          len(list(filter(lambda k: 'body' in k, no_legs))))
    print('no vehicle body found after time check, probably because with zero duration (discarded by PT):', 
          len(list(filter(lambda k: 'body' in k, no_legs_after_time_check))))

    print('Tot created PtoPTs = ', len(PtoPTss))
    PtoPTss.index = range(len(PtoPTss))
    print('Total time:', time.time()-start)
    return PtoPTss

In [8]:
def updateTransitMode(MC):
    transit = MC['mode'] == 'walk_transit'
    try:
        railTrips = transit & MC['legModes'].str.contains("RAIL")
        ferryTrips = transit & ~railTrips & MC['legModes'].str.contains("FERRY")
        subwayTrips = transit & ~railTrips & ~ferryTrips & MC['legModes'].str.contains("SUBWAY")
        busTrips = transit & ~railTrips & ~ferryTrips & ~subwayTrips
        MC.loc[subwayTrips, 'mode'] = "subway"
        MC.loc[railTrips, 'mode'] = "rail"
        MC.loc[ferryTrips, 'mode'] = "ferry"
        MC.loc[busTrips, 'mode'] = "bus"
    except:
        print('Warning: probably no legModes')

    MC['miles'] = MC['length'] / 1609.34
    MC['distBin'] = [binnames.get(val, 'Other') for val in np.digitize(MC['miles'], distanceBinsMiles)]
    return MC

In [9]:
def processPlans(directory):
    #fullPath = directory + 'plans.csv.gz'
    print('processPlans...')
    start = time.time()
    trips = []
    activities = []
    personToTripDeparture = {}
    print(directory)
    df = pd.read_csv(directory, nrows = None)
    # print(df.keys())
    df = df[df['planSelected']==True]
    # print(df[df['personId']==194])
    df = addTimesToPlans(df)
    legs = df.loc[(df['planElementType'].str.lower().str.contains('leg'))].dropna(how='all', axis=1)
    legs = (legs[legs['legDepartureTime']>=0])
    # print(legs.keys())
    # print(legs)
    try:
        legsSub = legs[['personId', 'legDepartureTime',  'planElementIndex', 'legMode', 'originX', 'originY', 'destinationX', 'destinationY']]
        is_leg_mode = True
    except:
        legsSub = legs[['personId', 'legDepartureTime',  'planElementIndex', 'originX', 'originY', 'destinationX', 'destinationY']]
        is_leg_mode = False

    for rowID, val in legsSub.iterrows():
        personToTripDeparture.setdefault(val.personId, []).append(
            {"planID": val.planElementIndex, "departureTime": val.legDepartureTime})
    #TRIPS
    trips.append(legsSub)
    #ACTS
    acts = df.loc[(df['planElementType'].str.lower().str.contains('activity'))].dropna(how='all', axis=1)
    actsSub = acts[['personId', 'activityType', 'activityLocationX', 'activityLocationY', 'activityEndTime']]
    activities.append(actsSub)
    print('Total time:', time.time()-start)
    return pd.concat(trips), pd.concat(activities), personToTripDeparture, is_leg_mode

In [10]:
def addTimesToPlans(plans):
    print('addTimesToPlans...')
    start = time.time()
    legInds = np.where(plans['planElementType'].str.lower() == "leg")[0]
    plans.loc[:, 'legDepartureTime'] = np.nan
    plans.iloc[legInds, plans.columns.get_loc('legDepartureTime')] = plans['activityEndTime'].iloc[legInds - 1].copy()
    plans.loc[:, 'originX'] = np.nan
    plans.iloc[legInds, plans.columns.get_loc('originX')] = plans['activityLocationX'].iloc[legInds - 1].copy()
    plans.loc[:, 'originY'] = np.nan
    plans.iloc[legInds, plans.columns.get_loc('originY')] = plans['activityLocationY'].iloc[legInds - 1].copy()
    plans.loc[:, 'destinationX'] = np.nan
    plans.iloc[legInds, plans.columns.get_loc('destinationX')] = plans['activityLocationX'].iloc[legInds + 1].copy()
    plans.loc[:, 'destinationY'] = np.nan
    plans.iloc[legInds, plans.columns.get_loc('destinationY')] = plans['activityLocationY'].iloc[legInds + 1].copy()
    print('Total time:', time.time()-start)
    return plans

In [11]:
def readEvents(directory):
    #fullPath = directory + 'ITERS/it.0/0.events.csv.gz'
    PTs = []
    PEVs = []
    PLVs = []
    MCs = []
    RPs = []
    ASs = []

    print('Reading ', directory)
    for chunk in pd.read_csv(directory, chunksize=2500000, nrows = nrows):
        if sum((chunk['type'] == 'PathTraversal')) > 0:
            chunk['vehicle'] = chunk['vehicle'].astype(str)
            
            #PT
            print(len(chunk.loc[(chunk['type'] == 'PathTraversal')]),': len chunk PT')
            PT = chunk.loc[(chunk['type'] == 'PathTraversal') & (chunk['length'] > 0)].dropna(how='all', axis=1)
            PT['links'] =  PT['links'].fillna('0')
            PT['departureTime'] = PT['departureTime'].astype(int)
            PT['arrivalTime'] = PT['arrivalTime'].astype(int)
            PTs.append(PT[PTsColumns])
            print(len(PT),': after filtering zero-length PT')
            #PEV
            print(len(chunk.loc[(chunk['type'] == 'PersonEntersVehicle')]),': len chunk PEV')
#             PEV = chunk.loc[(chunk.type == "PersonEntersVehicle") & 
#                             ~(chunk['person'].apply(str).str.contains('Agent').fillna(False)) & 
#                             ~(chunk['vehicle'].str.contains('body').fillna(False)), :].dropna(how='all', axis=1)
            PEV = chunk.loc[(chunk.type == "PersonEntersVehicle") &
                            ~(chunk['person'].apply(str).str.contains('Agent').fillna(False))
                            , :].dropna(how='all', axis=1)
            print(len(PEV),': after filtering drivers')
                                                                                                                    
            if len(PEV)>0:
                PEV['person'] = PEV['person'].astype(int)
                PEV['time'] = PEV['time'].astype(int)
                PEVs.append(PEV)

            #PLV
#             PLV = chunk.loc[(chunk.type == "PersonLeavesVehicle") & 
#                             ~(chunk['person'].apply(str).str.contains('Agent').fillna(False)) & 
#                             ~(chunk['vehicle'].str.contains('body').fillna(False)), :].dropna(how='all', axis=1)
            print(len(chunk.loc[(chunk['type'] == 'PersonLeavesVehicle')]),': len chunk PLV')
            PLV = chunk.loc[(chunk.type == "PersonLeavesVehicle") &
                            ~(chunk['person'].apply(str).str.contains('Agent').fillna(False))
                            , :].dropna(how='all', axis=1) 
            print(len(PLV),': after filtering drivers')
            if len(PLV)>0:
                PLV['person'] = PLV['person'].astype(int)
                PLV['time'] = PLV['time'].astype(int)
                PLVs.append(PLV)
        if sum((chunk['type'] == 'ModeChoice')) > 0:
            #MC
            MC = chunk.loc[(chunk['type'] == 'ModeChoice') & (chunk['length'] > 0)].dropna(how='all', axis=1)
            try:
                MCs.append(MC[MCsColumns])
            except:
                MCs.append(MC[MCsColumns_no_leg_modes])
                print('WARNING: probably no legModes')
            
        if sum((chunk['type'] == 'Replanning')) > 0:
            #RP
            RP = chunk.loc[(chunk['type'] == 'Replanning')].dropna(how='all', axis=1)
            RPs.append(RP)
            
        if sum((chunk['type'] == 'actstart')) > 0:
            #RP
            AS = chunk.loc[(chunk['type'] == 'actstart')].dropna(how='all', axis=1)
            ASs.append(AS)
        
        # print(chunk['type'].value_counts())
    print(len(pd.concat(PEVs)),':len PEVs')
    print(len(pd.concat(PLVs)),':len PLVs')
    
    
    PEVs = pd.concat(PEVs)
    PLVs = pd.concat(PLVs)
    PTs = pd.concat(PTs)
    MCs = pd.concat(MCs)
    RPs = pd.concat(RPs)
    ASs = pd.concat(ASs)

    
    print(len(PTs),':len PTs')
    print(len(MCs),':len MCs')
    print(len(RPs),':len RPs')
    print(len(ASs),':len ASs')

    
    return MCs, PTs, PEVs, PLVs, RPs, ASs



In [12]:
def fixData(Mcs, PTs, PEVs, PLVs,len_id_transit):


    PTs['duration'] = PTs['arrivalTime'] - PTs['departureTime']
    PTs['gallonsGasoline'] = 0
    PTs.loc[PTs['primaryFuelType'] == 'Gasoline',
            'gallonsGasoline'] += PTs.loc[PTs['primaryFuelType'] == 'Gasoline', 'primaryFuel'] * 8.3141841e-9
    PTs.loc[PTs['secondaryFuelType'] == 'Gasoline',
            'gallonsGasoline'] += PTs.loc[PTs['secondaryFuelType'] == 'Gasoline', 'secondaryFuel'] * 8.3141841e-9
    PTs['occupancy'] = PTs['numPassengers']
    
    PTs['isCAV'] = PTs['vehicleType'].str.contains('L5')
    PTs['isRH'] = PTs['vehicle'].str.contains('rideHail')
    PTs['isBS'] = PTs['vehicle'].str.contains('bay_wheels')
    PTs['isRH_WC'] = PTs['vehicleType'].str.contains('RH_Car-wheelchair')
    PTs['is_empty'] = PTs['numPassengers'] == 0
    PTs['is_RHempty'] = PTs['isRH']*PTs['is_empty']
    PTs['is_car_emer'] = PTs['vehicle'].str.contains('emergency')
    
    PTs.loc[PTs['mode'] == 'car', 'capacity'] += 1
    PTs.loc[PTs['mode'] == 'car_hov2', 'capacity'] += 1
    PTs.loc[PTs['mode'] == 'car_hov3', 'capacity'] += 1
    PTs.loc[PTs['mode'] == 'walk', 'capacity'] = 1
    PTs.loc[PTs['mode'] == 'bike', 'capacity'] = 1
    
    PTs.loc[PTs['isRH'], 'mode'] += '_RideHail'
    PTs.loc[PTs['isBS'], 'mode'] += '_Sharing'
    PTs.loc[PTs['isRH_WC'], 'mode'] += '_WC'
    PTs.loc[PTs['isCAV'], 'mode'] += '_CAV'
    PTs.loc[PTs['is_RHempty'], 'mode'] += '_empty'
    PTs.loc[PTs['is_car_emer'], 'mode'] += '_emer'
    
    PTs.loc[PTs['mode'] == 'car', 'occupancy'] = 1
    PTs.loc[PTs['mode'] == 'car_emer', 'occupancy'] = 1
    PTs.loc[PTs['mode'] == 'car_hov2', 'occupancy'] += 1
    PTs.loc[PTs['mode'] == 'car_hov3', 'occupancy'] += 1
    PTs.loc[PTs['mode'] == 'car_hov3_emer', 'occupancy'] = 1
    PTs.loc[PTs['mode'] == 'walk', 'occupancy'] = 1
    PTs.loc[PTs['mode'] == 'bike', 'occupancy'] = 1

    PTs['vehicleMiles'] = PTs['length'] / 1609.34
    PTs['passengerMiles'] = (PTs['length'] * PTs['occupancy']) / 1609.34
    PTs['totalEnergyInJoules'] = PTs['primaryFuel'] + PTs['secondaryFuel']
    
    for tm in transit_modes:
        PTs['is'+tm] = PTs['mode'].str.contains(tm)
    for tm in transit_modes:
        PTs['is_'+tm+'_empty'] = PTs['is'+tm]*PTs['is_empty']
    PTs['is_transit'] = 0
    for tm in transit_modes:
        PTs['is_transit']+=PTs['is'+tm]
    for tm in transit_modes:
        PTs.loc[PTs['is_'+tm+'_empty'], 'mode'] += '_empty'
        PTs.drop(columns=['is'+tm])
        PTs.drop(columns=['is_'+tm+'_empty'])
        
    PTs.drop(columns=['isCAV','is_empty','is_RHempty','isRH_WC','is_car_emer'])
    
    vehicles_2 = []
    vehicles = PTs['vehicle']
    for vehicle in vehicles:
        vehicles_2.append(vehicle[:len_id_transit])
    vehicles_2 = np.array(vehicles_2)
    PTs['vehicle2'] = vehicles_2
    
    vehicles_2 = []
    vehicles = PEVs['vehicle']
    for vehicle in vehicles:
        vehicles_2.append(vehicle[:len_id_transit])
    PEVs['vehicle2'] = vehicles_2
    
    vehicles_2 = []
    vehicles = PLVs['vehicle']
    for vehicle in vehicles:
        vehicles_2.append(vehicle[:len_id_transit])
    PLVs['vehicle2'] = vehicles_2
    
    
    if is_LT:
        links_PTs = []
        for links in PTs['links']:
            if ',' in str(links):
                links_PTs.append(links.split(','))
            else:
                links_PTs.append(['0'])
        PTs['links'] = links_PTs
    
    if is_GEO:
        PTs = addGeometryIdToDataFrame(PTs, BGs, 'startX', 'startY', 'startBlockGroup', df_geom='epsg:4326')
        PTs = addGeometryIdToDataFrame(PTs, BGs, 'endX', 'endY', 'endBlockGroup', df_geom='epsg:4326')
    
    return Mcs, PTs, PEVs, PLVs


In [13]:
def PLT_build(data_name,PtoPTss, network, link_taz):
    print('PLT..')
    link_map = {}
    PLT = pd.DataFrame()
    for link, length, fromx, tox, fromy, toy  in zip(network['linkId'],network['linkLength'],network['fromLocationX'],network['toLocationX'],
                                network['fromLocationY'],network['toLocationY']):
        link_map[link] = list([ int(length), fromx, tox, fromy, toy ])

    #link ID, person ID, vehicle ID, Time (av depart and arrive), link length, block ID, 
    #other link attributes, energy consumption (weighted based on link and trip lengths)
    # vehicleID	pathTraversalID	personID	planIndex	mode	length	duration	primaryFuelType	links	vehicle2
    print('len PTtoPTs = ', len(PtoPTss))
    len_links = 0
    for links  in PtoPTss['links']:
        if links != '0':
            len_links+=len(links)
    print('len linkstoP = ', len_links)
    linkss = list(np.zeros((len_links)))
    persons = list(np.zeros((len_links)))
    modes = list(np.zeros((len_links)))
    vehicles = list(np.zeros((len_links)))
    estTimes = list(np.zeros((len_links)))
    lengths = list(np.zeros((len_links)))
    xs_center = list(np.zeros((len_links)))
    ys_center = list(np.zeros((len_links)))
    estEnergies = list(np.zeros((len_links)))
    if is_GEO:
        tazs = list(np.zeros((len_links)))
    empty_links = 0
    i = -1
    for links, person, mode, arrTime, depTime, vehicle, primaryFuel, secondaryFuel, length  in zip(PtoPTss['links'],
                               PtoPTss['personID'],PtoPTss['mode'],PtoPTss['arrivalTime'],PtoPTss['departureTime'],
                               PtoPTss['vehicleID'],PtoPTss['primaryFuel'],PtoPTss['secondaryFuel'],PtoPTss['length']):
        
        
        if links != '0':
            for link in links:
                i+=1
                if i%1000000 == 0:
                    print(i,'/',len_links)
                try:
                    row = link_map[int(link)]
                    if is_GEO:
                        tazs[i] = link_taz[int(link)]
                    estEnergies[i] = row[0]/length*(primaryFuel+secondaryFuel)
                    lengths[i] = row[0]
                    xs_center[i] = np.mean([row[1],
                                                    row[2]])
                    ys_center[i] = np.mean([row[3],
                                                row[4]])
                except:
                    print('Warning:',link)
                    ys_center[i] = np.nan
                    xs_center[i] = np.nan
                    estEnergies[i] = np.nan
                    if is_GEO:
                        tazs[i] = np.nan
                linkss[i] = link
                persons[i] = person
                modes[i] = mode
                vehicles[i] = vehicle
                estTimes[i] = np.mean([arrTime,depTime])

        else:
            empty_links += 1
                
    PLT['linkId']=linkss
    PLT['personId']=persons
    PLT['mode']=modes
    PLT['vehicleID']=vehicles
    PLT['estTime']=estTimes
    PLT['estEnergies']=estEnergies
    PLT['length']=lengths
    PLT['X_center']=xs_center
    PLT['Y_center']=ys_center
    if is_GEO:
        PLT['BlockGroup']=tazs

    print( 'PtoPTs without link IDs:',   empty_links,'/', len(PtoPTss))

    print('len PLT = ', len(PLT))

    #link ID, vehicle ID,..
    # 'vehicle', 'time', 'type', 'mode', 'length', 'vehicleType',
    #    'arrivalTime', 'departureTime', 'capacity', 'secondaryFuel',
    #    'primaryFuelType', 'secondaryFuelType', 'numPassengers', 'primaryFuel',
    #    'links', 'duration', 'gallonsGasoline', 'occupancy', 'isCAV', 'isRH',
    #    'isRH_WC', 'is_empty', 'is_RHempty', 'is_car_emer', 'isbus', 'issubway',
    #    'istram', 'israil', 'iscable_car', 'isferry', 'is_bus_empty',
    #    'is_subway_empty', 'is_tram_empty', 'is_rail_empty',
    #    'is_cable_car_empty', 'is_ferry_empty', 'is_transit', 'vehicle2'
    # for links in zip(PTs['links'],PTs['vehicle'],PTs['links'],PTs['links'],PTs['links'],PTs['links'],):
    #     if links != '0':
    #         for link in links:
    #             pass
    #Aggregate: for each link guess VMT, PMT, Energy, Mode split and vehicle type split, speed

    # Aggregate per TAZ


    print("done")


    return PLT

In [14]:
def addGeometryIdToDataFrame(df, gdf, xcol, ycol, idColumn="geometry", df_geom='epsg:32610'): 
    gdf.crs = {'init': 'epsg:4326'}
    gdf_data = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[xcol], df[ycol]))
    gdf_data.crs = {'init': df_geom}
    joined = gpd.sjoin(gdf_data.to_crs('epsg:26910'), gdf.to_crs('epsg:26910'))
    gdf_data = gdf_data.merge(joined['blkgrpid'], left_index=True, right_index=True, how="left")
    gdf_data.rename(columns={'blkgrpid': idColumn}, inplace=True)
    df = pd.DataFrame(gdf_data.drop(columns='geometry'))
    # df.drop(columns=[xcol, ycol], inplace=True)
    print(len(df))
    print(len(df.loc[~df.index.duplicated(keep='first'), :]))

    return df.loc[~df.index.duplicated(keep='first'), :]

In [15]:
def link_TAZ_fun(network, link_stat):
    link_tazs = []
    for link in link_stat['link']:    
        link_tazs.append(list(network[network['linkId']==link]['BlockGroup'])[0])
    link_stat['BlockGroup']   =  link_tazs                                           
    return link_stat

In [16]:
def network_TAZ(network):
    network_x = []
    network_y = []
    link_taz = {}
    for link, linkfromx, linktox, linkfromy,linktoy in zip(network['linkId'],network['fromLocationX'],network['toLocationX'],
                                network['fromLocationY'],network['toLocationY']):    
        network_x.append(np.mean([linkfromx, linktox,]))
        network_y.append(np.mean([linkfromy, linktoy,]))
    network['X_center'] = network_x
    network['Y_center'] = network_y
    network = addGeometryIdToDataFrame(network, BGs, 'X_center', 'Y_center', 'BlockGroup')
    for link, block in zip(network['linkId'],network['BlockGroup']):
        link_taz[link] = block

                                                           
    return network, link_taz

In [17]:
def VLT_build(data_name,PTs, network, link_taz):
    print('VLT..')
    link_map = {}
    VLT = pd.DataFrame()
    
    for link, length, fromx, tox, fromy, toy  in zip(network['linkId'],network['linkLength'],network['fromLocationX'],network['toLocationX'],
                                network['fromLocationY'],network['toLocationY']):
        link_map[link] = list([int(length), fromx, tox, fromy, toy ])

    print('len PTs = ', len(PTs))
    len_links = 0
    for links  in PTs['links']:
        if links != '0':
            len_links+=len(links)
    print('len linkstoP = ', len_links)
    linkss = list(np.zeros((len_links)))
    modes = list(np.zeros((len_links)))
    vehicles = list(np.zeros((len_links)))
    estTimes = list(np.zeros((len_links)))
    lengths = list(np.zeros((len_links)))
    xs_center = list(np.zeros((len_links)))
    ys_center = list(np.zeros((len_links)))
    estEnergies = list(np.zeros((len_links)))
    if is_GEO:
        tazs = list(np.zeros((len_links)))
    empty_links = 0
    i = -1
    for links, mode, arrTime, depTime, vehicle, primaryFuel, secondaryFuel, length in zip(PTs['links'],
                                            PTs['mode'],
                                  PTs['arrivalTime'],PTs['departureTime'],PTs['vehicle']
                                                             ,PTs['primaryFuel'],PTs['secondaryFuel'],PTs['length'] ):
        if links != '0':
            for link in links:
                i+=1
                if i%1000000 == 0:
                    print(i,'/',len_links)
                try:
                    row = link_map[int(link)]
                    if is_GEO:
                        tazs[i] = link_taz[int(link)]
                    estEnergies[i] = row[0]/length*(primaryFuel+secondaryFuel)
                    lengths[i] = row[0]
                    xs_center[i] = np.mean([row[1],
                                                    row[2]])
                    ys_center[i] = np.mean([row[3],
                                                row[4]])
                except:
                    print('Warning:',link)
                    ys_center[i] = np.nan
                    xs_center[i] = np.nan
                    estEnergies[i] = np.nan
                    if is_GEO:
                        tazs[i] = np.nan
                linkss[i] = link
                modes[i] = mode
                vehicles[i] = vehicle
                estTimes[i] = np.mean([arrTime,depTime])


        else:
            empty_links += 1
    VLT['linkId']=linkss
    VLT['mode']=modes
    VLT['vehicleID']=vehicles
    VLT['estTime']=estTimes
    VLT['estEnergies']=estEnergies
    VLT['length']=lengths
    VLT['X_center']=xs_center
    VLT['Y_center']=ys_center
    if is_GEO:
        VLT['BlockGroup']=tazs

    print( 'PTs without link IDs:',   empty_links,'/', len(PTs))
    print('len VLT = ', len(VLT))

    print("done")

    return VLT

In [18]:
def SummaryTable(ST, data_name, name, plan_name, MCs, PTs, PEVs, PLVs, RPs, trips, PtoPTss, codes, transitCompanies,is_leg_mode):
    
#----------Indexes
    PTsModeIndexes = {}
    PTsTransitIndexes = {}
    PTsFuelIndexes = {}
    MCsModeIndexes = {}
    MCsReplanIndexes = {}
    MCsPlanIndexes = {}
    PtoPTssModeIndexes = {}
    PtoPTssTransitIndexes = {}
    PtoPTssFuelIndexes = {}
    #Replanning
    reasons = []
    for reason in RPs['reason']:
        reasons.append(reason.split()[1].lower())
    RPs['mode'] = reasons
    totalTrips_replan = len(RPs['mode'])
    
    for MCsMode, MCsModesName in zip(MCsModes, MCsModesNames):
        MCsReplanIndexes[MCsMode] = RPs[(RPs['mode'] == MCsMode)].index
    for PTsMode, PTsModesName in zip(PTsModes, PTsModesNames):
        PTsModeIndexes[PTsMode] = PTs[(PTs['mode'] == PTsMode)].index
        PtoPTssModeIndexes[PTsMode] = PtoPTss[(PtoPTss['mode'] == PTsMode)].index
    for company in transitCompanies:
        PTsTransitIndexes[company] = PTs[(PTs['vehicle2'] == company)].index
        PtoPTssTransitIndexes[company] = PtoPTss[(PtoPTss['vehicle2'] == company)].index
    for primaryFuelType in primaryFuelTypes:
        PTsFuelIndexes[primaryFuelType] = PTs[(PTs['primaryFuelType'] == primaryFuelType)].index
        PtoPTssFuelIndexes[primaryFuelType] = PtoPTss[(PtoPTss['primaryFuelType'] == primaryFuelType)].index
    for MCsMode, MCsModesName in zip(MCsModes, MCsModesNames):
        MCsModeIndexes[MCsMode] = MCs[(MCs['mode'] == MCsMode)].index
    if is_leg_mode:
        for MCsMode, MCsModesName in zip(MCsModes, MCsModesNames):
            MCsPlanIndexes[MCsMode] = trips[(trips['legMode'] == MCsMode)].index

    ST.at['Simulated Agents ', name] = len(pd.unique(trips['personId'])) 
    ST.at['Trips per Agent AV ', name] = len(trips)/len(pd.unique(trips['personId']))
    
#----------Number Trips
#check plans for estimated mode share, trip per person
    totalTrips_vehicle = len(PTs['mode'])
    totalTrips_est = len(trips)
    totalTrips_mode = len(MCs['mode'])
    totalTrips_replan = len(RPs)
    totalTrips_exec = totalTrips_mode-len(RPs)
    
    print('Number Trips...',name)
    ST.at['Trip Vehicle Total ', name] = totalTrips_vehicle
    ST.at['Trip Est Total ', name] = totalTrips_est
    ST.at['Trip Mode Total ', name] = totalTrips_mode
    ST.at['Trip Replanning Total ', name] = totalTrips_replan
    ST.at['Trip Exectuted Total ', name] = totalTrips_exec

    
    
    for PTsMode, PTsModesName in zip(PTsModes, PTsModesNames):
        ST.at['Trip Vehicle '+PTsModesName, name] = len(PTsModeIndexes[PTsMode])
    if is_leg_mode:
        for MCsMode, MCsName in zip(MCsModes, MCsModesNames):
            ST.at['Trip Est '+MCsName, name] = len(MCsPlanIndexes[MCsMode])
    for MCsMode, MCsName in zip(MCsModes, MCsModesNames):
        ST.at['Trip Mode '+MCsName, name] = len(MCsModeIndexes[MCsMode])
    for MCsMode, MCsName in zip(MCsModes, MCsModesNames):
        ST.at['Trip Replan '+MCsName, name] = len(MCsReplanIndexes[MCsMode])
    transit_exec = 0
    for MCsMode, MCsName in zip(MCsModes, MCsModesNames):
        ST.at['Trip Exec '+MCsName, name] = len(MCsModeIndexes[MCsMode])-len(MCsReplanIndexes[MCsMode])
        if MCsMode in transit_MCmodes:
            transit_exec += len(MCsModeIndexes[MCsMode])-len(MCsReplanIndexes[MCsMode])
    for primaryFuelType in primaryFuelTypes:
        ST.at['Trip Vehicle '+primaryFuelType, name] = len(PTsFuelIndexes[primaryFuelType])
    for company in transitCompanies:
        ST.at['Trip Vehicle '+company, name] = len(PTsTransitIndexes[company])
        
#----------Share Trips
    print('Share Trips...',name)
    for PTsMode, PTsModesName in zip(PTsModes, PTsModesNames):
        ST.at['Trip Vehicle Share '+PTsModesName, name] = len(PTsModeIndexes[PTsMode])/totalTrips_vehicle
    for company in transitCompanies:
        ST.at['Trip Vehicle Share '+company, name] = len(PTsTransitIndexes[company])/totalTrips_vehicle
    if is_leg_mode:
        for MCsMode, MCsName in zip(MCsModes, MCsModesNames):
            ST.at['Trip Est Share '+MCsName, name] = len(MCsPlanIndexes[MCsMode])/totalTrips_est
    for MCsMode, MCsName in zip(MCsModes, MCsModesNames):
        ST.at['Trip Mode Share '+MCsName, name] = len(MCsModeIndexes[MCsMode])/totalTrips_mode
    for MCsMode, MCsName in zip(MCsModes, MCsModesNames):
        ST.at['Trip Replan Share '+MCsName, name] = len(MCsReplanIndexes[MCsMode])/totalTrips_replan
    for MCsMode, MCsName in zip(MCsModes, MCsModesNames):
        ST.at['Trip Exec Share '+MCsName, name] = (len(MCsModeIndexes[MCsMode])-len(MCsReplanIndexes[MCsMode]))/totalTrips_exec
    for primaryFuelType in primaryFuelTypes:
        ST.at['Trip Vehicle Share '+primaryFuelType, name] = len(PTsFuelIndexes[primaryFuelType])/totalTrips_vehicle



#----------Trip Lengths
    #----------------------Vehicles
    print('Lengths Vehicles...',name)
    lengths = PTs['length']/1000.
    for code in codes:
        ST.at['Length Vehicle '+CtV[code]+' [km]', name] = DA(lengths, code)
        for PTsMode, PTsModesName in zip(PTsModes, PTsModesNames):
            lengths_mode = lengths[PTsModeIndexes[PTsMode]]
            ST.at['Length Vehicle '+CtV[code]+' '+PTsModesName+' [km]', name] = DA(lengths_mode, code)
        for company in transitCompanies:
            lengths_company = lengths[PTsTransitIndexes[company]]
            ST.at['Length Vehicle '+CtV[code]+' '+company+' [km]', name] = DA(lengths_company, code)
        for primaryFuelType in primaryFuelTypes:
            lengths_fueltype = lengths[PTsFuelIndexes[primaryFuelType]]
            ST.at['Length Vehicle '+CtV[code]+' '+primaryFuelType+' [km]', name] = DA(lengths_fueltype, code)  
    
    #----------------------Persons
    print('Lengths Persons...',name)
    lengths = PtoPTss['length']/1000.
    for code in codes:
        ST.at['Length Person '+CtV[code]+' [km]', name] = DA(lengths, code)
        for PTsMode, PTsModesName in zip(PTsModes, PTsModesNames):
            lengths_mode = lengths[PtoPTssModeIndexes[PTsMode]]
            ST.at['Length Person '+CtV[code]+' '+PTsModesName+' [km]', name] = DA(lengths_mode, code)
        for company in transitCompanies:
            lengths_company = lengths[PtoPTssTransitIndexes[company]]
            ST.at['Length Person '+CtV[code]+' '+company+' [km]', name] = DA(lengths_company, code)
        for primaryFuelType in primaryFuelTypes:
            lengths_fueltype = lengths[PtoPTssFuelIndexes[primaryFuelType]]
            ST.at['Length Person '+CtV[code]+' '+primaryFuelType+' [km]', name] = DA(lengths_fueltype, code)  
    
    
    #----------------------Modes
    print('Lengths Modes...',name)
    lengths = MCs['length']/1000.
    for code in codes:
        ST.at['Length Mode '+CtV[code]+'[km]', name] = DA(lengths, code)
        for MCsMode, MCsModesName in zip(MCsModes, MCsModesNames):
            lengths_mode = lengths[MCsModeIndexes[MCsMode]]
            ST.at['Length Mode '+CtV[code]+' '+MCsModesName+' [km]', name] = DA(lengths_mode, code)
   
    lengths = PTs['length']/1000.
    
 #----------Trip Durations
    print('Durations Vehicle...',name)
    durations = PTs['duration']/3600.
    for code in codes:
        ST.at['Duration Vehicle'+CtV[code]+' [h]', name] = DA(durations, code)
        for PTsMode, PTsModesName in zip(PTsModes, PTsModesNames):
            durations_mode = durations[PTsModeIndexes[PTsMode]]
            ST.at['Duration Vehicle '+CtV[code]+' '+PTsModesName+' [h]', name] = DA(durations_mode, code)
        for company in transitCompanies:
            durations_company = durations[PTsTransitIndexes[company]]
            ST.at['Duration Vehicle '+CtV[code]+' '+company+' [h]', name] = DA(durations_company, code)
        for primaryFuelType in primaryFuelTypes:
            durations_fueltype = durations[PTsFuelIndexes[primaryFuelType] ]
            ST.at['Duration Vehicle '+CtV[code]+' '+primaryFuelType+' [h]', name] = DA(durations_fueltype, code)   
     
    #----------Persons
    print('Durations Person...',name)
    durations = PtoPTss['duration']/3600.
    for code in codes:
        ST.at['Duration Person'+CtV[code]+' [h]', name] = DA(durations, code)
        for PTsMode, PTsModesName in zip(PTsModes, PTsModesNames):
            durations_mode = durations[PtoPTssModeIndexes[PTsMode]]
            ST.at['Duration Person '+CtV[code]+' '+PTsModesName+' [h]', name] = DA(durations_mode, code)
        for company in transitCompanies:
            durations_company = durations[PtoPTssTransitIndexes[company]]
            ST.at['Duration Person '+CtV[code]+' '+company+' [h]', name] = DA(durations_company, code)
        for primaryFuelType in primaryFuelTypes:
            durations_fueltype = durations[PtoPTssFuelIndexes[primaryFuelType] ]
            ST.at['Duration Person '+CtV[code]+' '+primaryFuelType+' [h]', name] = DA(durations_fueltype, code)   
    
    
#----------Trip Speeds
    print('Speeds Vehicle...',name)
    speeds = lengths/durations[(durations>0)]
    for code in codes:
        ST.at['Speed Vehicle'+CtV[code]+' [km/h]', name] = DA(speeds, code)
        for PTsMode, PTsModesName in zip(PTsModes, PTsModesNames):
            speeds_mode = speeds[PTsModeIndexes[PTsMode]]
            ST.at['Speed Vehicle '+CtV[code]+' '+PTsModesName+' [km/h]', name] = DA(speeds_mode, code)
        for company in transitCompanies:
            speeds_company = speeds[PTsTransitIndexes[company]]
            ST.at['Speed Vehicle '+CtV[code]+' '+company+' [km/h]', name] = DA(speeds_company, code)
        for primaryFuelType in primaryFuelTypes:
            speeds_fueltype = speeds[PTsFuelIndexes[primaryFuelType] ]
            ST.at['Speed Vehicle '+CtV[code]+' '+primaryFuelType+' [km/h]', name] = DA(speeds_fueltype, code)   
    
#----------Energy Consumption
    #----------Route-e correction
    corrections_routee = []
    fot prim, sec in zip(PTs['primaryFuelType'], PTs['secondaryFuelType']):
        if prim in 'Electricity' and sec in ['Biodiesel','Diesel','Gasoline']:
            corrections_routee.append(route_e_correction_hev)
        elif prim in ['Biodiesel','Diesel','Gasoline']:
            corrections_routee.append(route_e_correction_cv)
        elif prim in ['Electricity']:
            corrections_routee.append(route_e_correction_ev)
        else:
            corrections_routee.append(1)
    PTs['routee correction'] = corrections_routee     
    print(PTs['routee correction'].value_counts())

    print('Energy Usage Vehicle...',name)
    energies = (PTs['primaryFuel']/1000000000.+PTs['secondaryFuel']/1000000000.)*PTs['routee correction']
    for code in codes:
        ST.at['Energy Vehicle'+CtV[code]+' [GJ]', name] = DA(energies, code)
        for PTsMode, PTsModesName in zip(PTsModes, PTsModesNames):
            energies_mode = energies[PTsModeIndexes[PTsMode]]
            ST.at['Energy Vehicle '+CtV[code]+' '+PTsModesName+' [GJ]', name] = DA(energies_mode, code)
        for company in transitCompanies:
            energies_company = energies[PTsTransitIndexes[company]]
            ST.at['Energy Vehicle '+CtV[code]+' '+company+' [GJ]', name] = DA(energies_company, code)
        for primaryFuelType in primaryFuelTypes:
            energies_fueltype = energies[PTsFuelIndexes[primaryFuelType] ]
            ST.at['Energy Vehicle '+CtV[code]+' '+primaryFuelType+' [GJ]', name] = DA(energies_fueltype, code)   

#----------Trip Gallons
    print('Trip Gallons Vehicle...',name)
    gallons = PTs['gallonsGasoline']*PTs['routee correction']
    for code in codes:
        ST.at['Gallons Gas Vehicle'+CtV[code]+' [gallon]', name] = DA(gallons, code)
        for PTsMode, PTsModesName in zip(PTsModes, PTsModesNames):
            gallons_mode = gallons[PTsModeIndexes[PTsMode]]
            ST.at['Gallons Gas Vehicle '+CtV[code]+' '+PTsModesName+' [gallon]', name] = DA(gallons_mode*beta_routee_cv, code)
        for company in transitCompanies:
            gallons_company = gallons[PTsTransitIndexes[company]]
            ST.at['Gallons Gas Vehicle '+CtV[code]+' '+company+' [gallon]', name] = DA(gallons_company*beta_routee_cv, code)
        for primaryFuelType in primaryFuelTypes:
            gallons_fueltype = gallons[PTsFuelIndexes[primaryFuelType] ]
            ST.at['Gallons Gas Vehicle '+CtV[code]+' '+primaryFuelType+' [gallon]', name] = DA(gallons_fueltype, code)   

            
#----------Occupancy
    print('Occupancy Vehicle...',name)
    passengers = PTs['occupancy']
    capacities = PTs['capacity']

    for company in transitCompanies:
        passenger_company = passengers[PTsTransitIndexes[company]]
        ST.at['Vehicle Passengers stops '+company, name] = np.sum(passenger_company)

    for company in transitCompanies:
        passenger_company = passengers[PTsTransitIndexes[company]]
        lengths_company = lengths[PTsTransitIndexes[company]]
        ST.at['Vehicle Passengers km '+company, name] = np.sum(passenger_company*lengths_company)

    for company in transitCompanies:
        lengths_company = lengths[PTsTransitIndexes[company]]
        capacities_company = capacities[PTsTransitIndexes[company]]
        ST.at['Vehicle Capacity km '+company, name] = np.sum(capacities_company*lengths_company)

    for company in transitCompanies:
        passenger_company = passengers[PTsTransitIndexes[company]]
        lengths_company = lengths[PTsTransitIndexes[company]]
        capacities_company = capacities[PTsTransitIndexes[company]]
        if np.sum(capacities_company*lengths_company)>0:
            ST.at['Vehicle Load Factor '+company, name] = np.sum(passenger_company*lengths_company)/np.sum(capacities_company*lengths_company)

    ST.at['Vehicle Person km Total ', name] = np.sum(lengths*passengers)

    for PTsMode, PTsModesName in zip(PTsModes, PTsModesNames):
        if PTsMode != 'bike' and PTsMode != 'walk':
            lengths_mode = lengths[PTsModeIndexes[PTsMode]]
            passengers_mode = passengers[PTsModeIndexes[PTsMode]] 
            ST.at['Vehicle Person km '+PTsModesName, name] = np.sum(lengths_mode*passengers_mode)

    for PTsMode, PTsModesName in zip(PTsModes, PTsModesNames):
        if PTsMode != 'bike' and PTsMode != 'walk':
            lengths_mode = lengths[PTsModeIndexes[PTsMode]]
            capacities_mode = capacities[PTsModeIndexes[PTsMode]] 
            ST.at['Vehicle Capacity km '+PTsModesName, name] = np.sum(lengths_mode*capacities_mode)

    for PTsMode, PTsModesName in zip(PTsModes, PTsModesNames):
        if PTsMode != 'bike' and PTsMode != 'walk':
            lengths_mode = lengths[PTsModeIndexes[PTsMode]]
            passengers_mode = passengers[PTsModeIndexes[PTsMode]] 
            capacities_mode = capacities[PTsModeIndexes[PTsMode]] 
            if np.sum(lengths_mode*capacities_mode)>0:
                ST.at['Vehicle Load Factor '+PTsModesName, name] = np.sum(lengths_mode*passengers_mode)/np.sum(lengths_mode*capacities_mode)

#----------Ridership
    print('Ridership Transit...',name)
    total_rs = 0
    for company in transitCompanies:
        ridership_company = len(PEVs['vehicle'][(PEVs['vehicle2'] == company)])
        ST.at['Ridership '+company, name] = ridership_company
        total_rs += ridership_company
    for company in transitCompanies:
        ridership_company = len(PEVs['vehicle'][(PEVs['vehicle2'] == company)])
        ST.at['Ridership '+company+' Share', name] = ridership_company/total_rs

################################################################################################
# ################################################EXTRA FOR NYC################################################
    if is_NYC:
        PEVs_NJ = PEVs[(PEVs['vehicle2'] == 'NJ_Transit')]
        NJ_ridership_bus = 0
        NJ_ridership_rail = 0
        NJ_ridership_lrail = 0
        GTFS_NJ_RAIL_trips = pd.read_csv('GTFS/trips.txt')
        for NJ_vehicle in PEVs_NJ['vehicle']:
            if NJ_vehicle[:12] == 'NJ_Transit_B':
                NJ_ridership_bus += 1
            elif NJ_vehicle[:12] == 'NJ_Transit_R':
                trip_id = NJ_vehicle.split(':')[1]
                route_id = list(GTFS_NJ_RAIL_trips['route_id'][GTFS_NJ_RAIL_trips['trip_id'].astype(str)==trip_id])[0]
                if route_id in [4,12,16]:
                    NJ_ridership_lrail += 1
                else:
                    NJ_ridership_rail +=1

        ST.at['Ridership NJ Transit Bus', name] = NJ_ridership_bus
        ST.at['Ridership NJ Transit Rail', name] = NJ_ridership_rail
        ST.at['Ridership NJ Transit Light Rail', name] = NJ_ridership_lrail
        ST.at['Ridership NJ Transit Bus Share', name] = NJ_ridership_bus/total_rs
        ST.at['Ridership NJ Transit Rail Share', name] = NJ_ridership_rail/total_rs
        ST.at['Ridership NJ Transit Light Rail Share', name] = NJ_ridership_lrail/total_rs
        
    #Check ridership Subway

        agencies = []
        for vehicle in PtoPTss['vehicleID']:
            agencies.append(vehicle[:20])
        PtoPTss['agency'] = agencies

        grouping = PtoPTss.groupby(['personID','planIndex']).apply(lambda x: [y[0] for y in groupby(x.agency)]).to_dict()

        combinations = []
        i = 0
        for key in grouping.keys():
            i+=1
            combinations.append(grouping[key])

        ridership_Subway = 0
        ridership_LIRR = 0
        ridership_MN = 0
        for comb in combinations:
            for c in comb:
                if c[:10] =='NYC_Subway':
                    ridership_Subway += 1
                if c[:10] =='Long_Islan':
                    ridership_LIRR += 1
                if c[:10] =='Metro-Nort':
                    ridership_MN += 1
    
        ST.at['Ridership NYC Subway Without Transfers', name] = ridership_Subway
        ST.at['Ridership LIRR Without Transfers', name] = ridership_LIRR
        ST.at['Ridership Metro North Without Transfers', name] = ridership_MN

################################################################################################
################################################################################################

    ST.at['Ridership Total', name] = total_rs
    ST.at['Transit Transfer per trip AV', name] = total_rs/transit_exec-1.
#----------RH


    start_RH = time.time()
    print('Ride hail...',name)
    
    try:
        rh_waittime = pd.read_csv(fp+data_name[:-13]+'rideHailIndividualWaitingTimes.csv')
        ST.at['Number RH vehicles', name] = len(np.unique(list(rh_waittime['rideHailVehicleId'])))
        ST.at['AV Waiting Time RH', name] = np.mean(list(rh_waittime['waitingTimeInSeconds']))
        ST.at['AV Waiting Time RH Pooled', name] = np.mean(list(rh_waittime[rh_waittime['modeChoice']=='ride_hail_pooled']['waitingTimeInSeconds']))
        ST.at['AV Waiting Time RH Single', name]  = np.mean(list(rh_waittime[rh_waittime['modeChoice']=='ride_hail']['waitingTimeInSeconds']))    
    except:
        print('WARNING: probably not modeChoice column on pd.read_csv(fp+data_name[:-13]+\'rideHailIndividualWaitingTimes.csv\'')
    
    PTsRH = PTs[PTs['isRH']]
    ST.at['Empty Trips RH', name] = len(PTsRH['vehicle'][PTsRH['numPassengers']==0])
    ST.at['Not Empty Trips RH', name] = len(PTsRH['vehicle'][PTsRH['numPassengers']>0])
    if len(PTsRH['vehicle']) >0:
        ST.at['Empty Trips RH Share', name] = len(PTsRH['vehicle'][PTsRH['numPassengers']==0])/len(PTsRH['vehicle'])
        ST.at['Not Empty Trips RH Share', name] = len(PTsRH['vehicle'][PTsRH['numPassengers']>0])/len(PTsRH['vehicle'])
    
    for code in codes:
        ST.at[CtV[code]+' Trips per RH Vehicle', name] = DA(PTsRH['vehicle'].value_counts(), code)
        
    rh_vehicles = pd.unique(PTsRH['vehicle'])
    n_empty = []
    n_notempty = []
    first_trip = []
    last_trip = []
    for rh_vehicle in rh_vehicles:
        PTs_rh_vehicle = PTsRH[PTsRH['vehicle']==rh_vehicle]
        n_empty.append(len(PTs_rh_vehicle['vehicle'][PTs_rh_vehicle['numPassengers']==0]))
        n_notempty.append(len(PTs_rh_vehicle['vehicle'][PTs_rh_vehicle['numPassengers']>0]))
        PTs_rh_vehicle = PTs_rh_vehicle.sort_values(by='time', ascending=True)
        first_trip.append(list(PTs_rh_vehicle['numPassengers'])[0])
        last_trip.append(list(PTs_rh_vehicle['numPassengers'])[-1])
        
    share_empty = np.array(n_empty)-np.array(n_notempty)
    for code in codes:
        ST.at[CtV[code]+' RH Vehicle (Empty - not Empty) Trips', name] = DA(share_empty, code)
    
    if len(first_trip) >0:
        ST.at['RH Empty Share Firts Trip', name] = np.count_nonzero(np.array(first_trip) == 0)/len(first_trip)
        ST.at['RH Empty Share Last Trip', name] = np.count_nonzero(np.array(last_trip) == 0)/len(first_trip)
        ST.at['RH not Empty Share Firts Trip', name] = np.count_nonzero(np.array(first_trip) == 1)/len(first_trip)
        ST.at['RH not Empty Share Last Trip', name] = np.count_nonzero(np.array(last_trip) == 1)/len(first_trip)

#----------RH - WC
    if is_WC == True:
        
        start_RH = time.time()
        print('Ride hail WC...',name)
        
        try:
            rh_waittime_wc =rh_waittime[rh_waittime['rideHailVehicleId'].isin(list(wc_vehicles))]
            ST.at['Number RH WC vehicles', name] = len(np.unique(list(rh_waittime_wc['rideHailVehicleId'])))
            ST.at['AV Waiting Time RH WC', name] = np.mean(list(rh_waittime_wc['waitingTimeInSeconds']))
            ST.at['AV Waiting Time RH WC Pooled', name] = np.mean(list(rh_waittime_wc[rh_waittime_wc['modeChoice']=='ride_hail_pooled']['waitingTimeInSeconds']))
            ST.at['AV Waiting Time RH WC Single', name]  = np.mean(list(rh_waittime_wc[rh_waittime_wc['modeChoice']=='ride_hail']['waitingTimeInSeconds'])) 
        except:
            print('WARNING: probably not modeChoice column on pd.read_csv(fp+data_name[:-13]+\'rideHailIndividualWaitingTimes.csv\'')

        PTsRH = PTs[PTs['isRH_WC']]
        ST.at['Empty Trips RH WC', name] = len(PTsRH['vehicle'][PTsRH['numPassengers']==0])
        ST.at['Not Empty Trips RH WC', name] = len(PTsRH['vehicle'][PTsRH['numPassengers']>0])
        if len(PTsRH['vehicle']) >0:
            ST.at['Empty Trips RH WC Share', name] = len(PTsRH['vehicle'][PTsRH['numPassengers']==0])/len(PTsRH['vehicle'])
            ST.at['Not Empty Trips RH WC Share', name] = len(PTsRH['vehicle'][PTsRH['numPassengers']>0])/len(PTsRH['vehicle'])

        for code in codes:
            ST.at[CtV[code]+' Trips per RH WC Vehicle', name] = DA(PTsRH['vehicle'].value_counts(), code)

        rh_vehicles = pd.unique(PTsRH['vehicle'])
        n_empty = []
        n_notempty = []
        first_trip = []
        last_trip = []
        wc_vehicles = []
        for rh_vehicle in rh_vehicles:
            PTs_rh_vehicle = PTsRH[PTsRH['vehicle']==rh_vehicle]
            n_empty.append(len(PTs_rh_vehicle['vehicle'][PTs_rh_vehicle['numPassengers']==0]))
            n_notempty.append(len(PTs_rh_vehicle['vehicle'][PTs_rh_vehicle['numPassengers']>0]))
            PTs_rh_vehicle = PTs_rh_vehicle.sort_values(by='time', ascending=True)
            first_trip.append(list(PTs_rh_vehicle['numPassengers'])[0])
            last_trip.append(list(PTs_rh_vehicle['numPassengers'])[-1])
            wc_vehicles.append(rh_vehicle)
            
        wc_vehicles = np.unique(wc_vehicles)

        share_empty = np.array(n_empty)-np.array(n_notempty)
        for code in codes:
            ST.at[CtV[code]+' RH WC Vehicle (Empty - not Empty) Trips', name] = DA(share_empty, code)

        if len(first_trip) >0:
            ST.at['RH WC Empty Share Firts Trip', name] = np.count_nonzero(np.array(first_trip) == 0)/len(first_trip)
            ST.at['RH WC Empty Share Last Trip', name] = np.count_nonzero(np.array(last_trip) == 0)/len(first_trip)
            ST.at['RH WC not Empty Share Firts Trip', name] = np.count_nonzero(np.array(first_trip) == 1)/len(first_trip)
            ST.at['RH WC not Empty Share Last Trip', name] = np.count_nonzero(np.array(last_trip) == 1)/len(first_trip)

# #----------RH - Multi Agency
    
#     PTsRH['vehicle']
#     RH_agencies = np.unique(PTsRH)

            
            
    #Mode share per distance ranges
    MCs = updateTransitMode(MCs)
    gb = MCs[['mode','person', 'distBin']].groupby(['mode', 'distBin']).agg('size').unstack().fillna(0.0)
    tot = gb.sum(axis=0)
    gb = gb / gb.sum(axis=0)
    gb.loc['total',:] = tot/tot.sum()
    for mode in gb.index:
        for key in gb.keys():
            ST.at['Share Mode '+mode+' Miles '+key, name] = gb.at[mode,key]
    
    tripsPerPerson = (ASs[['person','actType']].groupby('actType').agg('size') / len(ASs['person'].value_counts())).to_dict()

    for act in tripsPerPerson.keys():
        ST.at['Activities AV per person '+act, name] = tripsPerPerson[act]

    print("done")

    return ST

SyntaxError: invalid syntax (3252292339.py, line 188)

In [ ]:
ST = pd.DataFrame()


for data_name, name, plan_name, i in zip(data_names, names, plan_names, range(len(data_names))):
    
    MCs = []
    PTs = []
    PEVs = []
    PLVs = []
    PtoPTss = []
    LT = False
    PLT = False
    VLT = False
    MCs, PTs, PEVs, PLVs, RPs, ASs = readEvents(fp+data_name)
    MCs, PTs, PEVs, PLVs = fixData(MCs, PTs, PEVs, PLVs, len_id_transit)
    trips, activities, personToTripDepartures, is_leg_mode = processPlans(fp+plan_name)
    PtoPTss = personToPathTraversal(PTs,PEVs,PLVs,personToTripDepartures)
    
    if is_filter_by_region:
        MCs, PTs, PEVs, PLVs, RPs, trips, PtoPTss = filter_by_region(region_filter, MCs, PTs, PEVs, PLVs, RPs, trips, PtoPTss,)
    
    # if is_save_PTs_PtoPTs:
    #     PTs.to_csv(fp_res+'PTs_'+name+'.csv')
    #     PtoPTss.to_csv(fp_res+'PtoPTss_'+name+'.csv')
    
    codes = [0,1,2,3,4,5,6,7,8] 
    transitCompanies = PTs['vehicle2'][PTs['is_transit']>0].value_counts().keys()
    
    ST = SummaryTable(ST, data_name, name, plan_name, MCs, PTs, PEVs, PLVs, RPs, trips, PtoPTss, codes, transitCompanies, is_leg_mode)
    
    if is_LT:
        # link_fp = fp+data_name[:-13]+'linkstats.csv.gz'
        # VLT, PLT = LinkTable(VLT, PLT, name, link_fp, PTs, PtoPTss)
        network = pd.read_csv(fp+data_name[:-(len(data_name.split('/')[-1])+len(data_name.split('/')[-2])+len(data_name.split('/')[-3])+2)]+'network.csv.gz')
        if is_GEO:
            network, link_taz = network_TAZ(network)
        else:
            link_taz = ''
        LT = pd.read_csv(fp+data_name[:-13]+'linkstats.csv.gz')
        if is_GEO:
            LT = link_TAZ_fun(network, LT)
        PLT = PLT_build(data_name,PtoPTss, network, link_taz)
        VLT = VLT_build(data_name,PTs, network, link_taz)
        # VLT.to_csv(fp_res+'VLT_'+name+'.csv')
        # PLT.to_csv(fp_res+'PLT_'+name+'.csv')
        # LT.to_csv(fp_res+'LT_'+name+'.csv')

    print('Total Time', time.time()-start)
    start = time.time()
    print(ST[-6:],'Number of attributes',len(ST))
    ST.to_csv(fp_res+output_nm)

    if is_collect_data:
        collectAllData(fp+data_name[:-(len(data_name.split('/')[-1])+len(data_name.split('/')[-2])+len(data_name.split('/')[-3])+len(data_name.split('/')[-4])+len(data_name.split('/')[-5])+4)],
                  data_name.split('/')[-4],trips, activities, PTs, PtoPTss, LT, PLT, VLT, ST, name)
        
ST['code'] = range(len(ST[ST.keys()[0]]))
print(ST[-6:],'Number of attributes',len(ST))
ST.to_csv(fp_res+output_nm)

end = time.time()
print('Total time',end- start, 's')

In [ ]:
#Validation NYC

ST2 = pd.DataFrame()
ST = pd.read_csv('outputs/SummaryTable_NYC_baseline16.csv', index_col='Unnamed: 0')

# subway_notransfs = [ 
            # 2922226,#1/3 part 1
            # 345063,#7
            # 241275,#11
            # 251684,#12
            # 270082,#13
            # 223795,#15
            # 196499,#16
            # 167302,#17
            # 255169,#18
            # 239665,#19
            # 231035,#20
            # 516343,#0/10
            # 514203,#3/10
            # 518130,#4/10
            # 515515,#5/10
            # 518068,#6/10
            # 511123,#7/10
            # 517393,#8/10
            # 517393,517393,517393
                    # ]

for name, share_pop in zip(names,shares_pop):
    
    ST2.at['Original Population share', name] = share_pop
    
    ST2.at['Scaled Total Simulated Agents', name] = ST.at['Simulated Agents ', name]/share_pop
    

    ST2.at['Total Trips Estimated per Agent in a Day', name] = ST.at['Trips per Agent AV ', name]


    ST2.at['Scaled Total Trips Estimated in a Day', name] = ST.at['Trips per Agent AV ', name]*ST.at['Simulated Agents ', name]/share_pop

    if is_leg_mode:

        ST2.at['Scaled Total Estimated Walk-Transit Trips in a Day', name] = ST.at['Trip Est Walk-Transit', name]/share_pop


    ST2.at['Scaled Total Replanned Walk-Transit Trips in a Day', name] = ST.at['Trip Replan Walk-Transit', name]/share_pop


    ST2.at['Scaled Total Executed Walk-Transit Trips in a Day', name] = ST.at['Trip Exec Walk-Transit', name]/share_pop


    ST2.at['Scaled Total Modechoice Walk-Transit Trips in a Day', name] = ST.at['Trip Mode Walk-Transit', name]/share_pop

    if is_leg_mode:

        ST2.at['Share Estimated Walk-Transit Trips in a Day', name] = ST.at['Trip Est Share Walk-Transit', name]


    ST2.at['Share Replanned Walk-Transit Trips in a Day', name] = ST.at['Trip Replan Share Walk-Transit', name]


    ST2.at['Share Executed Walk-Transit Trips in a Day', name] = ST.at['Trip Exec Share Walk-Transit', name]
    ST2.at['Share Executed Bike-Transit Trips in a Day', name] = ST.at['Trip Exec Share Bike-Transit', name]
    ST2.at['Share Executed Ride Hail-Transit Trips in a Day', name] = ST.at['Trip Exec Share Ride Hail-Transit', name]
    ST2.at['Share Executed Drive-Transit Trips in a Day', name] = ST.at['Trip Exec Share Drive-Transit', name]
    ST2.at['Share Executed Transit Related Trips in a Day', name] = (ST.at['Trip Exec Share Walk-Transit', name] +
                                                                    ST.at['Trip Exec Share Bike-Transit', name] +
                                                                    ST.at['Trip Exec Share Ride Hail-Transit', name] +
                                                                    ST.at['Trip Exec Share Drive-Transit', name])
    ST2.at['Share Executed Bike Trips in a Day', name] = ST.at['Trip Exec Share Bike', name]
    ST2.at['Share Executed Car Trips in a Day', name] = ST.at['Trip Exec Share Car', name]
    ST2.at['Share Executed Ride Hail Trips in a Day', name] = (ST.at['Trip Exec Share Ride Hail', name] +
                                                            ST.at['Trip Exec Share Ride Hail Pooled', name])
    ST2.at['Share Executed Walk Trips in a Day', name] = ST.at['Trip Exec Share Walk', name]
    ST2.at['Share Executed Other Trips in a Day', name] = 1-(ST.at['Trip Exec Share Walk-Transit', name] +
                                                                    ST.at['Trip Exec Share Bike-Transit', name] +
                                                                    ST.at['Trip Exec Share Ride Hail-Transit', name] +
                                                                    ST.at['Trip Exec Share Drive-Transit', name] +
                                                                    ST.at['Trip Exec Share Bike', name] +
                                                                    ST.at['Trip Exec Share Car', name] +
                                                                    ST.at['Trip Exec Share Ride Hail', name] +
                                                                    ST.at['Trip Exec Share Ride Hail Pooled', name] +
                                                                    ST.at['Trip Exec Share Walk', name])


    ST2.at['Share Executed Walk-Transit Trips in a Day', name] = ST.at['Trip Exec Share Walk-Transit', name]


    ST2.at['AV Transit Transfers per trip', name] = ST.at['Transit Transfer per trip AV', name]


    ST2.at['Scaled MTA BUS Ridership (with transfers)', name] = (ST.at['Ridership MTA_Brookl', name]+
                                                             ST.at['Ridership MTA_Bronx_', name]+
                                                             ST.at['Ridership MTA_Queens', name]+
                                                             ST.at['Ridership MTA_Staten', name]+
                                                             ST.at['Ridership MTA_Manhat', name]+
                                                             ST.at['Ridership NYC_Bus_Co', name])/share_pop


    ST2.at['Scaled MTA SUB Ridership (with transfers)', name] = ST.at['Ridership NYC_Subway', name]/share_pop
    
    ST2.at['Scaled MTA SUB Ridership (without transfers)', name] = ST.at['Ridership NYC Subway Without Transfers', name]/share_pop
    
    ST2.at['Subway vs Bus', name] = ST2.at['Scaled MTA SUB Ridership (without transfers)', name]/ST2.at['Scaled MTA BUS Ridership (with transfers)', name]


    ST2.at['Scaled Metro North Ridership (with transfers)', name] = ST.at['Ridership Metro-Nort', name]/share_pop
    ST2.at['Scaled Metro North Ridership (without transfers)', name] = ST.at['Ridership Metro North Without Transfers', name]/share_pop


    ST2.at['Scaled LIRR Ridership (with transfers)', name] = ST.at['Ridership Long_Islan', name]/share_pop
    ST2.at['Scaled LIRR Ridership (without transfers)', name] = ST.at['Ridership LIRR Without Transfers', name]/share_pop
    
    try:
        if ST.at['Ridership 151_631:t_', name] > 0:
            ST2.at['Scaled PATH Ridership (with transfers)', name] = ST.at['Ridership 151_631:t_', name]/share_pop
    except:      
        ST2.at['Scaled PATH Ridership (with transfers)', name] = ST.at['Ridership path-nj-us', name]/share_pop

    ST2.at['Scaled NJ BUS Ridership (with transfers)', name] = ST.at['Ridership NJ Transit Bus', name]/share_pop
    ST2.at['Scaled NJ RAIL Ridership (with transfers)', name] = ST.at['Ridership NJ Transit Rail', name]/share_pop
    ST2.at['Scaled NJ LIGHT RAIL Ridership (with transfers)', name] = ST.at['Ridership NJ Transit Light Rail', name]/share_pop
    
    for bin in range(len(binnames)):
        bin = binnames[bin+1]
        ST2.at['Share Mode Bike Miles '+bin, name] =  ST.at['Share Mode bike Miles '+bin, name]#+ST.at['Share Mode bike_transit Miles '+bin, name]
        ST2.at['Share Mode Car Miles '+bin, name] =  ST.at['Share Mode car Miles '+bin, name]#+ST.at['Share Mode drive_transit Miles '+bin, name]
        multimodal_shares = (ST.at['Share Mode bike_transit Miles '+bin, name]+
                             ST.at['Share Mode drive_transit Miles '+bin, name]+
                            ST.at['Share Mode ride_hail_transit Miles '+bin, name])
        tot_transit = (ST.at['Share Mode bus Miles '+bin, name]+
            ST.at['Share Mode ferry Miles '+bin, name]+
            ST.at['Share Mode rail Miles '+bin, name]+
            ST.at['Share Mode subway Miles '+bin, name]
                    )
        share_bus=ST.at['Share Mode bus Miles '+bin, name]/tot_transit
        share_fer=ST.at['Share Mode ferry Miles '+bin, name]/tot_transit
        share_rai=ST.at['Share Mode rail Miles '+bin, name]/tot_transit
        share_sub=ST.at['Share Mode subway Miles '+bin, name]/tot_transit
        ST2.at['Share Mode Bus Miles '+bin, name] =  ST.at['Share Mode bus Miles '+bin, name]+multimodal_shares*share_bus
        ST2.at['Share Mode Ferry Miles '+bin, name] =  ST.at['Share Mode ferry Miles '+bin, name]+multimodal_shares*share_fer
        ST2.at['Share Mode Rail Miles '+bin, name] =  ST.at['Share Mode rail Miles '+bin, name]+multimodal_shares*share_rai
        ST2.at['Share Mode Subway Miles '+bin, name] =  ST.at['Share Mode subway Miles '+bin, name]+multimodal_shares*share_sub

        try:
            # st_rh = ST.at['Share Mode ride_hail Miles '+bin, name]
            ST2.at['Share Mode Ridehail Miles '+bin, name] =  ST.at['Share Mode ride_hail Miles '+bin, name]#+ST.at['Share Mode ride_hail_transit Miles '+bin, name]
            try:
                ST2.at['Share Mode Ridehail Miles '+bin, name] =  ST.at['Share Mode ride_hail Miles '+bin, name]+ST.at['Share Mode ride_hail_pooled Miles '+bin, name]
            except:
                print('Ridehail is present, but not ridehail pooled')
        except:
            ST2.at['Share Mode Ridehail Miles '+bin, name] = 0
            print('Warning, ride hail not in the simulation')
        ST2.at['Share Mode Walk Miles '+bin, name] =  ST.at['Share Mode walk Miles '+bin, name]
        ST2.at['Share Mode Other Miles '+bin, name] =  1-(ST2.at['Share Mode Bike Miles '+bin, name]+
                                                            ST2.at['Share Mode Bus Miles '+bin, name]+
                                                            ST2.at['Share Mode Car Miles '+bin, name]+
                                                            ST2.at['Share Mode Ferry Miles '+bin, name]+
                                                            ST2.at['Share Mode Rail Miles '+bin, name]+
                                                            ST2.at['Share Mode Ridehail Miles '+bin, name]+
                                                            ST2.at['Share Mode Subway Miles '+bin, name] +
                                                            ST2.at['Share Mode Walk Miles '+bin, name])
    
    ST2.at['Activities AV per person Home', name] = ST.at['Activities AV per person Home', name]
    ST2.at['Activities AV per person Meal', name] = ST.at['Activities AV per person Meal', name]  
    ST2.at['Activities AV per person Shopping', name] = ST.at['Activities AV per person Shopping', name]  
    ST2.at['Activities AV per person SocRec', name] = ST.at['Activities AV per person SocRec', name]  
    ST2.at['Activities AV per person Work', name] = ST.at['Activities AV per person Work', name] 
    ST2.at['Activities AV per person Other', name] = ST.at['Activities AV per person Other', name] 
    
ST2.at['Share Executed Transit Related Trips in a Day', 'Target Baseline'] = 'NHTS is 16.6%'
ST2.at['Share Executed Bike Trips in a Day', 'Target Baseline'] = 'NHTS is 1.0%'
ST2.at['Share Executed Car Trips in a Day', 'Target Baseline'] = 'NHTS is 53.9%'
ST2.at['Share Executed Walk Trips in a Day', 'Target Baseline'] = 'NHTS is 26.1%'
ST2.at['Share Executed Ride Hail Trips in a Day', 'Target Baseline'] = 'NHTS is 1.8%'
ST2.at['Share Executed Other Trips in a Day', 'Target Baseline'] = 'NHTS is 0.6%'

    
ST2.at['Scaled PATH Ridership (with transfers)', 'Target Baseline'] = '297k'
ST2.at['Scaled NJ BUS Ridership (with transfers)', 'Target Baseline'] =  '451k'
ST2.at['Scaled NJ RAIL Ridership (with transfers)', 'Target Baseline'] =  '127k'
ST2.at['Scaled NJ LIGHT RAIL Ridership (with transfers)', 'Target Baseline'] =  '14k'

ST2.at['Scaled MTA BUS Ridership (with transfers)', 'Target Baseline'] = '≈2.19M'
ST2.at['Scaled MTA BUS Ridership (with transfers)', 'Target 1 - 04-2020'] = '≈506k'
ST2.at['Scaled MTA BUS Ridership (with transfers)', 'Target 2 - 08-2020'] = '≈1.21M'
ST2.at['Scaled MTA BUS Ridership (with transfers)', 'Target 4 - 08-2021'] = '≈1.24M'
ST2.at['Scaled MTA BUS Ridership (with transfers)', 'Target 3 - 01-2022'] = '≈1.04M'
ST2.at['Scaled MTA BUS Ridership (with transfers)', 'Target 5 - 05-2022'] = '≈1.46M'
ST2.at['Scaled MTA SUB Ridership (without transfers)', 'Target Baseline'] = '≈5.42M'
ST2.at['Scaled MTA SUB Ridership (without transfers)', 'Target 1 - 04-2020'] = '≈508k'
ST2.at['Scaled MTA SUB Ridership (without transfers)', 'Target 2 - 08-2020'] = '≈1.26M'
ST2.at['Scaled MTA SUB Ridership (without transfers)', 'Target 3 - 01-2022'] = '≈2.25M'
ST2.at['Scaled MTA SUB Ridership (without transfers)', 'Target 4 - 08-2021'] = '≈2.48M'
ST2.at['Scaled MTA SUB Ridership (without transfers)', 'Target 5 - 05-2022'] = '≈3.34M'
ST2.at['Subway vs Bus', 'Target Baseline', ] = '2.48'
ST2.at['Subway vs Bus', 'Target 1 - 04-2020'] = '1.00'
ST2.at['Subway vs Bus', 'Target 2 - 08-2020'] = '1.04'
ST2.at['Subway vs Bus', 'Target 3 - 01-2022'] = '2.17'
ST2.at['Subway vs Bus', 'Target 4 - 08-2021'] = '2.00'
ST2.at['Subway vs Bus', 'Target 5 - 05-2022'] = '2.28'
ST2.at['Scaled LIRR Ridership (without transfers)', 'Target Baseline'] = '≈250k'
ST2.at['Scaled LIRR Ridership (without transfers)', 'Target 1 - 04-2020'] = '≈10k'
ST2.at['Scaled LIRR Ridership (without transfers)', 'Target 2 - 08-2020'] = '≈60k'
ST2.at['Scaled LIRR Ridership (without transfers)', 'Target 3 - 01-2022'] = '≈100k'
ST2.at['Scaled LIRR Ridership (without transfers)', 'Target 4 - 08-2021'] = '≈123k'
ST2.at['Scaled LIRR Ridership (without transfers)', 'Target 5 - 05-2022'] = '≈170k'
ST2.at['Scaled Metro North Ridership (without transfers)','Target Baseline'] = '≈200k'
ST2.at['Scaled Metro North Ridership (without transfers)', 'Target 1 - 04-2020'] = '≈14k'
ST2.at['Scaled Metro North Ridership (without transfers)', 'Target 2 - 08-2020'] = '≈39k'
ST2.at['Scaled Metro North Ridership (without transfers)', 'Target 3 - 01-2022'] = '≈80k'
ST2.at['Scaled Metro North Ridership (without transfers)', 'Target 4 - 08-2021'] = '≈101k'
ST2.at['Scaled Metro North Ridership (without transfers)', 'Target 5 - 05-2022'] = '≈149k'

#########
           
ST2.at['Scaled PATH Ridership (with transfers)', 'Target 1 - 04-2020'] = '≈15k'
ST2.at['Scaled PATH Ridership (with transfers)', 'Target 2 - 08-2020'] = '≈54k'
ST2.at['Scaled PATH Ridership (with transfers)', 'Target 3 - 01-2022'] = '≈98k'
ST2.at['Scaled PATH Ridership (with transfers)', 'Target 4 - 08-2021'] = '≈85k'
# ST2.at['Ridership Metro North Subway Without Transfers', 'Target 5 - 07-2022'] = '≈5.4M'
#########
ST2.at['Scaled NJ BUS Ridership (with transfers)', 'Target 1 - 04-2020'] = '≈93k'
ST2.at['Scaled NJ BUS Ridership (with transfers)', 'Target 2 - 08-2020'] = '≈219k'
ST2.at['Scaled NJ BUS Ridership (with transfers)', 'Target 3 - 01-2022'] = '≈280k'
ST2.at['Scaled NJ BUS Ridership (with transfers)', 'Target 4 - 08-2021'] = '≈232k'
# ST2.at['Ridership Metro North Subway Without Transfers', 'Target 5 - 07-2022'] = '≈5.4M'
#########
ST2.at['Scaled NJ RAIL Ridership (with transfers)', 'Target 1 - 04-2020'] = '≈4k'
ST2.at['Scaled NJ RAIL Ridership (with transfers)', 'Target 2 - 08-2020'] = '≈31k'
ST2.at['Scaled NJ RAIL Ridership (with transfers)', 'Target 3 - 01-2022'] = '≈73k'
ST2.at['Scaled NJ RAIL Ridership (with transfers)', 'Target 4 - 08-2021'] = '≈52k'
# ST2.at['Ridership Metro North Subway Without Transfers', 'Target 5 - 07-2022'] = '≈5.4M'
#########
ST2.at['Scaled NJ LIGHT RAIL Ridership (with transfers)', 'Target 1 - 04-2020'] = '≈2k'
ST2.at['Scaled NJ LIGHT RAIL Ridership (with transfers)', 'Target 2 - 08-2020'] = '≈6k'
ST2.at['Scaled NJ LIGHT RAIL Ridership (with transfers)', 'Target 3 - 01-2022'] = '≈8k'
ST2.at['Scaled NJ LIGHT RAIL Ridership (with transfers)', 'Target 4 - 08-2021'] = '≈7k'
# ST2.at['Ridership Metro North Subway Without Transfers', 'Target 5 - 07-2022'] = '≈5.4M'

   
ST2.at['Share Mode Bike Miles 0-1', 'Target Baseline'] =  '0.86%'
ST2.at['Share Mode Bus Miles 0-1', 'Target Baseline'] =  '1.92%'
ST2.at['Share Mode Car Miles 0-1', 'Target Baseline'] =  '27.02%'
ST2.at['Share Mode Ferry Miles 0-1', 'Target Baseline'] =  '0.00%'
ST2.at['Share Mode Rail Miles 0-1', 'Target Baseline'] =  '0.03%'
ST2.at['Share Mode Ridehail Miles 0-1', 'Target Baseline'] =  '0.51%'
ST2.at['Share Mode Subway Miles 0-1', 'Target Baseline'] =  '0.73%'
ST2.at['Share Mode Walk Miles 0-1', 'Target Baseline'] =  '68.61%'
ST2.at['Share Mode Other Miles 0-1', 'Target Baseline'] =  '0.32%'

ST2.at['Share Mode Bike Miles 1-2', 'Target Baseline'] =  '1.88%'
ST2.at['Share Mode Bus Miles 1-2', 'Target Baseline'] =  '10.51%'
ST2.at['Share Mode Car Miles 1-2', 'Target Baseline'] =  '62.69%'
ST2.at['Share Mode Ferry Miles 1-2', 'Target Baseline'] =  '0.01%'
ST2.at['Share Mode Rail Miles 1-2', 'Target Baseline'] =  '0.05%'
ST2.at['Share Mode Ridehail Miles 1-2', 'Target Baseline'] =  '4.89%'
ST2.at['Share Mode Subway Miles 1-2', 'Target Baseline'] =  '4.07%'
ST2.at['Share Mode Walk Miles 1-2', 'Target Baseline'] =  '14.90%'
ST2.at['Share Mode Other Miles 1-2', 'Target Baseline'] =  '1.00%'

ST2.at['Share Mode Bike Miles 2-5', 'Target Baseline'] =  '1.57%'
ST2.at['Share Mode Bus Miles 2-5', 'Target Baseline'] =  '9.69%'
ST2.at['Share Mode Car Miles 2-5', 'Target Baseline'] =  '68.28%'
ST2.at['Share Mode Ferry Miles 2-5', 'Target Baseline'] =  '0.00%'
ST2.at['Share Mode Rail Miles 2-5', 'Target Baseline'] =  '0.00%'
ST2.at['Share Mode Ridehail Miles 2-5', 'Target Baseline'] =  '3.03%'
ST2.at['Share Mode Subway Miles 2-5', 'Target Baseline'] =  '13.26%'
ST2.at['Share Mode Walk Miles 2-5', 'Target Baseline'] =  '3.18%'
ST2.at['Share Mode Other Miles 2-5', 'Target Baseline'] =  '0.99%'

ST2.at['Share Mode Bike Miles 5-15', 'Target Baseline'] =  '0.38%'
ST2.at['Share Mode Bus Miles 5-15', 'Target Baseline'] =  '7.45%'
ST2.at['Share Mode Car Miles 5-15', 'Target Baseline'] =  '67.58%'
ST2.at['Share Mode Ferry Miles 5-15', 'Target Baseline'] =  '0.34%'
ST2.at['Share Mode Rail Miles 5-15', 'Target Baseline'] =  '1.22%'
ST2.at['Share Mode Ridehail Miles 5-15', 'Target Baseline'] =  '0.83%'
ST2.at['Share Mode Subway Miles 5-15', 'Target Baseline'] =  '20.64%'
ST2.at['Share Mode Walk Miles 5-15', 'Target Baseline'] =  '1.19%'
ST2.at['Share Mode Other Miles 5-15', 'Target Baseline'] =  '0.38%'

ST2.at['Share Mode Bike Miles 15+', 'Target Baseline'] =  '0.10%'
ST2.at['Share Mode Bus Miles 15+', 'Target Baseline'] =  '6.05%'
ST2.at['Share Mode Car Miles 15+', 'Target Baseline'] =  '72.21%'
ST2.at['Share Mode Ferry Miles 15+', 'Target Baseline'] =  '0.33%'
ST2.at['Share Mode Rail Miles 15+', 'Target Baseline'] =  '8.42%'
ST2.at['Share Mode Ridehail Miles 15+', 'Target Baseline'] =  '1.43%'
ST2.at['Share Mode Subway Miles 15+', 'Target Baseline'] =  '10.15%'
ST2.at['Share Mode Walk Miles 15+', 'Target Baseline'] =  '0.61%'
ST2.at['Share Mode Other Miles 15+', 'Target Baseline'] =  '0.70%'

ST2.at['Activities AV per person Home', 'Target Baseline'] =  '1.307'
ST2.at['Activities AV per person Meal', 'Target Baseline'] =  '0.260'
ST2.at['Activities AV per person Shopping', 'Target Baseline'] =  '0.515'
ST2.at['Activities AV per person SocRec', 'Target Baseline'] =  '0.431'
ST2.at['Activities AV per person Work', 'Target Baseline'] =  '0.742'
ST2.at['Activities AV per person Other', 'Target Baseline'] =  '0.566'



ST2.to_csv('outputs/validationNYC16.csv')


In [ ]:
bridge_link_ids=[542014,88230,853928,779898,853900,30814,154,731748,1367888,487778,13722,
                 1071576,62416,729848,765880,1090614,788122,788140,788118,1341064,53416,
                 732358,865062,999610,
                 308, 540388, #lincloln tunnel I 2
                 87186, #lincloln tunnel O 2
                 87176, 767820, #washington I
                 88110, 735454, #washington O
                 540918, #Holland Tunnel I
                 87188, #Holland Tunnel O
                 852460, #Outerbridge Crossing I
                 852458, #Outerbridge Crossing O 
                 1226968, #Goethals Bridge I 
                 1292432, #Goethals Bridge O
                 759498, #Bayonne Bridge I
                 759500, #Bayonne Bridge O
                ]

for data_name, name, plan_name, share_pop in zip(data_names, names, plan_names, shares_pop):
    # print(data_name)
    #Contains flows
    print(name,'...')
    try:
        pte = pd.read_csv(fp+data_name[:-13]+'linkstats.csv.gz')
        #Contains coord
        network = pd.read_csv(fp+data_name[:-(len(data_name.split('/')[-1])+len(data_name.split('/')[-2])+len(data_name.split('/')[-3])+2)]+'network.csv.gz')
        pte = pte[pte['link'].isin(bridge_link_ids)]
        network = network[network['linkId'].isin(bridge_link_ids)]

        #Transfer volume info to the netork database
        # share_pop = 0.1
        volumes = []
        pte.index = pte['link']
        for link in network['linkId']:
            volumes.append(np.sum(pte.at[link,'volume'])/share_pop)
        network['volume'] = volumes

        # Bridge_flows = pd.DataFrame()
        ST2.at['henry_hudson_bridge I',name] = network.at[542014, 'volume']
        ST2.at['henry_hudson_bridge O', name] =  network.at[88230, 'volume']
        ST2.at['robert_f_kennedy_bridge Queen I',name] =  network.at[853928, 'volume']
        ST2.at['robert_f_kennedy_bridge Queen O', name] =  network.at[779898, 'volume']
        ST2.at['robert_f_kennedy_bridge Bronx I',name] =  network.at[853900, 'volume']
        ST2.at['robert_f_kennedy_bridge Bronx O', name] =  network.at[30814, 'volume']
        ST2.at['robert_f_kennedy_bridge Manh I',name] =  network.at[154, 'volume']
        ST2.at['robert_f_kennedy_bridge Manh O', name] =  network.at[731748, 'volume']
        ST2.at['queens_midtown_tunnel I',name] =  network.at[1367888, 'volume']
        ST2.at['queens_midtown_tunnel O', name] =  network.at[487778, 'volume']
        ST2.at['hugh_l_carey_tunnel I',name] =  network.at[13722, 'volume']
        ST2.at['hugh_l_carey_tunnel O', name] =  network.at[1071576, 'volume']
        ST2.at['bronx_whitestone_bridge I',name] =  network.at[62416, 'volume']
        ST2.at['bronx_whitestone_bridge O', name] =  network.at[729848, 'volume']
        ST2.at['throgs_neck_bridge I',name] =  network.at[765880, 'volume']
        ST2.at['throgs_neck_bridge O',name] =  network.at[1090614, 'volume']
        ST2.at['varrazzano_narrows_bridge I',name] =  network.at[788122, 'volume']+network.at[788140, 'volume']
        ST2.at['varrazzano_narrows_bridge O',name] =  network.at[788118, 'volume']+ network.at[1341064, 'volume']
        ST2.at['marine_parkwaygil_hodges_memorial_bridge I',name] =  network.at[53416, 'volume']
        ST2.at['marine_parkwaygil_hodges_memorial_bridge O', name] =  network.at[732358, 'volume']
        ST2.at['cross_bay_veterans_memorial_bridge I',name] =  network.at[865062, 'volume']
        ST2.at['cross_bay_veterans_memorial_bridge O', name] =  network.at[999610, 'volume']

        ST2.at['Lincoln Tunnel Total', name] =  network.at[540388, 'volume']+network.at[87186, 'volume'] #network.at[308, 'volume']
        ST2.at['George Washington Bridge Total', name] =  network.at[87176, 'volume']+network.at[767820, 'volume']+network.at[88110, 'volume']+network.at[735454, 'volume']
        ST2.at['Holland Tunnel Total', name] =  network.at[540918, 'volume']+network.at[87188, 'volume']
        ST2.at['Outerbridge Crossing Total', name] =  network.at[852458, 'volume']+network.at[852460, 'volume']
        ST2.at['Goethals Bridge Total',name] =  network.at[1226968, 'volume']+network.at[1292432, 'volume']
        ST2.at['Bayonne Bridge Total', name] =  network.at[759498, 'volume']+network.at[759500, 'volume']

        ST2.at['Lincoln Tunnel I', name] =  network.at[540388, 'volume'] #network.at[308, 'volume']
        ST2.at['George Washington Bridge I', name] =  network.at[87176, 'volume']+network.at[767820, 'volume']
        ST2.at['Holland Tunnel I', name] =  network.at[540918, 'volume']
        ST2.at['Outerbridge Crossing I', name] =  network.at[852460, 'volume']
        ST2.at['Goethals Bridge I', name] =  network.at[1226968, 'volume']
        ST2.at['Bayonne Bridge I', name] =  network.at[759498, 'volume']

        ST2.at['Lincoln Tunnel O', name] =  network.at[87186, 'volume']
        ST2.at['George Washington Bridge O', name] = network.at[88110, 'volume']+network.at[735454, 'volume']
        ST2.at['Holland Tunnel O', name] =  network.at[87188, 'volume']
        ST2.at['Outerbridge Crossing O', name] =  network.at[852458, 'volume']
        ST2.at['Goethals Bridge O', name] = network.at[1292432, 'volume']
        ST2.at['Bayonne Bridge O', name] =  network.at[759500, 'volume']

        ST2.at['PATH Bridge and Tunnel Counts Total', name] = (ST2.at['Lincoln Tunnel Total', name]+
                                                             ST2.at['George Washington Bridge Total', name]+
                                                             ST2.at['Holland Tunnel Total', name]+
                                                             ST2.at['Outerbridge Crossing Total', name]+
                                                             ST2.at['Goethals Bridge Total',name]+
                                                             ST2.at['Bayonne Bridge Total', name])
        ST2.at['PATH Bridge and Tunnel Counts Total O', name] = (ST2.at['Lincoln Tunnel O', name]+
                                                             ST2.at['George Washington Bridge O', name]+
                                                             ST2.at['Holland Tunnel O', name]+
                                                             ST2.at['Outerbridge Crossing O', name]+
                                                             ST2.at['Goethals Bridge O',name]+
                                                             ST2.at['Bayonne Bridge O', name])
        ST2.at['PATH Bridge and Tunnel Counts Total I', name] = (ST2.at['Lincoln Tunnel I', name]+
                                                             ST2.at['George Washington Bridge I', name]+
                                                             ST2.at['Holland Tunnel I', name]+
                                                             ST2.at['Outerbridge Crossing I', name]+
                                                             ST2.at['Goethals Bridge I',name]+
                                                             ST2.at['Bayonne Bridge I', name])
        ST2.at['MTA Bridge and Tunnel Counts Total', name] =     (ST2.at['henry_hudson_bridge I',name] +
                                                            ST2.at['henry_hudson_bridge O', name] +
                                                            ST2.at['robert_f_kennedy_bridge Queen I',name] +
                                                            ST2.at['robert_f_kennedy_bridge Queen O', name] +
                                                            ST2.at['robert_f_kennedy_bridge Bronx I',name] +
                                                            ST2.at['robert_f_kennedy_bridge Bronx O', name] +
                                                            ST2.at['robert_f_kennedy_bridge Manh I',name] +
                                                            ST2.at['robert_f_kennedy_bridge Manh O', name] +
                                                            ST2.at['queens_midtown_tunnel I',name] +
                                                            ST2.at['queens_midtown_tunnel O', name] +
                                                            ST2.at['hugh_l_carey_tunnel I',name] +
                                                            ST2.at['hugh_l_carey_tunnel O', name] +
                                                            ST2.at['bronx_whitestone_bridge I',name] +
                                                            ST2.at['bronx_whitestone_bridge O', name] +
                                                            ST2.at['throgs_neck_bridge I',name] +
                                                            ST2.at['throgs_neck_bridge O',name] +
                                                            ST2.at['varrazzano_narrows_bridge I',name] +
                                                            ST2.at['varrazzano_narrows_bridge O',name] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge I',name] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge O', name] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge I',name] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge O', name])
        ST2.at['MTA Bridge and Tunnel Counts Total I', name] =     (ST2.at['henry_hudson_bridge I',name] +
                                                            ST2.at['robert_f_kennedy_bridge Queen I',name] +
                                                            ST2.at['robert_f_kennedy_bridge Bronx I',name] +
                                                            ST2.at['robert_f_kennedy_bridge Manh I',name] +
                                                            ST2.at['queens_midtown_tunnel I',name] +
                                                            ST2.at['hugh_l_carey_tunnel I',name] +
                                                            ST2.at['bronx_whitestone_bridge I',name] +
                                                            ST2.at['throgs_neck_bridge I',name] +
                                                            ST2.at['varrazzano_narrows_bridge I',name] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge I',name] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge I',name] )
        ST2.at['MTA Bridge and Tunnel Counts Total O', name] =     (
                                                            ST2.at['henry_hudson_bridge O', name] +
                                                            ST2.at['robert_f_kennedy_bridge Queen O', name] +
                                                            ST2.at['robert_f_kennedy_bridge Bronx O', name] +
                                                            ST2.at['robert_f_kennedy_bridge Manh O', name] +
                                                            ST2.at['queens_midtown_tunnel O', name] +
                                                            ST2.at['hugh_l_carey_tunnel O', name] +
                                                            ST2.at['bronx_whitestone_bridge O', name] +
                                                            ST2.at['throgs_neck_bridge O',name] +
                                                            ST2.at['varrazzano_narrows_bridge O',name] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge O', name] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge O', name])
        ST2.at['MTA Bridge and Tunnel Counts Total - only observed bridges/tunnels', name] =     (ST2.at['henry_hudson_bridge I',name] +
                                                            ST2.at['henry_hudson_bridge O', name] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen I',name] +
                                                            # ST2.at['robert_f_kennedy_bridge Manh O', name] +
                                                            ST2.at['queens_midtown_tunnel I',name] +
                                                            ST2.at['queens_midtown_tunnel O', name] +
                                                            ST2.at['hugh_l_carey_tunnel I',name] +
                                                            ST2.at['hugh_l_carey_tunnel O', name] +
                                                            ST2.at['bronx_whitestone_bridge I',name] +
                                                            ST2.at['bronx_whitestone_bridge O', name] +
                                                            ST2.at['throgs_neck_bridge I',name] +
                                                            ST2.at['throgs_neck_bridge O',name] +
                                                            # ST2.at['varrazzano_narrows_bridge O',name] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge I',name] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge O', name] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge I',name] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge O', name])
        ST2.at['MTA Bridge and Tunnel Counts Total I- only observed bridges/tunnels', name] =     (ST2.at['henry_hudson_bridge I',name] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen I',name] +
                                                            ST2.at['queens_midtown_tunnel I',name] +
                                                            ST2.at['hugh_l_carey_tunnel I',name] +
                                                            ST2.at['bronx_whitestone_bridge I',name] +
                                                            ST2.at['throgs_neck_bridge I',name] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge I',name] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge I',name])
        ST2.at['MTA Bridge and Tunnel Counts Total O- only observed bridges/tunnels', name] =     (
                                                            ST2.at['henry_hudson_bridge O', name] +
                                                            # ST2.at['robert_f_kennedy_bridge Manh O', name] +
                                                            ST2.at['queens_midtown_tunnel O', name] +
                                                            ST2.at['hugh_l_carey_tunnel O', name] +
                                                            ST2.at['bronx_whitestone_bridge O', name] +
                                                            ST2.at['throgs_neck_bridge O',name] +
                                                            # ST2.at['varrazzano_narrows_bridge O',name] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge O', name] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge O', name])
    except:
        print('Warning: no linkstat')
ST2.at['MTA Bridge and Tunnel Counts Total', 'Target Baseline'] = '≈907k'
ST2.at['MTA Bridge and Tunnel Counts Total', 'Target 1 - 04-2020'] = '≈341k'
ST2.at['MTA Bridge and Tunnel Counts Total', 'Target 2 - 08-2020'] = '≈787k'
ST2.at['MTA Bridge and Tunnel Counts Total', 'Target 3 - 01-2022'] = '≈695k'
ST2.at['MTA Bridge and Tunnel Counts Total', 'Target 4 - 08-2021'] = '≈933k'
ST2.at['MTA Bridge and Tunnel Counts Total', 'Target 5 - 05-2022'] = '≈955k'

######################BASELINE############################################

ST2.at['Lincoln Tunnel I', 'Target Baseline'] =  40652
ST2.at['George Washington Bridge I', 'Target Baseline'] =  128820
ST2.at['Holland Tunnel I', 'Target Baseline'] = 41931
ST2.at['Outerbridge Crossing I', 'Target Baseline'] =  38062
ST2.at['Goethals Bridge I','Target Baseline'] =  42795
ST2.at['Bayonne Bridge I', 'Target Baseline'] =  9603
ST2.at['PATH Bridge and Tunnel Counts Total I', 'Target Baseline'] =  (ST2.at['Lincoln Tunnel I', 'Target Baseline']+
                                                               ST2.at['George Washington Bridge I', 'Target Baseline']+
                                                               ST2.at['Holland Tunnel I', 'Target Baseline']+
                                                               ST2.at['Outerbridge Crossing I', 'Target Baseline']+
                                                               ST2.at['Goethals Bridge I','Target Baseline']+
                                                               ST2.at['Bayonne Bridge I', 'Target Baseline'])


ST2.at['Lincoln Tunnel I', 'Target 1 - 04-2020'] =  383971/(8*1.182+22)
ST2.at['George Washington Bridge I', 'Target 1 - 04-2020'] =  1529569/(8*1.133+22)
ST2.at['Holland Tunnel I', 'Target 1 - 04-2020'] = 426493/(8*1.088+22)
ST2.at['Outerbridge Crossing I', 'Target 1 - 04-2020'] =  491532/(8*1.156+22)
ST2.at['Goethals Bridge I','Target 1 - 04-2020'] =  486631/(8*1.1566+22)
ST2.at['Bayonne Bridge I', 'Target 1 - 04-2020'] =  127847/(8*1.041+22)
ST2.at['PATH Bridge and Tunnel Counts Total I', 'Target 1 - 04-2020'] =  (ST2.at['Lincoln Tunnel I', 'Target 1 - 04-2020']+
                                                               ST2.at['George Washington Bridge I', 'Target 1 - 04-2020']+
                                                               ST2.at['Holland Tunnel I', 'Target 1 - 04-2020']+
                                                               ST2.at['Outerbridge Crossing I', 'Target 1 - 04-2020']+
                                                               ST2.at['Goethals Bridge I','Target 1 - 04-2020']+
                                                               ST2.at['Bayonne Bridge I', 'Target 1 - 04-2020'])
                                                                                                   

ST2.at['Lincoln Tunnel I', 'Target 2 - 08-2020'] =  1130222/(10*1.182+21)
ST2.at['George Washington Bridge I', 'Target 2 - 08-2020'] =  3818395/(10*1.133+21)
ST2.at['Holland Tunnel I', 'Target 2 - 08-2020'] = 1012757/(10*1.088+21)
ST2.at['Outerbridge Crossing I', 'Target 2 - 08-2020'] =  1188595/(10*1.156+21)
ST2.at['Goethals Bridge I','Target 2 - 08-2020'] =  1303366/(10*1.1566+21)
ST2.at['Bayonne Bridge I', 'Target 2 - 08-2020'] =  257493/(10*1.041+21)
ST2.at['PATH Bridge and Tunnel Counts Total I', 'Target 2 - 08-2020'] =  (ST2.at['Lincoln Tunnel I', 'Target 2 - 08-2020']+
                                                               ST2.at['George Washington Bridge I', 'Target 2 - 08-2020']+
                                                               ST2.at['Holland Tunnel I', 'Target 2 - 08-2020']+
                                                               ST2.at['Outerbridge Crossing I', 'Target 2 - 08-2020']+
                                                               ST2.at['Goethals Bridge I','Target 2 - 08-2020']+
                                                               ST2.at['Bayonne Bridge I', 'Target 2 - 08-2020'])
                                                                

ST2.at['Lincoln Tunnel I', 'Target 3 - 01-2022'] =  1135056/(12*1.182+19)
ST2.at['George Washington Bridge I', 'Target 3 - 01-2022'] =  3190102/(12*1.133+19)
ST2.at['Holland Tunnel I', 'Target 3 - 01-2022'] = 1013607/(12*1.088+19)
ST2.at['Outerbridge Crossing I', 'Target 3 - 01-2022'] =  933625/(12*1.156+19)
ST2.at['Goethals Bridge I','Target 3 - 01-2022'] =  1096469/(12*1.1566+19)
ST2.at['Bayonne Bridge I', 'Target 3 - 01-2022'] =  235078/(12*1.041+19)
ST2.at['PATH Bridge and Tunnel Counts Total I', 'Target 3 - 01-2022'] =  (ST2.at['Lincoln Tunnel I', 'Target 3 - 01-2022']+
                                                               ST2.at['George Washington Bridge I', 'Target 3 - 01-2022']+
                                                               ST2.at['Holland Tunnel I', 'Target 3 - 01-2022']+
                                                               ST2.at['Outerbridge Crossing I', 'Target 3 - 01-2022']+
                                                               ST2.at['Goethals Bridge I','Target 3 - 01-2022']+
                                                               ST2.at['Bayonne Bridge I', 'Target 3 - 01-2022'])
ST2.at['Lincoln Tunnel I', 'Target 4 - 08-2021'] =  1406138/(9*1.182+22)
ST2.at['George Washington Bridge I', 'Target 4 - 08-2021'] =  4216928/(9*1.133+22)
ST2.at['Holland Tunnel I', 'Target 4 - 08-2021'] = 1211681/(9*1.088+22)
ST2.at['Outerbridge Crossing I', 'Target 4 - 08-2021'] =  1235441/(9*1.156+22)
ST2.at['Goethals Bridge I','Target 4 - 08-2021'] =  1428752/(9*1.1566+22)
ST2.at['Bayonne Bridge I', 'Target 4 - 08-2021'] =  321481/(9*1.041+22)
ST2.at['PATH Bridge and Tunnel Counts Total I', 'Target 4 - 08-2021'] =  (ST2.at['Lincoln Tunnel I', 'Target 4 - 08-2021']+
                                                               ST2.at['George Washington Bridge I', 'Target 4 - 08-2021']+
                                                               ST2.at['Holland Tunnel I', 'Target 4 - 08-2021']+
                                                               ST2.at['Outerbridge Crossing I', 'Target 4 - 08-2021']+
                                                               ST2.at['Goethals Bridge I','Target 4 - 08-2021']+
                                                               ST2.at['Bayonne Bridge I', 'Target 4 - 08-2021'])
obs_data = pd.read_csv('inputs/Daily_Traffic_on_MTA_Bridges___Tunnels.csv')

obs_data=obs_data[(obs_data['Date']=='03/02/2020')|
                 (obs_data['Date']=='03/03/2020')|
                 (obs_data['Date']=='03/04/2020')|
                 (obs_data['Date']=='03/05/2020')|
                 (obs_data['Date']=='03/06/2020')]

obs_data['Total Flow'] = obs_data['# Vehicles - E-ZPass']+obs_data['# Vehicles - VToll']
ST2.at['henry_hudson_bridge I','Target Baseline'] =  np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==24)&(obs_data['Direction']=='I')])
ST2.at['henry_hudson_bridge O', 'Target Baseline'] =     np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==24)&(obs_data['Direction']=='O')])
# ST2.at['robert_f_kennedy_bridge Queen I','Target Baseline'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==21)&(obs_data['Direction']=='I')])
# ST2.at['robert_f_kennedy_bridge Queen O', 'Target Baseline'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==21)&(obs_data['Direction']=='O')])
# Bridge_flows.at['robert_f_kennedy_bridge Bronx I','observed'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='I')])
# Bridge_flows.at['robert_f_kennedy_bridge Bronx O', 'observed'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='O')])
# ST2.at['robert_f_kennedy_bridge Manh I','Target Baseline'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==22)&(obs_data['Direction']=='I')])
# ST2.at['robert_f_kennedy_bridge Manh O', 'Target Baseline'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==22)&(obs_data['Direction']=='O')])
ST2.at['queens_midtown_tunnel I','Target Baseline'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==27)&(obs_data['Direction']=='I')])
ST2.at['queens_midtown_tunnel O', 'Target Baseline'] =     np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==27)&(obs_data['Direction']=='O')])
ST2.at['hugh_l_carey_tunnel I','Target Baseline'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==28)&(obs_data['Direction']=='I')])
ST2.at['hugh_l_carey_tunnel O', 'Target Baseline'] =     np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==28)&(obs_data['Direction']=='O')])
ST2.at['bronx_whitestone_bridge I','Target Baseline'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==23)&(obs_data['Direction']=='I')])
ST2.at['bronx_whitestone_bridge O', 'Target Baseline'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==23)&(obs_data['Direction']=='O')])
ST2.at['throgs_neck_bridge I','Target Baseline'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==29)&(obs_data['Direction']=='I')])
ST2.at['throgs_neck_bridge O', 'Target Baseline'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==29)&(obs_data['Direction']=='O')])
# ST2.at['varrazzano_narrows_bridge I','Target Baseline'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==30)&(obs_data['Direction']=='I')])
# ST2.at['varrazzano_narrows_bridge O', 'Target Baseline'] =     np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==30)&(obs_data['Direction']=='O')])
ST2.at['marine_parkwaygil_hodges_memorial_bridge I','Target Baseline'] =   np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==25)&(obs_data['Direction']=='I')])
ST2.at['marine_parkwaygil_hodges_memorial_bridge O', 'Target Baseline'] =   np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==25)&(obs_data['Direction']=='O')])
ST2.at['cross_bay_veterans_memorial_bridge I','Target Baseline'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==26)&(obs_data['Direction']=='I')])
ST2.at['cross_bay_veterans_memorial_bridge O', 'Target Baseline'] =   np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==26)&(obs_data['Direction']=='O')])
ST2.at['MTA Bridge and Tunnel Counts Total - only observed bridges/tunnels', 'Target Baseline'] = (ST2.at['henry_hudson_bridge I','Target Baseline'] +
                                                            ST2.at['henry_hudson_bridge O', 'Target Baseline'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen I','Target Baseline'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen O', 'Target Baseline'] +
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx I','observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='I')])
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx O', 'observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='O')])
                                                            # ST2.at['robert_f_kennedy_bridge Manh I','Target Baseline'] +
                                                            # ST2.at['robert_f_kennedy_bridge Manh O', 'Target Baseline'] +
                                                            ST2.at['queens_midtown_tunnel I','Target Baseline'] +
                                                            ST2.at['queens_midtown_tunnel O', 'Target Baseline'] +
                                                            ST2.at['hugh_l_carey_tunnel I','Target Baseline'] +
                                                            ST2.at['hugh_l_carey_tunnel O', 'Target Baseline'] +
                                                            ST2.at['bronx_whitestone_bridge I','Target Baseline'] +
                                                            ST2.at['bronx_whitestone_bridge O', 'Target Baseline'] +
                                                            ST2.at['throgs_neck_bridge I','Target Baseline'] +
                                                            ST2.at['throgs_neck_bridge O', 'Target Baseline'] +
                                                            # ST2.at['varrazzano_narrows_bridge I','Target Baseline'] +
                                                            # ST2.at['varrazzano_narrows_bridge O', 'Target Baseline'] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge I','Target Baseline'] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge O', 'Target Baseline'] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge I','Target Baseline'] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge O', 'Target Baseline']) 
ST2.at['MTA Bridge and Tunnel Counts Total I- only observed bridges/tunnels', 'Target Baseline'] = (ST2.at['henry_hudson_bridge I','Target Baseline'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen I','Target Baseline'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen O', 'Target Baseline'] +
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx I','observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='I')])
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx O', 'observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='O')])
                                                            # ST2.at['robert_f_kennedy_bridge Manh I','Target Baseline'] +
                                                            ST2.at['queens_midtown_tunnel I','Target Baseline'] +
                                                            ST2.at['hugh_l_carey_tunnel I','Target Baseline'] +
                                                            ST2.at['bronx_whitestone_bridge I','Target Baseline'] +
                                                            ST2.at['throgs_neck_bridge I','Target Baseline'] +
                                                            # ST2.at['varrazzano_narrows_bridge I','Target Baseline'] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge I','Target Baseline'] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge I','Target Baseline'] )
ST2.at['MTA Bridge and Tunnel Counts Total O- only observed bridges/tunnels', 'Target Baseline'] = (
                                                            ST2.at['henry_hudson_bridge O', 'Target Baseline'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen O', 'Target Baseline'] +
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx I','observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='I')])
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx O', 'observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='O')])
                                                            # ST2.at['robert_f_kennedy_bridge Manh I','Target Baseline'] +
                                                            # ST2.at['robert_f_kennedy_bridge Manh O', 'Target Baseline'] +
                                                            ST2.at['queens_midtown_tunnel O', 'Target Baseline'] +
                                                            ST2.at['hugh_l_carey_tunnel O', 'Target Baseline'] +
                                                            ST2.at['bronx_whitestone_bridge O', 'Target Baseline'] +
                                                            ST2.at['throgs_neck_bridge O', 'Target Baseline'] +
                                                            # ST2.at['varrazzano_narrows_bridge I','Target Baseline'] +
                                                            # ST2.at['varrazzano_narrows_bridge O', 'Target Baseline'] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge O', 'Target Baseline'] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge O', 'Target Baseline']) 
######################Scenario1############################################
obs_data = pd.read_csv('inputs/Daily_Traffic_on_MTA_Bridges___Tunnels.csv')

obs_data=obs_data[(obs_data['Date']=='04/03/2020')|
                 (obs_data['Date']=='04/02/2020')|
                 (obs_data['Date']=='04/01/2020')|
                 (obs_data['Date']=='03/31/2020')|
                 (obs_data['Date']=='03/30/2020')]

obs_data['Total Flow'] = obs_data['# Vehicles - E-ZPass']+obs_data['# Vehicles - VToll']
ST2.at['henry_hudson_bridge I','Target 1 - 04-2020'] =  np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==24)&(obs_data['Direction']=='I')])
ST2.at['henry_hudson_bridge O', 'Target 1 - 04-2020'] =     np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==24)&(obs_data['Direction']=='O')])
# ST2.at['robert_f_kennedy_bridge Queen I','Target 1 - 04-2020'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==21)&(obs_data['Direction']=='I')])
# ST2.at['robert_f_kennedy_bridge Queen O', 'Target 1 - 04-2020'] =  np.mean(obs_data['Total Flow']
                                                               # [(obs_data['Plaza ID']==21)&(obs_data['Direction']=='O')])
# Bridge_flows.at['robert_f_kennedy_bridge Bronx I','observed'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='I')])
# Bridge_flows.at['robert_f_kennedy_bridge Bronx O', 'observed'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='O')])
# ST2.at['robert_f_kennedy_bridge Manh I','Target 1 - 04-2020'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==22)&(obs_data['Direction']=='I')])
# ST2.at['robert_f_kennedy_bridge Manh O', 'Target 1 - 04-2020'] =  np.mean(obs_data['Total Flow']
                                                               # [(obs_data['Plaza ID']==22)&(obs_data['Direction']=='O')])
ST2.at['queens_midtown_tunnel I','Target 1 - 04-2020'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==27)&(obs_data['Direction']=='I')])
ST2.at['queens_midtown_tunnel O', 'Target 1 - 04-2020'] =     np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==27)&(obs_data['Direction']=='O')])
ST2.at['hugh_l_carey_tunnel I','Target 1 - 04-2020'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==28)&(obs_data['Direction']=='I')])
ST2.at['hugh_l_carey_tunnel O', 'Target 1 - 04-2020'] =     np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==28)&(obs_data['Direction']=='O')])
ST2.at['bronx_whitestone_bridge I','Target 1 - 04-2020'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==23)&(obs_data['Direction']=='I')])
ST2.at['bronx_whitestone_bridge O', 'Target 1 - 04-2020'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==23)&(obs_data['Direction']=='O')])
ST2.at['throgs_neck_bridge I','Target 1 - 04-2020'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==29)&(obs_data['Direction']=='I')])
ST2.at['throgs_neck_bridge O', 'Target 1 - 04-2020'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==29)&(obs_data['Direction']=='O')])
# ST2.at['varrazzano_narrows_bridge I','Target 1 - 04-2020'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==30)&(obs_data['Direction']=='I')])
# ST2.at['varrazzano_narrows_bridge O', 'Target 1 - 04-2020'] =     np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==30)&(obs_data['Direction']=='O')])
ST2.at['marine_parkwaygil_hodges_memorial_bridge I','Target 1 - 04-2020'] =   np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==25)&(obs_data['Direction']=='I')])
ST2.at['marine_parkwaygil_hodges_memorial_bridge O', 'Target 1 - 04-2020'] =   np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==25)&(obs_data['Direction']=='O')])
ST2.at['cross_bay_veterans_memorial_bridge I','Target 1 - 04-2020'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==26)&(obs_data['Direction']=='I')])
ST2.at['cross_bay_veterans_memorial_bridge O', 'Target 1 - 04-2020'] =   np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==26)&(obs_data['Direction']=='O')])

ST2.at['MTA Bridge and Tunnel Counts Total - only observed bridges/tunnels', 'Target 1 - 04-2020'] = (ST2.at['henry_hudson_bridge I','Target 1 - 04-2020'] +
                                                            ST2.at['henry_hudson_bridge O', 'Target 1 - 04-2020'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen I','Target 1 - 04-2020'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen O', 'Target 1 - 04-2020'] +
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx I','observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='I')])
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx O', 'observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='O')])
                                                            # ST2.at['robert_f_kennedy_bridge Manh I','Target 1 - 04-2020'] +
                                                            # ST2.at['robert_f_kennedy_bridge Manh O', 'Target 1 - 04-2020'] +
                                                            ST2.at['queens_midtown_tunnel I','Target 1 - 04-2020'] +
                                                            ST2.at['queens_midtown_tunnel O', 'Target 1 - 04-2020'] +
                                                            ST2.at['hugh_l_carey_tunnel I','Target 1 - 04-2020'] +
                                                            ST2.at['hugh_l_carey_tunnel O', 'Target 1 - 04-2020'] +
                                                            ST2.at['bronx_whitestone_bridge I','Target 1 - 04-2020'] +
                                                            ST2.at['bronx_whitestone_bridge O', 'Target 1 - 04-2020'] +
                                                            ST2.at['throgs_neck_bridge I','Target 1 - 04-2020'] +
                                                            ST2.at['throgs_neck_bridge O', 'Target 1 - 04-2020'] +
                                                            # ST2.at['varrazzano_narrows_bridge I','Target 1 - 04-2020'] +
                                                            # ST2.at['varrazzano_narrows_bridge O', 'Target 1 - 04-2020'] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge I','Target 1 - 04-2020'] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge O', 'Target 1 - 04-2020'] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge I','Target 1 - 04-2020'] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge O', 'Target 1 - 04-2020'])
ST2.at['MTA Bridge and Tunnel Counts Total I- only observed bridges/tunnels', 'Target 1 - 04-2020'] = (ST2.at['henry_hudson_bridge I','Target 1 - 04-2020'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen I','Target 1 - 04-2020'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen O', 'Target 1 - 04-2020'] +
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx I','observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='I')])
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx O', 'observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='O')])
                                                            # ST2.at['robert_f_kennedy_bridge Manh I','Target 1 - 04-2020'] +
                                                            ST2.at['queens_midtown_tunnel I','Target 1 - 04-2020'] +
                                                            ST2.at['hugh_l_carey_tunnel I','Target 1 - 04-2020'] +
                                                            ST2.at['bronx_whitestone_bridge I','Target 1 - 04-2020'] +
                                                            ST2.at['throgs_neck_bridge I','Target 1 - 04-2020'] +
                                                            # ST2.at['varrazzano_narrows_bridge I','Target 1 - 04-2020'] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge I','Target 1 - 04-2020'] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge I','Target 1 - 04-2020'])
ST2.at['MTA Bridge and Tunnel Counts Total O- only observed bridges/tunnels', 'Target 1 - 04-2020'] = (
                                                            ST2.at['henry_hudson_bridge O', 'Target 1 - 04-2020'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen O', 'Target 1 - 04-2020'] +
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx I','observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='I')])
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx O', 'observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='O')])
                                                            # ST2.at['robert_f_kennedy_bridge Manh I','Target 1 - 04-2020'] +
                                                            # ST2.at['robert_f_kennedy_bridge Manh O', 'Target 1 - 04-2020'] +
                                                            ST2.at['queens_midtown_tunnel O', 'Target 1 - 04-2020'] +
                                                            ST2.at['hugh_l_carey_tunnel O', 'Target 1 - 04-2020'] +
                                                            ST2.at['bronx_whitestone_bridge O', 'Target 1 - 04-2020'] +
                                                            ST2.at['throgs_neck_bridge O', 'Target 1 - 04-2020'] +
                                                            # ST2.at['varrazzano_narrows_bridge I','Target 1 - 04-2020'] +
                                                            # ST2.at['varrazzano_narrows_bridge O', 'Target 1 - 04-2020'] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge O', 'Target 1 - 04-2020'] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge O', 'Target 1 - 04-2020'])
######################Scenario2############################################
obs_data = pd.read_csv('inputs/Daily_Traffic_on_MTA_Bridges___Tunnels.csv')

obs_data=obs_data[(obs_data['Date']=='08/07/2020')|
                 (obs_data['Date']=='08/03/2020')|
                 (obs_data['Date']=='08/04/2020')|
                 (obs_data['Date']=='08/05/2020')|
                 (obs_data['Date']=='08/06/2020')]

obs_data['Total Flow'] = obs_data['# Vehicles - E-ZPass']+obs_data['# Vehicles - VToll']
ST2.at['henry_hudson_bridge I','Target 2 - 08-2020'] =  np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==24)&(obs_data['Direction']=='I')])
ST2.at['henry_hudson_bridge O', 'Target 2 - 08-2020'] =     np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==24)&(obs_data['Direction']=='O')])
# ST2.at['robert_f_kennedy_bridge Queen I','Target 2 - 08-2020'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==21)&(obs_data['Direction']=='I')])
# ST2.at['robert_f_kennedy_bridge Queen O', 'Target 2 - 08-2020'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==21)&(obs_data['Direction']=='O')])
# Bridge_flows.at['robert_f_kennedy_bridge Bronx I','observed'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='I')])
# Bridge_flows.at['robert_f_kennedy_bridge Bronx O', 'observed'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='O')])
# ST2.at['robert_f_kennedy_bridge Manh I','Target 2 - 08-2020'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==22)&(obs_data['Direction']=='I')])
# ST2.at['robert_f_kennedy_bridge Manh O', 'Target 2 - 08-2020'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==22)&(obs_data['Direction']=='O')])
ST2.at['queens_midtown_tunnel I','Target 2 - 08-2020'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==27)&(obs_data['Direction']=='I')])
ST2.at['queens_midtown_tunnel O', 'Target 2 - 08-2020'] =     np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==27)&(obs_data['Direction']=='O')])
ST2.at['hugh_l_carey_tunnel I','Target 2 - 08-2020'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==28)&(obs_data['Direction']=='I')])
ST2.at['hugh_l_carey_tunnel O', 'Target 2 - 08-2020'] =     np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==28)&(obs_data['Direction']=='O')])
ST2.at['bronx_whitestone_bridge I','Target 2 - 08-2020'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==23)&(obs_data['Direction']=='I')])
ST2.at['bronx_whitestone_bridge O', 'Target 2 - 08-2020'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==23)&(obs_data['Direction']=='O')])
ST2.at['throgs_neck_bridge I','Target 2 - 08-2020'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==29)&(obs_data['Direction']=='I')])
ST2.at['throgs_neck_bridge O', 'Target 2 - 08-2020'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==29)&(obs_data['Direction']=='O')])
# ST2.at['varrazzano_narrows_bridge I','Target 2 - 08-2020'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==30)&(obs_data['Direction']=='I')])
# ST2.at['varrazzano_narrows_bridge O', 'Target 2 - 08-2020'] =     np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==30)&(obs_data['Direction']=='O')])
ST2.at['marine_parkwaygil_hodges_memorial_bridge I','Target 2 - 08-2020'] =   np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==25)&(obs_data['Direction']=='I')])
ST2.at['marine_parkwaygil_hodges_memorial_bridge O', 'Target 2 - 08-2020'] =   np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==25)&(obs_data['Direction']=='O')])
ST2.at['cross_bay_veterans_memorial_bridge I','Target 2 - 08-2020'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==26)&(obs_data['Direction']=='I')])
ST2.at['cross_bay_veterans_memorial_bridge O', 'Target 2 - 08-2020'] =   np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==26)&(obs_data['Direction']=='O')])
ST2.at['MTA Bridge and Tunnel Counts Total - only observed bridges/tunnels', 'Target 2 - 08-2020'] = (ST2.at['henry_hudson_bridge I','Target 2 - 08-2020'] +
                                                            ST2.at['henry_hudson_bridge O', 'Target 2 - 08-2020'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen I','Target 2 - 08-2020'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen O', 'Target 2 - 08-2020'] +
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx I','observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='I')])
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx O', 'observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='O')])
                                                            # ST2.at['robert_f_kennedy_bridge Manh I','Target 2 - 08-2020'] +
                                                            # ST2.at['robert_f_kennedy_bridge Manh O', 'Target 2 - 08-2020'] +
                                                            ST2.at['queens_midtown_tunnel I','Target 2 - 08-2020'] +
                                                            ST2.at['queens_midtown_tunnel O', 'Target 2 - 08-2020'] +
                                                            ST2.at['hugh_l_carey_tunnel I','Target 2 - 08-2020'] +
                                                            ST2.at['hugh_l_carey_tunnel O', 'Target 2 - 08-2020'] +
                                                            ST2.at['bronx_whitestone_bridge I','Target 2 - 08-2020'] +
                                                            ST2.at['bronx_whitestone_bridge O', 'Target 2 - 08-2020'] +
                                                            ST2.at['throgs_neck_bridge I','Target 2 - 08-2020'] +
                                                            ST2.at['throgs_neck_bridge O', 'Target 2 - 08-2020'] +
                                                            # ST2.at['varrazzano_narrows_bridge I','Target 2 - 08-2020'] +
                                                            # ST2.at['varrazzano_narrows_bridge O', 'Target 2 - 08-2020'] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge I','Target 2 - 08-2020'] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge O', 'Target 2 - 08-2020'] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge I','Target 2 - 08-2020'] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge O', 'Target 2 - 08-2020'])
ST2.at['MTA Bridge and Tunnel Counts Total I- only observed bridges/tunnels', 'Target 2 - 08-2020'] = (ST2.at['henry_hudson_bridge I','Target 2 - 08-2020'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen I','Target 2 - 08-2020'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen O', 'Target 2 - 08-2020'] +
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx I','observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='I')])
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx O', 'observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='O')])
                                                            # ST2.at['robert_f_kennedy_bridge Manh I','Target 2 - 08-2020'] +
                                                            ST2.at['queens_midtown_tunnel I','Target 2 - 08-2020'] +
                                                            ST2.at['hugh_l_carey_tunnel I','Target 2 - 08-2020'] +
                                                            ST2.at['bronx_whitestone_bridge I','Target 2 - 08-2020'] +
                                                            ST2.at['throgs_neck_bridge I','Target 2 - 08-2020'] +
                                                            # ST2.at['varrazzano_narrows_bridge I','Target 2 - 08-2020'] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge I','Target 2 - 08-2020'] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge I','Target 2 - 08-2020'])
ST2.at['MTA Bridge and Tunnel Counts Total O- only observed bridges/tunnels', 'Target 2 - 08-2020'] = (
                                                            ST2.at['henry_hudson_bridge O', 'Target 2 - 08-2020'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen O', 'Target 2 - 08-2020'] +
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx I','observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='I')])
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx O', 'observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='O')])
                                                            # ST2.at['robert_f_kennedy_bridge Manh I','Target 2 - 08-2020'] +
                                                            # ST2.at['robert_f_kennedy_bridge Manh O', 'Target 2 - 08-2020'] +
                                                            ST2.at['queens_midtown_tunnel O', 'Target 2 - 08-2020'] +
                                                            ST2.at['hugh_l_carey_tunnel O', 'Target 2 - 08-2020'] +
                                                            ST2.at['bronx_whitestone_bridge O', 'Target 2 - 08-2020'] +
                                                            ST2.at['throgs_neck_bridge O', 'Target 2 - 08-2020'] +
                                                            # ST2.at['varrazzano_narrows_bridge I','Target 2 - 08-2020'] +
                                                            # ST2.at['varrazzano_narrows_bridge O', 'Target 2 - 08-2020'] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge O', 'Target 2 - 08-2020'] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge O', 'Target 2 - 08-2020'])
######################Scenario3############################################
obs_data = pd.read_csv('inputs/Daily_Traffic_on_MTA_Bridges___Tunnels.csv')

obs_data=obs_data[(obs_data['Date']=='01/07/2022')|
                 (obs_data['Date']=='01/03/2022')|
                 (obs_data['Date']=='01/04/2022')|
                 (obs_data['Date']=='01/05/2022')|
                 (obs_data['Date']=='01/06/2022')]

obs_data['Total Flow'] = obs_data['# Vehicles - E-ZPass']+obs_data['# Vehicles - VToll']
ST2.at['henry_hudson_bridge I','Target 3 - 01-2022'] =  np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==24)&(obs_data['Direction']=='I')])
ST2.at['henry_hudson_bridge O', 'Target 3 - 01-2022'] =     np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==24)&(obs_data['Direction']=='O')])
# ST2.at['robert_f_kennedy_bridge Queen I','Target 3 - 01-2022'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==21)&(obs_data['Direction']=='I')])
# ST2.at['robert_f_kennedy_bridge Queen O', 'Target 3 - 01-2022'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==21)&(obs_data['Direction']=='O')])
# Bridge_flows.at['robert_f_kennedy_bridge Bronx I','observed'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='I')])
# Bridge_flows.at['robert_f_kennedy_bridge Bronx O', 'observed'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='O')])
# ST2.at['robert_f_kennedy_bridge Manh I','Target 3 - 01-2022'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==22)&(obs_data['Direction']=='I')])
# ST2.at['robert_f_kennedy_bridge Manh O', 'Target 3 - 01-2022'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==22)&(obs_data['Direction']=='O')])
ST2.at['queens_midtown_tunnel I','Target 3 - 01-2022'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==27)&(obs_data['Direction']=='I')])
ST2.at['queens_midtown_tunnel O', 'Target 3 - 01-2022'] =     np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==27)&(obs_data['Direction']=='O')])
ST2.at['hugh_l_carey_tunnel I','Target 3 - 01-2022'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==28)&(obs_data['Direction']=='I')])
ST2.at['hugh_l_carey_tunnel O', 'Target 3 - 01-2022'] =     np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==28)&(obs_data['Direction']=='O')])
ST2.at['bronx_whitestone_bridge I','Target 3 - 01-2022'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==23)&(obs_data['Direction']=='I')])
ST2.at['bronx_whitestone_bridge O', 'Target 3 - 01-2022'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==23)&(obs_data['Direction']=='O')])
ST2.at['throgs_neck_bridge I','Target 3 - 01-2022'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==29)&(obs_data['Direction']=='I')])
ST2.at['throgs_neck_bridge O', 'Target 3 - 01-2022'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==29)&(obs_data['Direction']=='O')])
# ST2.at['varrazzano_narrows_bridge I','Target 3 - 01-2022'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==30)&(obs_data['Direction']=='I')])
# ST2.at['varrazzano_narrows_bridge O', 'Target 3 - 01-2022'] =     np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==30)&(obs_data['Direction']=='O')])
ST2.at['marine_parkwaygil_hodges_memorial_bridge I','Target 3 - 01-2022'] =   np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==25)&(obs_data['Direction']=='I')])
ST2.at['marine_parkwaygil_hodges_memorial_bridge O', 'Target 3 - 01-2022'] =   np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==25)&(obs_data['Direction']=='O')])
ST2.at['cross_bay_veterans_memorial_bridge I','Target 3 - 01-2022'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==26)&(obs_data['Direction']=='I')])
ST2.at['cross_bay_veterans_memorial_bridge O', 'Target 3 - 01-2022'] =   np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==26)&(obs_data['Direction']=='O')])
ST2.at['MTA Bridge and Tunnel Counts Total - only observed bridges/tunnels', 'Target 3 - 01-2022'] = (ST2.at['henry_hudson_bridge I','Target 3 - 01-2022'] +
                                                            ST2.at['henry_hudson_bridge O', 'Target 3 - 01-2022'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen I','Target 3 - 01-2022'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen O', 'Target Baseline'] +
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx I','observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='I')])
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx O', 'observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='O')])
                                                            # ST2.at['robert_f_kennedy_bridge Manh I','Target Baseline'] +
                                                            # ST2.at['robert_f_kennedy_bridge Manh O', 'Target 3 - 01-2022'] +
                                                            ST2.at['queens_midtown_tunnel I','Target 3 - 01-2022'] +
                                                            ST2.at['queens_midtown_tunnel O', 'Target 3 - 01-2022'] +
                                                            ST2.at['hugh_l_carey_tunnel I','Target 3 - 01-2022'] +
                                                            ST2.at['hugh_l_carey_tunnel O', 'Target 3 - 01-2022'] +
                                                            ST2.at['bronx_whitestone_bridge I','Target 3 - 01-2022'] +
                                                            ST2.at['bronx_whitestone_bridge O', 'Target 3 - 01-2022'] +
                                                            ST2.at['throgs_neck_bridge I','Target 3 - 01-2022'] +
                                                            ST2.at['throgs_neck_bridge O', 'Target 3 - 01-2022'] +
                                                            # ST2.at['varrazzano_narrows_bridge I','Target Baseline'] +
                                                            # ST2.at['varrazzano_narrows_bridge O', 'Target 3 - 01-2022'] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge I','Target 3 - 01-2022'] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge O', 'Target 3 - 01-2022'] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge I','Target 3 - 01-2022'] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge O', 'Target 3 - 01-2022'])
ST2.at['MTA Bridge and Tunnel Counts Total I- only observed bridges/tunnels', 'Target 3 - 01-2022'] = (ST2.at['henry_hudson_bridge I','Target 3 - 01-2022'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen I','Target 3 - 01-2022'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen O', 'Target Baseline'] +
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx I','observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='I')])
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx O', 'observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='O')])
                                                            # ST2.at['robert_f_kennedy_bridge Manh I','Target Baseline'] +
                                                            ST2.at['queens_midtown_tunnel I','Target 3 - 01-2022'] +
                                                            ST2.at['hugh_l_carey_tunnel I','Target 3 - 01-2022'] +
                                                            ST2.at['bronx_whitestone_bridge I','Target 3 - 01-2022'] +
                                                            ST2.at['throgs_neck_bridge I','Target 3 - 01-2022'] +
                                                            # ST2.at['varrazzano_narrows_bridge I','Target Baseline'] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge I','Target 3 - 01-2022'] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge I','Target 3 - 01-2022'] )
ST2.at['MTA Bridge and Tunnel Counts Total O- only observed bridges/tunnels', 'Target 3 - 01-2022'] = (
                                                            ST2.at['henry_hudson_bridge O', 'Target 3 - 01-2022'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen O', 'Target Baseline'] +
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx I','observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='I')])
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx O', 'observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='O')])
                                                            # ST2.at['robert_f_kennedy_bridge Manh I','Target Baseline'] +
                                                            # ST2.at['robert_f_kennedy_bridge Manh O', 'Target 3 - 01-2022'] +
                                                            ST2.at['queens_midtown_tunnel O', 'Target 3 - 01-2022'] +
                                                            ST2.at['hugh_l_carey_tunnel O', 'Target 3 - 01-2022'] +
                                                            ST2.at['bronx_whitestone_bridge O', 'Target 3 - 01-2022'] +
                                                            ST2.at['throgs_neck_bridge O', 'Target 3 - 01-2022'] +
                                                            # ST2.at['varrazzano_narrows_bridge I','Target Baseline'] +
                                                            # ST2.at['varrazzano_narrows_bridge O', 'Target 3 - 01-2022'] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge O', 'Target 3 - 01-2022'] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge O', 'Target 3 - 01-2022'])
######################Scenario4############################################
obs_data = pd.read_csv('inputs/Daily_Traffic_on_MTA_Bridges___Tunnels.csv')

obs_data=obs_data[(obs_data['Date']=='08/02/2021')|
                 (obs_data['Date']=='08/03/2021')|
                 (obs_data['Date']=='08/04/2021')|
                 (obs_data['Date']=='08/05/2021')|
                 (obs_data['Date']=='08/06/2021')]

obs_data['Total Flow'] = obs_data['# Vehicles - E-ZPass']+obs_data['# Vehicles - VToll']
ST2.at['henry_hudson_bridge I','Target 4 - 08-2021'] =  np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==24)&(obs_data['Direction']=='I')])
ST2.at['henry_hudson_bridge O', 'Target 4 - 08-2021'] =     np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==24)&(obs_data['Direction']=='O')])
# ST2.at['robert_f_kennedy_bridge Queen I','Target 4 - 08-2021'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==21)&(obs_data['Direction']=='I')])
# ST2.at['robert_f_kennedy_bridge Queen O', 'Target 4 - 08-2021'] =  np.mean(obs_data['Total Flow']
                                                               # [(obs_data['Plaza ID']==21)&(obs_data['Direction']=='O')])
# Bridge_flows.at['robert_f_kennedy_bridge Bronx I','observed'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='I')])
# Bridge_flows.at['robert_f_kennedy_bridge Bronx O', 'observed'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='O')])
# ST2.at['robert_f_kennedy_bridge Manh I','Target 4 - 08-2021'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==22)&(obs_data['Direction']=='I')])
# ST2.at['robert_f_kennedy_bridge Manh O', 'Target 4 - 08-2021'] =  np.mean(obs_data['Total Flow']
                                                               # [(obs_data['Plaza ID']==22)&(obs_data['Direction']=='O')])
ST2.at['queens_midtown_tunnel I','Target 4 - 08-2021'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==27)&(obs_data['Direction']=='I')])
ST2.at['queens_midtown_tunnel O', 'Target 4 - 08-2021'] =     np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==27)&(obs_data['Direction']=='O')])
ST2.at['hugh_l_carey_tunnel I','Target 4 - 08-2021'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==28)&(obs_data['Direction']=='I')])
ST2.at['hugh_l_carey_tunnel O', 'Target 4 - 08-2021'] =     np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==28)&(obs_data['Direction']=='O')])
ST2.at['bronx_whitestone_bridge I','Target 4 - 08-2021'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==23)&(obs_data['Direction']=='I')])
ST2.at['bronx_whitestone_bridge O', 'Target 4 - 08-2021'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==23)&(obs_data['Direction']=='O')])
ST2.at['throgs_neck_bridge I','Target 4 - 08-2021'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==29)&(obs_data['Direction']=='I')])
ST2.at['throgs_neck_bridge O', 'Target 4 - 08-2021'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==29)&(obs_data['Direction']=='O')])
# ST2.at['varrazzano_narrows_bridge I','Target 4 - 08-2021'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==30)&(obs_data['Direction']=='I')])
# ST2.at['varrazzano_narrows_bridge O', 'Target 4 - 08-2021'] =     np.mean(obs_data['Total Flow']
                                                               # [(obs_data['Plaza ID']==30)&(obs_data['Direction']=='O')])
ST2.at['marine_parkwaygil_hodges_memorial_bridge I','Target 4 - 08-2021'] =   np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==25)&(obs_data['Direction']=='I')])
ST2.at['marine_parkwaygil_hodges_memorial_bridge O', 'Target 4 - 08-2021'] =   np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==25)&(obs_data['Direction']=='O')])
ST2.at['cross_bay_veterans_memorial_bridge I','Target 4 - 08-2021'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==26)&(obs_data['Direction']=='I')])
ST2.at['cross_bay_veterans_memorial_bridge O', 'Target 4 - 08-2021'] =   np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==26)&(obs_data['Direction']=='O')])
ST2.at['MTA Bridge and Tunnel Counts Total - only observed bridges/tunnels', 'Target 4 - 08-2021'] = (ST2.at['henry_hudson_bridge I','Target 4 - 08-2021'] +
                                                            ST2.at['henry_hudson_bridge O', 'Target 4 - 08-2021'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen I','Target 4 - 08-2021'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen O', 'Target Baseline'] +
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx I','observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='I')])
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx O', 'observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='O')])
                                                            # ST2.at['robert_f_kennedy_bridge Manh I','Target Baseline'] +
                                                            # ST2.at['robert_f_kennedy_bridge Manh O', 'Target 4 - 08-2021'] +
                                                            ST2.at['queens_midtown_tunnel I','Target 4 - 08-2021'] +
                                                            ST2.at['queens_midtown_tunnel O', 'Target 4 - 08-2021'] +
                                                            ST2.at['hugh_l_carey_tunnel I','Target 4 - 08-2021'] +
                                                            ST2.at['hugh_l_carey_tunnel O', 'Target 4 - 08-2021'] +
                                                            ST2.at['bronx_whitestone_bridge I','Target 4 - 08-2021'] +
                                                            ST2.at['bronx_whitestone_bridge O', 'Target 4 - 08-2021'] +
                                                            ST2.at['throgs_neck_bridge I','Target 4 - 08-2021'] +
                                                            ST2.at['throgs_neck_bridge O', 'Target 4 - 08-2021'] +
                                                            # ST2.at['varrazzano_narrows_bridge I','Target Baseline'] +
                                                            # ST2.at['varrazzano_narrows_bridge O', 'Target 4 - 08-2021'] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge I','Target 4 - 08-2021'] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge O', 'Target 4 - 08-2021'] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge I','Target 4 - 08-2021'] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge O', 'Target 4 - 08-2021'])
ST2.at['MTA Bridge and Tunnel Counts Total I- only observed bridges/tunnels', 'Target 4 - 08-2021'] = (ST2.at['henry_hudson_bridge I','Target 4 - 08-2021'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen I','Target 4 - 08-2021'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen O', 'Target Baseline'] +
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx I','observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='I')])
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx O', 'observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='O')])
                                                            # ST2.at['robert_f_kennedy_bridge Manh I','Target Baseline'] +
                                                            ST2.at['queens_midtown_tunnel I','Target 4 - 08-2021'] +
                                                            ST2.at['hugh_l_carey_tunnel I','Target 4 - 08-2021'] +
                                                            ST2.at['bronx_whitestone_bridge I','Target 4 - 08-2021'] +
                                                            ST2.at['throgs_neck_bridge I','Target 4 - 08-2021'] +
                                                            # ST2.at['varrazzano_narrows_bridge I','Target Baseline'] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge I','Target 4 - 08-2021'] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge I','Target 4 - 08-2021'])
ST2.at['MTA Bridge and Tunnel Counts Total O- only observed bridges/tunnels', 'Target 4 - 08-2021'] = (ST2.at['henry_hudson_bridge O', 'Target 4 - 08-2021'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen O', 'Target Baseline'] +
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx I','observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='I')])
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx O', 'observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='O')])
                                                            # ST2.at['robert_f_kennedy_bridge Manh I','Target Baseline'] +
                                                            # ST2.at['robert_f_kennedy_bridge Manh O', 'Target 4 - 08-2021'] +
                                                            ST2.at['queens_midtown_tunnel O', 'Target 4 - 08-2021'] +
                                                            ST2.at['hugh_l_carey_tunnel O', 'Target 4 - 08-2021'] +
                                                            ST2.at['bronx_whitestone_bridge O', 'Target 4 - 08-2021'] +
                                                            ST2.at['throgs_neck_bridge O', 'Target 4 - 08-2021'] +
                                                            # ST2.at['varrazzano_narrows_bridge I','Target Baseline'] +
                                                            # ST2.at['varrazzano_narrows_bridge O', 'Target 4 - 08-2021'] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge O', 'Target 4 - 08-2021'] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge O', 'Target 4 - 08-2021'])

######################Scenario5############################################
obs_data = pd.read_csv('inputs/Daily_Traffic_on_MTA_Bridges___Tunnels.csv')

obs_data=obs_data[(obs_data['Date']=='05/09/2021')|
                 (obs_data['Date']=='05/10/2021')|
                 (obs_data['Date']=='05/11/2021')|
                 (obs_data['Date']=='05/12/2021')|
                 (obs_data['Date']=='05/13/2021')]

obs_data['Total Flow'] = obs_data['# Vehicles - E-ZPass']+obs_data['# Vehicles - VToll']
ST2.at['henry_hudson_bridge I','Target 5 - 05-2022'] =  np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==24)&(obs_data['Direction']=='I')])
ST2.at['henry_hudson_bridge O', 'Target 5 - 05-2022'] =     np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==24)&(obs_data['Direction']=='O')])
# ST2.at['robert_f_kennedy_bridge Queen I','Target 4 - 08-2021'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==21)&(obs_data['Direction']=='I')])
# ST2.at['robert_f_kennedy_bridge Queen O', 'Target 4 - 08-2021'] =  np.mean(obs_data['Total Flow']
                                                               # [(obs_data['Plaza ID']==21)&(obs_data['Direction']=='O')])
# Bridge_flows.at['robert_f_kennedy_bridge Bronx I','observed'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='I')])
# Bridge_flows.at['robert_f_kennedy_bridge Bronx O', 'observed'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='O')])
# ST2.at['robert_f_kennedy_bridge Manh I','Target 4 - 08-2021'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==22)&(obs_data['Direction']=='I')])
# ST2.at['robert_f_kennedy_bridge Manh O', 'Target 4 - 08-2021'] =  np.mean(obs_data['Total Flow']
                                                               # [(obs_data['Plaza ID']==22)&(obs_data['Direction']=='O')])
ST2.at['queens_midtown_tunnel I','Target 5 - 05-2022'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==27)&(obs_data['Direction']=='I')])
ST2.at['queens_midtown_tunnel O', 'Target 5 - 05-2022'] =     np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==27)&(obs_data['Direction']=='O')])
ST2.at['hugh_l_carey_tunnel I','Target 5 - 05-2022'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==28)&(obs_data['Direction']=='I')])
ST2.at['hugh_l_carey_tunnel O', 'Target 5 - 05-2022'] =     np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==28)&(obs_data['Direction']=='O')])
ST2.at['bronx_whitestone_bridge I','Target 5 - 05-2022'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==23)&(obs_data['Direction']=='I')])
ST2.at['bronx_whitestone_bridge O', 'Target 5 - 05-2022'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==23)&(obs_data['Direction']=='O')])
ST2.at['throgs_neck_bridge I','Target 5 - 05-2022'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==29)&(obs_data['Direction']=='I')])
ST2.at['throgs_neck_bridge O', 'Target 5 - 05-2022'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==29)&(obs_data['Direction']=='O')])
# ST2.at['varrazzano_narrows_bridge I','Target 4 - 08-2021'] =  np.mean(obs_data['Total Flow']
#                                                                [(obs_data['Plaza ID']==30)&(obs_data['Direction']=='I')])
# ST2.at['varrazzano_narrows_bridge O', 'Target 4 - 08-2021'] =     np.mean(obs_data['Total Flow']
                                                               # [(obs_data['Plaza ID']==30)&(obs_data['Direction']=='O')])
ST2.at['marine_parkwaygil_hodges_memorial_bridge I','Target 5 - 05-2022'] =   np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==25)&(obs_data['Direction']=='I')])
ST2.at['marine_parkwaygil_hodges_memorial_bridge O', 'Target 5 - 05-2022'] =   np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==25)&(obs_data['Direction']=='O')])
ST2.at['cross_bay_veterans_memorial_bridge I','Target 5 - 05-2022'] =    np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==26)&(obs_data['Direction']=='I')])
ST2.at['cross_bay_veterans_memorial_bridge O', 'Target 5 - 05-2022'] =   np.mean(obs_data['Total Flow']
                                                               [(obs_data['Plaza ID']==26)&(obs_data['Direction']=='O')])
ST2.at['MTA Bridge and Tunnel Counts Total - only observed bridges/tunnels', 'Target 5 - 05-2022'] = (ST2.at['henry_hudson_bridge I','Target 5 - 05-2022'] +
                                                            ST2.at['henry_hudson_bridge O', 'Target 5 - 05-2022'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen I','Target 4 - 08-2021'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen O', 'Target Baseline'] +
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx I','observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='I')])
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx O', 'observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='O')])
                                                            # ST2.at['robert_f_kennedy_bridge Manh I','Target Baseline'] +
                                                            # ST2.at['robert_f_kennedy_bridge Manh O', 'Target 4 - 08-2021'] +
                                                            ST2.at['queens_midtown_tunnel I','Target 5 - 05-2022'] +
                                                            ST2.at['queens_midtown_tunnel O', 'Target 5 - 05-2022'] +
                                                            ST2.at['hugh_l_carey_tunnel I','Target 5 - 05-2022'] +
                                                            ST2.at['hugh_l_carey_tunnel O', 'Target 5 - 05-2022'] +
                                                            ST2.at['bronx_whitestone_bridge I','Target 5 - 05-2022'] +
                                                            ST2.at['bronx_whitestone_bridge O', 'Target 5 - 05-2022'] +
                                                            ST2.at['throgs_neck_bridge I','Target 5 - 05-2022'] +
                                                            ST2.at['throgs_neck_bridge O', 'Target 5 - 05-2022'] +
                                                            # ST2.at['varrazzano_narrows_bridge I','Target Baseline'] +
                                                            # ST2.at['varrazzano_narrows_bridge O', 'Target 4 - 08-2021'] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge I','Target 5 - 05-2022'] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge O', 'Target 5 - 05-2022'] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge I','Target 5 - 05-2022'] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge O', 'Target 5 - 05-2022'])
ST2.at['MTA Bridge and Tunnel Counts Total I- only observed bridges/tunnels', 'Target 5 - 05-2022'] = (ST2.at['henry_hudson_bridge I','Target 5 - 05-2022'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen I','Target 4 - 08-2021'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen O', 'Target Baseline'] +
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx I','observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='I')])
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx O', 'observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='O')])
                                                            # ST2.at['robert_f_kennedy_bridge Manh I','Target Baseline'] +
                                                            ST2.at['queens_midtown_tunnel I','Target 5 - 05-2022'] +
                                                            ST2.at['hugh_l_carey_tunnel I','Target 5 - 05-2022'] +
                                                            ST2.at['bronx_whitestone_bridge I','Target 5 - 05-2022'] +
                                                            ST2.at['throgs_neck_bridge I','Target 5 - 05-2022'] +
                                                            # ST2.at['varrazzano_narrows_bridge I','Target Baseline'] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge I','Target 5 - 05-2022'] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge I','Target 5 - 05-2022'])
ST2.at['MTA Bridge and Tunnel Counts Total O- only observed bridges/tunnels', 'Target 5 - 05-2022'] = (ST2.at['henry_hudson_bridge O', 'Target 5 - 05-2022'] +
                                                            # ST2.at['robert_f_kennedy_bridge Queen O', 'Target Baseline'] +
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx I','observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='I')])
                                                            # Bridge_flows.at['robert_f_kennedy_bridge Bronx O', 'observed'] =  np.mean(obs_data['Total Flow']
                                                            #                                                                [(obs_data['Plaza ID']==)&(obs_data['Direction']=='O')])
                                                            # ST2.at['robert_f_kennedy_bridge Manh I','Target Baseline'] +
                                                            # ST2.at['robert_f_kennedy_bridge Manh O', 'Target 4 - 08-2021'] +
                                                            ST2.at['queens_midtown_tunnel O', 'Target 5 - 05-2022'] +
                                                            ST2.at['hugh_l_carey_tunnel O', 'Target 5 - 05-2022'] +
                                                            ST2.at['bronx_whitestone_bridge O', 'Target 5 - 05-2022'] +
                                                            ST2.at['throgs_neck_bridge O', 'Target 5 - 05-2022'] +
                                                            # ST2.at['varrazzano_narrows_bridge I','Target Baseline'] +
                                                            # ST2.at['varrazzano_narrows_bridge O', 'Target 4 - 08-2021'] +
                                                            ST2.at['marine_parkwaygil_hodges_memorial_bridge O', 'Target 5 - 05-2022'] +
                                                            ST2.at['cross_bay_veterans_memorial_bridge O', 'Target 5 - 05-2022'])

ST2.to_csv('outputs/validationNYC16.csv')


In [ ]:
ST = pd.read_csv('outputs/SummaryTable_Pilates_RHM5.csv', index_col = 'Unnamed: 0')

#Validation RHM
STRH = pd.DataFrame()

for name in names:
    
    STRH.at['Trip Est Ride Hail', name] = ST.at['Trip Est Ride Hail', name]
    STRH.at['Trip Est Ride Hail-Transit', name] = ST.at['Trip Est Ride Hail-Transit', name]
    STRH.at['Trip Est Ride Hail Pooled', name] = ST.at['Trip Est Ride Hail Pooled', name]
    STRH.at['Trip Mode Ride Hail', name] = ST.at['Trip Mode Ride Hail', name]
    STRH.at['Trip Mode Ride Hail-Transit', name] = ST.at['Trip Mode Ride Hail-Transit', name]
    STRH.at['Trip Mode Ride Hail Pooled', name] = ST.at['Trip Mode Ride Hail Pooled', name]
    STRH.at['Trip Replan Ride Hail Share', name] = (ST.at['Trip Replan Ride Hail', name]+
                                                    ST.at['Trip Replan Ride Hail-Transit', name]+
                                                    ST.at['Trip Replan Ride Hail Pooled', name])/(ST.at['Trip Mode Ride Hail', name]+
                                                                                                  ST.at['Trip Mode Ride Hail-Transit', name]+
                                                                                                  ST.at['Trip Mode Ride Hail Pooled', name])
                                                        
    STRH.at['Trip Replan Ride Hail', name] = ST.at['Trip Replan Ride Hail', name]
    STRH.at['Trip Replan Ride Hail-Transit', name] = ST.at['Trip Replan Ride Hail-Transit', name]
    STRH.at['Trip Replan Ride Hail Pooled', name] = ST.at['Trip Replan Ride Hail Pooled', name]
    STRH.at['Trip Exec Ride Hail', name] = ST.at['Trip Exec Ride Hail', name]
    STRH.at['Trip Exec Ride Hail-Transit', name] = ST.at['Trip Exec Ride Hail-Transit', name]
    STRH.at['Trip Exec Ride Hail Pooled', name] = ST.at['Trip Exec Ride Hail Pooled', name]
    STRH.at['Percentage RH Pooled Trips', name] = (ST.at['Trip Exec Ride Hail Pooled', name])/(ST.at['Trip Exec Ride Hail', name]+ST.at['Trip Exec Ride Hail Pooled', name])
    STRH.at['Daily Trips per RH Driver', name] = (ST.at['Trip Exec Ride Hail', name]+
                                                 ST.at['Trip Exec Ride Hail-Transit', name]+
                                                 ST.at['Trip Exec Ride Hail Pooled', name])/ST.at['Number RH vehicles', name]
    STRH.at['Daily RH Trips', name] = (ST.at['Trip Exec Ride Hail', name]+
                                                 ST.at['Trip Exec Ride Hail-Transit', name]+
                                                 ST.at['Trip Exec Ride Hail Pooled', name])
    STRH.at['VMT Ride Hail [km]', name] = ST.at['Length Vehicle SUM Ride Hail [km]', name]+ST.at['Length Vehicle SUM Empty Ride Hail [km]', name]
    STRH.at['Length Vehicle AV Ride Hail [km]', name] = ST.at['Length Vehicle AV Ride Hail [km]', name]
    STRH.at['Length Vehicle AV Empty Ride Hail [km]', name] = ST.at['Length Vehicle AV Empty Ride Hail [km]', name]
    STRH.at['Share deadhead VMT', name] = STRH.at['Length Vehicle AV Empty Ride Hail [km]', name]/(STRH.at['Length Vehicle AV Ride Hail [km]', name]+STRH.at['Length Vehicle AV Empty Ride Hail [km]', name])
    STRH.at['Duration Vehicle AV Ride Hail [h]', name] = ST.at['Duration Vehicle AV Ride Hail [h]', name]
    STRH.at['Duration Vehicle AV Empty Ride Hail [h]', name] = ST.at['Duration Vehicle AV Empty Ride Hail [h]', name]
    STRH.at['Duration Person AV Ride Hail [h]', name] = ST.at['Duration Person AV Ride Hail [h]', name]
    STRH.at['Duration Person AV Empty Ride Hail [h]', name] = ST.at['Duration Person AV Empty Ride Hail [h]', name]
    STRH.at['Speed Vehicle AV Ride Hail [km/h]', name] = (ST.at['Length Vehicle SUM Ride Hail [km]', name])/(
                                                        ST.at['Duration Vehicle SUM Ride Hail [h]', name])
    STRH.at['Number RH vehicles', name] = ST.at['Number RH vehicles', name]
    STRH.at['AV Waiting Time RH [s]', name] = ST.at['AV Waiting Time RH', name]
    STRH.at['AV Waiting Time RH Pooled [s]', name] = ST.at['AV Waiting Time RH Pooled', name]
    STRH.at['AV Waiting Time RH Single [s]', name] = ST.at['AV Waiting Time RH Single', name]

STRH.at['Number RH vehicles', 'CPUC'] = 16920
STRH.at['Length Vehicle AV Ride Hail [km]', 'CPUC'] = 6.88*1.60934
STRH.at['Length Vehicle AV Empty Ride Hail [km]', 'CPUC'] = 2.54*1.60934
STRH.at['Daily Trips per RH Driver', 'CPUC'] = 26.8
STRH.at['Daily RH Trips', 'CPUC'] = 353000
STRH.at['Duration Vehicle AV Ride Hail [h]', 'CPUC'] = 0.286
STRH.at['AV Waiting Time RH [s]', 'CPUC'] = 344
STRH.at['Percentage RH Pooled Trips', 'CPUC'] = 0.14
STRH.at['Speed Vehicle AV Ride Hail [km/h]', 'CPUC'] = 24.4*1.60934
STRH.at['Trip Replan Ride Hail Share', 'CPUC'] = 0.018
STRH.at['VMT Ride Hail [km]', 'CPUC'] = 3300000*1.60934

STRH.at['Number RH vehicles', 'CPUC 10% Sample'] = 16920/10
STRH.at['Length Vehicle AV Ride Hail [km]', 'CPUC 10% Sample'] = 6.88*1.60934
STRH.at['Length Vehicle AV Empty Ride Hail [km]', 'CPUC 10% Sample'] = 2.54*1.60934
STRH.at['Daily Trips per RH Driver', 'CPUC 10% Sample'] = 26.8
STRH.at['Daily RH Trips', 'CPUC 10% Sample'] = 353000/10
STRH.at['Duration Vehicle AV Ride Hail [h]', 'CPUC 10% Sample'] = 0.286
STRH.at['AV Waiting Time RH [s]', 'CPUC 10% Sample'] = 344
STRH.at['Percentage RH Pooled Trips', 'CPUC 10% Sample'] = 0.14
STRH.at['Speed Vehicle AV Ride Hail [km/h]', 'CPUC 10% Sample'] = 24.4*1.60934
STRH.at['Trip Replan Ride Hail Share', 'CPUC 10% Sample'] = 0.018
STRH.at['VMT Ride Hail [km]', 'CPUC 10% Sample'] = 3300000*1.60934/10

STRH.at['Share deadhead VMT', name] = STRH.at['Length Vehicle AV Empty Ride Hail [km]', 'CPUC 10% Sample']/(STRH.at['Length Vehicle AV Empty Ride Hail [km]', 'CPUC 10% Sample']+STRH.at['Length Vehicle AV Ride Hail [km]', 'CPUC 10% Sample'])


STRH.to_csv('outputs/validationRH5.csv')

In [ ]:
def mergePathTraversals(names, PTss, PtoPTss):
    print('---------------------------------------------------------------')
    print('mergePathTraversals...')
    start = time.time()
    count = 0
    for name in names:
        PTs = PTss[name]
        PtoPTs = PtoPTss[name]
        PtoPTs_toappend = []
        if count == 0:
            MergedPTs = PTs
            MergedPtoPTs = PtoPTs
            len_initial_PtoPTs = len(MergedPtoPTs)
            len_initial_PTs = len(MergedPTs)
            print('len(MergedPTs)',len_initial_PTs)
            print('len(MergedPtoPTs)',len_initial_PtoPTs)

        else:
            
            new_PtoPTs_rows_tr_exp = 0

            print('modes recorded:',PTs['mode'].value_counts().keys())
            print('you are considering walk, bike, car, bus, subway, rail, tram, ferry')
            

            #Public modes
            PTs_Toup = PTs[(PTs['mode']=='bus')|(PTs['mode']=='subway')|(PTs['mode']=='rail')|(PTs['mode']=='tram')|(PTs['mode']=='ferry')]
            PTs_indexes = list(PTs_Toup.index)
            #Filtert public PtoPTs
            PtoPTs_toadd = PtoPTs[PtoPTs['pathTraversalID'].isin(PTs_indexes)]
            print('Pts to update for public transport mode',len(PTs_Toup))
            #Group PTS per 'departureTime', 'vehicle','startX'
            PTs_Toup_lookup = PTs_Toup.groupby(['departureTime', 'vehicle','startX','endY','duration']).apply(lambda x: [x.index,
                                                    x.occupancy,x.passengerMiles,x.capacity]).to_dict()
            #Group mergedPTS per 'departureTime', 'vehicle','startX'
            MergedPTs_lookup = MergedPTs.groupby(['departureTime', 'vehicle','startX','endY','duration']).apply(lambda x: list(x.index)).to_dict()
            
            i=0
            
            a = time.time()
            b = time.time()
            for key, val in PTs_Toup_lookup.items():
                #tracking remaining time
                i+=1
                if i%10000 == 0:
                    print(i)
                    print(time.time()-a,'remaining time:',(time.time()-a)*(len(PTs_Toup)-1)/10000-(a-b),'s')
                    a = time.time()
                try:
                    #look for the relative pathtraversal index on first simulation
                    index = MergedPTs_lookup[(key[0],key[1],key[2],key[3],key[4])]
                    #check number of pathtraversal found...if more than one probably is due to zero durations
                    if len(index)!=1:
                        print('mmh...')
                        print('too many correspondances in the Merged PTs',len(index))
                        print(index)
                        print('PTs to update: depTime, vehicle, startX,endY,duration,capacity,passengerMiles,occupancy-------->',
                              key[0],key[1],key[2],key[3],key[4],[list(val[3])[0],list(val[2])[0],list(val[1])[0]])
                        print('Merget Pts -------->',MergedPTs.loc[index[:], ['vehicle','departureTime','capacity','passengerMiles','occupancy','startX']])
                    else:
                        PtoPTs_toadd = PtoPTs.iloc[np.where(PtoPTs.pathTraversalID.values==list(val[0])[0])]
                        new_PtoPTs_rows_tr_exp += list(val[1])[0]
                        #check if we found the right number of PtoPTs to add
                        if len(PtoPTs_toadd)!=list(val[1])[0]:
                            print('hmm... not all VVP trips were found')
                            print(list(val[1])[0], 'expected, ',len(PtoPTs_toadd), 'found' )
                            print('PTs to update: depTime, vehicle, startX,endY,duration, capacity,passengerMiles,occupancy -------->',
                                  key[0],key[1],key[2],key[3],key[4],[list(val[3])[0],list(val[2])[0],list(val[1])[0]])
                            print('Merget Pts -------->',MergedPTs.loc[index[0], ['vehicle','departureTime','capacity','passengerMiles','occupancy','startX']])
                            print('PtoPTs_toadd', PtoPTs_toadd)
                        #PtoPTs update pathTraversalID and add to the merged file
                        # agents = ''
                        if len(PtoPTs_toadd)>0:
                            PtoPTs_toadd['pathTraversalID'] = PtoPTs_toadd['pathTraversalID'].apply(lambda x: index[0])
                            PtoPTs_toappend.append(PtoPTs_toadd)

                            
                            # is_first_agent = True
                            # for person in list(PtoPTs_toadd['personID']):
                            #     if is_first_agent:
                            #         agents = ':%s'%(str(person))
                            #         is_first_agent = False
                            #     else:
                            #         agents += ':%s'%(str(person))
                        MergedPTs.loc[index[0], ['capacity','passengerMiles','occupancy']] += [list(val[3])[0],list(val[2])[0],list(val[1])[0]]
                        # MergedPTs.loc[index[0], ['riders']] += agents


                except:
                    print('mmh...')
                    print([(key[0],key[1],key[2],key[3],key[4])],'not in the merged PTs')

                
            MergedPtoPTs = pd.concat([MergedPtoPTs,pd.concat(PtoPTs_toappend)])
            new_PtoPTs_rows_tr = len(pd.concat(PtoPTs_toappend))


            print('---------------------------------------------------------------')
            #Private modes
            #filter PTs
            PTs_toadd = PTs[(PTs['mode']=='walk')|(PTs['mode']=='car')|(PTs['mode']=='bike')]
            #Update vehicle name
            PTs_toadd.loc[:,'vehicle'] = PTs_toadd['vehicle'].astype(str) + '_' + str(count)
            #Update indexes
            PTs_indexes = list(PTs_toadd.index)
            PTs_toadd.index = PTs_toadd.index+count*1000000000
            #Merge PTs
            MergedPTs = pd.concat([MergedPTs,PTs_toadd])
            #Filter PtoPTs
            PtoPTs_toadd = PtoPTs[PtoPTs['pathTraversalID'].isin(PTs_indexes)]
            #Update vehicle name
            PtoPTs_toadd.loc[:,'vehicleID'] = PtoPTs_toadd['vehicleID'].astype(str) + '_' + str(count)
            #Update indexes
            PtoPTs_toadd.loc[:,'pathTraversalID'] = PtoPTs_toadd['pathTraversalID']+count*1000000000
            #Merge PTs
            MergedPtoPTs = pd.concat([MergedPtoPTs,PtoPTs_toadd])
            
            new_PTs_rows = len(PTs_toadd)
            new_PtoPTs_rows = len(PtoPTs_toadd)
            new_PtoPTs_rows_exp = np.sum(PTs_toadd[PTs_toadd['mode']!='walk']['occupancy'])
        print('---------------------------------------------------------------')
        if count > 0:
            print('len(MergedPTs)',len(MergedPTs))
            print('len(MergedPtoPTs)',len(MergedPtoPTs))
            print('new PtoPTs rows =',len(MergedPtoPTs)-len_initial_PtoPTs)
            print('new PTs rows =',len(MergedPTs)-len_initial_PTs)
            print('expected new PtoPTs rows no transit:',new_PtoPTs_rows_exp)
            print('expected new PtoPTs rows transit:',new_PtoPTs_rows_tr_exp)
            print('new PtoPTs rows no transit:',new_PtoPTs_rows)
            print('new PtoPTs rows transit:',new_PtoPTs_rows_tr)
            print('new PTs rows no transit:',new_PTs_rows)
        count+=1
        print('---------------------------------------------------------------')
    print('Total time:', time.time()-start)
    return MergedPTs, MergedPtoPTs

In [ ]:
PTss = {}
PEVss = {}
PLVss = {}
tripss = {}
Rpss = {}
activitiess = {}
personToTripDepartures = {}
PtoPTss = {}
MCss = {}
EFPs = data.keys()
PFPs = plan_names
FP = fp
for EFP, PFP, name in zip(EFPs, PFPs, names):
    MCs, PTss[name], PEVss[name], PLVss[name], RPs, ASs = readEvents(FP+EFP)
    MCs, PTss[name], PEVss[name], PLVss[name] = fixData(MCs, PTss[name], PEVss[name], PLVss[name], len_id_transit)
    tripss[name], activitiess[name], personToTripDepartures[name], is_leg_mode = processPlans(FP+PFP)
    PtoPTss[name] = personToPathTraversal(PTss[name],
                    PEVss[name], PLVss[name], personToTripDepartures[name])
    

    
    
print('total time', time.time()-start)

In [ ]:
# Filter only Bus companies of interest
# 151_631:t_ 143005 trips
# Long_Islan 178710 trips
# Metro-Nort 105896 trips
# MTA_Bronx_ 420082 trips
# MTA_Brookl 783452 trips
# MTA_Manhat 444432 trips
# MTA_Queens 290973 trips
# MTA_Staten 188297 trips
# NJ_Transit 778050 trips
# NYC_Bus_Co 486766 trips
# NYC_Ferry_ 9810 trips
# NYC_Subway 4430783 trips  NO!
# Staten_Isl 5419 trips
# Suffolk_Co 1043 trips
# WCDOT_2019 138950 trips
PTss_copy = PTss
PtoPTss_copy = PtoPTss
for name in names:
    print('len PTss[name] pre',len(PTss[name]))
    PTss[name] = PTss[name][(PTss[name]['vehicle'].str.contains('151_631:t_'))|
                            (PTss[name]['vehicle'].str.contains('Long_Islan'))|
                            (PTss[name]['vehicle'].str.contains('Metro-Nort'))|
                            (PTss[name]['vehicle'].str.contains('MTA_Bronx_'))|
                            (PTss[name]['vehicle'].str.contains('MTA_Brookl'))|
                            (PTss[name]['vehicle'].str.contains('MTA_Manhat'))|
                            (PTss[name]['vehicle'].str.contains('MTA_Queens'))|
                            (PTss[name]['vehicle'].str.contains('MTA_Staten'))|
                            (PTss[name]['vehicle'].str.contains('NJ_Transit'))|
                            (PTss[name]['vehicle'].str.contains('NYC_Bus_Co'))|
                            (PTss[name]['vehicle'].str.contains('NYC_Ferry_'))|
                            (PTss[name]['vehicle'].str.contains('NYC_Subway'))|
                            (PTss[name]['vehicle'].str.contains('Staten_Isl'))|
                            (PTss[name]['vehicle'].str.contains('WCDOT_2019'))|
                            # (PTss[name]['vehicle'].str.contains('Suffolk_Co'))|
                            # (PTss[name]['vehicle'].str.contains('NICE_117_2'))|
                            # (PTss[name]['vehicle'].str.contains('PATH_20200'))|
                            # (PTss[name]['vehicle'].str.contains('JFK_Airtra'))
                            ]
    print('len PTss[name] post',len(PTss[name]))
    print('len PtoPTss[name] pre',len(PtoPTss[name]))
    PtoPTss[name] = PtoPTss[name][(PtoPTss[name]['vehicleID'].str.contains('151_631:t_'))|
                            (PtoPTss[name]['vehicleID'].str.contains('Long_Islan'))|
                            (PtoPTss[name]['vehicleID'].str.contains('Metro-Nort'))|
                            (PtoPTss[name]['vehicleID'].str.contains('MTA_Bronx_'))|
                            (PtoPTss[name]['vehicleID'].str.contains('MTA_Brookl'))|
                            (PtoPTss[name]['vehicleID'].str.contains('MTA_Manhat'))|
                            (PtoPTss[name]['vehicleID'].str.contains('MTA_Queens'))|
                            (PtoPTss[name]['vehicleID'].str.contains('MTA_Staten'))|
                            (PtoPTss[name]['vehicleID'].str.contains('NJ_Transit'))|
                            (PtoPTss[name]['vehicleID'].str.contains('NYC_Bus_Co'))|
                            (PtoPTss[name]['vehicleID'].str.contains('NYC_Ferry_'))|
                            (PtoPTss[name]['vehicleID'].str.contains('NYC_Subway'))|
                            (PtoPTss[name]['vehicleID'].str.contains('Staten_Isl'))|
                            (PtoPTss[name]['vehicleID'].str.contains('WCDOT_2019'))|
                            # (PtoPTss[name]['vehicleID'].str.contains('Suffolk_Co'))|
                            # (PtoPTss[name]['vehicleID'].str.contains('NICE_117_2'))|
                            # (PtoPTss[name]['vehicleID'].str.contains('PATH_20200'))|
                            # (PtoPTss[name]['vehicleID'].str.contains('JFK_Airtra'))
                            ]
    
    print('len PtoPTss[name] post',len(PtoPTss[name]))


In [ ]:
start = time.time()
transit_agencies = ['NJ_Transit_Rail', 
                    'NJ_Transit_Bus', 
                    '151_631:t_',
                    'Long_Islan',
                    'Metro-Nort',
                    'MTA_Bronx_', 
                    'MTA_Brookl', 
                    'MTA_Manhat', 
                    'MTA_Queens', 
                    'MTA_Staten',
                    'NYC_Bus_Co', 
                    'NYC_Ferry_', 
                    'NYC_Subway', 
                    'Staten_Island_Ferry',
                    #'Suffolk_Co', 
                    'WCDOT_2019']
MergedPTs = []
MergedPtoPTs = []
for transit_agency in transit_agencies: 
    PTss_agency = {}
    PtoPTss_agency= {}
    print('---------------------------------------------------------------')
    print('Analyzing the agency', transit_agency)
    print('---------------------------------------------------------------')
    for name in names:
        PTss_agency[name] = PTss[name][PTss[name]['vehicle'].str.contains(transit_agency)]
        PtoPTss_agency[name] = PtoPTss[name][PtoPTss[name]['vehicleID'].str.contains(transit_agency)]
    MergedPTs_agency, MergedPtoPTs_agency = mergePathTraversals(names, PTss_agency,PtoPTss_agency)
    MergedPTs.append(MergedPTs_agency)
    MergedPtoPTs.append(MergedPtoPTs_agency)

MergedPTs_final = pd.concat(MergedPTs)
MergedPtoPTs_final = pd.concat(MergedPtoPTs)

#add riders
riders = {}
for person, i, pathID  in zip(MergedPtoPTs_final['personID'],range(len(MergedPtoPTs_final.index)),MergedPtoPTs_final['pathTraversalID']):
    if i%1000000 == 0:
        print(i, pathID)
    if pathID in riders:
        riders[pathID].append(person) 
    else:
        riders[pathID] = [person] 

persons = []
for pathID, i in zip(MergedPTs_final.index, range(len(MergedPTs_final.index))):
    if i%1000000 == 0:
        print(i, pathID)
    if pathID in riders:
        persons.append(riders[pathID])
    else:
        persons.append([])
MergedPTs_final['riders']=persons

right = 0
not_right = 0
for rider, occupancy, i in zip(MergedPTs_final['riders'],MergedPTs_final['occupancy'], range(len(MergedPTs_final.index))):
    if i%100000 == 0:
        print(i, pathID)
    # if rider != []:
        # print(len(rider), occupancy)
    if len(rider) == occupancy:
        right +=1
    else:
        not_right+=1
print('right', right,'not_right', not_right)




MergedPtoPTs_final['agency'] = MergedPtoPTs_final['agency'].apply(lambda x: x[:10])
#Update Nj transit agency
agencies = []
for vehicle, agency in zip(MergedPtoPTs_final['vehicleID'],MergedPtoPTs_final['agency']):
    if 'NJ_Transit_Bus' in vehicle:
        agencies.append('NJ_Transit_Bus')
    elif 'NJ_Transit_Rail' in vehicle:
        agencies.append('NJ_Transit_Rail')
    else:
        agencies.append(agency)
MergedPtoPTs_final['agency'] = agencies
print('total time', time.time()-start)

In [ ]:
#check
for agency in transit_agencies:
    print(agency)
    print(int(np.sum(PT_final2[PT_final2['vehicle'].str.contains(agency)]['occupancy'])))
    MergedPtoPTs_final['agency'].value_counts()

In [ ]:
#check
for agency in transit_agencies:
    print(agency)
    print(int(np.sum(PT_final2[PT_final2['vehicle'].str.contains(agency)]['occupancy'])))
    MergedPtoPTs_final['agency'].value_counts()

In [ ]:
name = 'Baseline'
MergedPTs_final.to_csv('outputs/MergedPTs_NYC_'+name+'.csv.gz')
MergedPtoPTs_final.to_csv('outputs/MergedPtoPTs_NYC_'+name+'.csv.gz')

In [ ]:
#check
for agency in transit_agencies:
    print(agency)
    print(int(np.sum(PT_final2[PT_final2['vehicle'].str.contains(agency)]['occupancy'])))
    MergedPtoPTs_final['agency'].value_counts()

In [ ]:
np.sum(PtoPTss['links']=='')

In [ ]:
wrong = 0
modes = []
for links, mode in zip(PtoPTss['links'],PtoPTss['mode']):
    if len(links)==1:
        wrong+=1
        modes.append(mode)
print(wrong)

In [ ]:
len(PtoPTss)

In [ ]:
a=PTs[PTs['links']=='']

In [ ]:
a

In [ ]:
np.unique(modes,return_counts=True)

In [ ]:
        network = pd.read_csv(fp+data_name[:-(len(data_name.split('/')[-1])+len(data_name.split('/')[-2])+len(data_name.split('/')[-3])+2)]+'network.csv.gz')


In [ ]:
network

In [ ]:
events = pd.read_csv('s3://beam-outputs/pilates-outputs/sfbay-rhprice0.5-rhfleet0.5-20220903/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz'
                     ,nrows = 900000)


In [ ]:
events[(events['type']=='ModeChoice')]['tripId']

In [ ]:
pd.read_csv('gs://beam-core-outputs/beam-core-outputs/sfbay-2010-accessibility-fleetpct-0.5-wheelchair-0.5-20220917/beam/year-2012-iteration-3/ITERS/it.0/0.events.csv.gz')
            
            
            
            

In [ ]:
url = (
                'https://opendata.arcgis.com/datasets/'
                'San_Francisco_Bay_Region_2010_Census_Block_Groups.geojson')
gdf = gpd.read_file(url, crs="EPSG:4326")